In [ ]:
# Import dependencies
import os
import zipfile
import errno
import json
import copy
import numpy
import pandas
import matplotlib
from itertools import cycle
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d, spline
from scipy.special import erf
from scipy.integrate import quad
plt.rcParams.update({'font.size': 35})
%matplotlib inline

In [ ]:
# Function defintions used throughout notebook

def get_watts_from_photons(photons, wavelengths):
    h = 6.626070040*10**-34 # Js
    c = 299792458 # m/s
    photon_energy = (h*c)/(wavelengths*10**-9)
    power = photons*photon_energy
    return power

def integrate_spectrum(amplitudes, wavelengths, a, b):
    """Integrate a spectrum from a to b
    :param amplitudes: An array of length N containing amplitudes for a given spectrum
    :param wavelengths: An array of length N containing wavelengths whigh correspond 
        to the amplitudes for a given spectrum
    :param a: The starting wavelength for integration of the spectrum
    :param b: The ending wavelength for integration of the spectrum
    :returns: The area under the spectrum using composite trapezoidal rule
    """
    #Find subset of array where spectrum is between a and b
    #num_measurements = amplitudes.shape[0][0]/amplitudes.shape[0][1]
    spec_roi = amplitudes[numpy.where((wavelengths>=a) & (wavelengths<=b))]
    wv_roi = wavelengths[numpy.where((wavelengths>=a) & (wavelengths<=b))]
    dx = (wv_roi[1] - wv_roi[0])
    area = numpy.trapz(spec_roi, wv_roi, dx=dx)
        
    return area

def gaussian(x, mu, sig):
    exponential = numpy.exp((-numpy.power(x - mu, 2.) / (2 * numpy.power(sig, 2.))))
    return exponential/(sig*numpy.sqrt(2*numpy.pi))

def read_gamos_fluence_file(file):
    # Read output of GAMOS simulations
    new_file = "{}.new".format(file)
    with open(file, "r") as f:
        file_contents = f.read()
        new_index = 0
        i = 0
        while i<5:
            new_index = file_contents.find("\n", new_index+1)
            i +=1
    with open(new_file, "w") as f:
        f.write(file_contents[new_index:])
    df = pandas.read_csv(new_file, delimiter=',', names=['voxel','fluence'])
    df = df[:-1]
    return df

def get_betas(wavelengths, u_a_ppix, beta_630=14):
    # Determine photobleaching constant
    u_a_ppix_630 = u_a_ppix[wavelengths==630][0]
    return beta_630*(u_a_ppix_630/u_a_ppix)*(630/wavelengths)

def get_pro_drug_conc(z, t, m_0=6.0E16, diffusion=0.69E-10):
    # Determine pro-drug (MAL/ALA) concentration at depth
    diffusion_cm2_s = 10000*diffusion
    rate = erf(z/(numpy.sqrt(4*diffusion_cm2_s*t)))
    m_z_t = m_0*(1-rate)
    return m_z_t

def get_ppix_conc_with_mal(t_prime, z, t, m_0=6.0E16, e_p=0.5, tau_ap=8640, tau_p=4680, diffusion=0.69E-10):
    # Determine the PpIX concentration
    constant_scale = (e_p)/tau_ap
    clearance_rate = numpy.exp(-(t-t_prime)/tau_p)
    integrand = constant_scale*(clearance_rate*(get_pro_drug_conc(z, t_prime, m_0, diffusion)))
    return integrand 


def get_umolar_from_particles_ppix(number_particles, volume=0.001, ppix_mol_w=562.6581):
    # Convert number of molecules PpIX to molar
    avogadro_number = 6.022e+23
    moles_ppix = number_particles/avogadro_number
    ppix_mass = moles_ppix*ppix_mol_w
    molarity = ppix_mass/(volume*ppix_mol_w)*10**6
    return molarity
 

In [ ]:
# Define input and output directories
input_dir = '../data/'
output_dir = '../results/'

# Define gemotry of voxels in GAMOS simulations
voxel_size_cm3 = 1*1*.001
scale_tracklength_mm_to_cm = 10
scale_fluence_per_cm2 = 1/(voxel_size_cm3*scale_tracklength_mm_to_cm)

# Source size investigations
The initial simulations assumed a 2.5mm radius cone beam with 0.97 degree divergence located 60mm above the skin surface.
This assumption was used for effiency of the simulation, so all photon events would be tracked within the [X,Y] 1cm$^2$ voxels.
For dayligh-PDT applications the beam size would cover a much larger field, thus resulting in more side scatter into the the treatment area, or voxels.

Additional simulations have been conducted to test the effect of the beam size. 
These simulations looked at a subset of wavelengths with differing beam sizes. 
The beam sizes investigated have a radius of [4.0mm, 5.0mm, 10.0mm, and 25.0mm].


In [ ]:
zip_filemname = 'multiple_source_sizes.zip'
cur_compressed_data_file = os.path.join(input_dir, zip_filemname)

new_beam_size_dir = os.path.splitext(os.path.basename(cur_compressed_data_file))[0]
if not os.path.isdir(cur_compressed_data_file):
    if not os.path.isdir(os.path.join(input_dir,new_beam_size_dir)):
        try:
            zo = zipfile.ZipFile(cur_compressed_data_file)
            os.mkdir(os.path.join(input_dir, new_beam_size_dir))
            zo.extractall(path=os.path.join(input_dir, new_beam_size_dir))
        except (zipfile.BadZipfile):
            print("BAD ZIP: " + cur_compressed_data_file)
            try:
                os.remove(zip_filemname)
            except OSError as e:
                if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
                    raise # re-raise exception if a different error occured  

In [ ]:
# Read GAMOS output fluence files for the new beam sizes
# Only simulated at a sub-set of the original wavelengths
# Listed beam sizes are radius measures at the source
search_term = 'fluence.csv'
search_dir = os.path.join(input_dir, new_beam_size_dir)
all_sub_dirs = os.listdir(search_dir)
sub_dirs_of_interest = [s for s in all_sub_dirs if 'PDT' in s]
beam_sizes = [s.split('_')[2] for s in sub_dirs_of_interest]
beam_size_wvs = [s.split('_')[1][:-2] for s in sub_dirs_of_interest]
unique_sizes = numpy.unique(beam_sizes)
unique_wvs = numpy.unique(beam_size_wvs)
beam_size_fluence = {}
for each_beam_size in unique_sizes:
    beam_size_fluence[each_beam_size] = {}
for each_sub_dir in all_sub_dirs:
    if 'PDT' in each_sub_dir:
        all_files = os.listdir(os.path.join(search_dir, each_sub_dir))
        cur_wv = each_sub_dir.split('_')[1][:-2]
        cur_beam_size = each_sub_dir.split('_')[2]
        # Return file names that match current category
        fluence_files = [s for s in all_files if search_term in s]
        for each_file in fluence_files:
            if '.new' not in each_file:
                wvnm = each_file.split('_')[0]
                print('Reading {}nm {} beam half-size file: {} ...'.format(wvnm, cur_beam_size, each_file))
                beam_size_fluence[cur_beam_size][cur_wv] = read_gamos_fluence_file(os.path.join(search_dir, each_sub_dir, each_file)) 
        

Since the beam source is places 60mm from the skin surface with an angle of divergence of 0.97 degrees, the area of the skin receiving the light dose is slightly larger.

In [ ]:
# 60mm*tan(0.97deg) + original radius
# Then find area in mm2 and divide by 100 to get cm2
# Use this to account for density of photons hitting voxel surface

beam_sizes_str = ['2.5mm', '4.0mm', '5.0mm', '10.0mm', '25.0mm']
beam_sizes_float = [float(s[:-2])for s in beam_sizes_str]
skin_area_cm2 = {}
beam_scale_factor = {}
for i in range(len(beam_sizes_str)):
    beam_spread = 60*numpy.tan(numpy.deg2rad(0.97))
    new_r_mm = beam_sizes_float[i]+beam_spread
    skin_area_mm2 = numpy.pi*new_r_mm**2
    skin_area_cm2[beam_sizes_str[i]] = skin_area_mm2/100
    voxel_surface_area_cm2 = 1
    # The beam scale factor accounts for the number of event photons starting within the voxel surface area
    # For beams with a surface area smaller than the voxel surface area, set the scale factor to 1
    # Otherwise use the beam surface area to find the fraction of events starting withing the voxel space
    beam_scale_factor[beam_sizes_str[i]] = numpy.min([1, voxel_surface_area_cm2/skin_area_cm2[beam_sizes_str[i]]])


In [ ]:
# Look at the 1st 5mm for each beam size
beam_size_fluence_crop = {}
beam_size_fluence_crop['2.5mm'] = numpy.zeros([2000, len(unique_wvs)])
for each_beam_size in unique_sizes:
    beam_size_fluence_crop[each_beam_size] = numpy.zeros([2000, len(unique_wvs)])
# For each of the new beam sizes and the original data find the relative fluence
# This is number of photons be cm2 at each wavelength
for each_beam_size in beam_size_fluence_crop.keys():
    i = 0
    for each_wv in sorted(unique_wvs):
        for each_voxel in beam_size_fluence[each_beam_size][each_wv]['voxel']:
            beam_size_fluence_crop[each_beam_size][1999-int(each_voxel),i] = (1/beam_scale_factor[each_beam_size])*scale_fluence_per_cm2*beam_size_fluence[each_beam_size][each_wv][beam_size_fluence[each_beam_size][each_wv]['voxel']==each_voxel]['fluence']
        i += 1
    beam_size_fluence_crop[each_beam_size] = beam_size_fluence_crop[each_beam_size][0:500, : ]

In [ ]:
cmap = plt.cm.inferno
# extract all colors from the inferno map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_sizes = len(beam_sizes_str)
size_colors = cmap_array[0::int(cmap.N/num_sizes)]
    
i = 0

color_cycler = cycle(size_colors)
# each_wv = sorted(unique_wvs)[i]
#color_cycler = cycle(depth_colors)
plt.figure(figsize=[20,20])
plt.xlabel('Depth (mm)', size=40)
ax1 = plt.gca()
ax1.set_ylim([0.001,10])
ax1.set_xlim([0,5])
ax1.tick_params(labelsize=45)
ax1.set_ylabel('Fluence Rate Relative to Surface', size=40)
ax1.set_yscale('log')
ax1.grid(True)
ax1.grid(True, which='minor', axis='y', ls=':')
for each_beam_size in beam_sizes_str:
    cur_color = next(color_cycler)

    plt.plot(numpy.linspace(10,5000,500)/1000,
             beam_size_fluence_crop[each_beam_size][:, 0],
             lw =8, color=cur_color,
             label='{}'.format(each_beam_size))
    plt.text(2.1,1.1, '630nm', size=50)

    plt.plot(numpy.linspace(10,5000,500)/1000,
             beam_size_fluence_crop[each_beam_size][:, 1],
             lw =8, color=cur_color)
    plt.text(1.3,0.13, '530nm', size=50)
    plt.plot(numpy.linspace(10,5000,500)/1000,
             beam_size_fluence_crop[each_beam_size][:, 2],
             lw =8, color=cur_color)
    plt.text(0.45,0.018, '410nm', size=50)

plt.legend(loc='best', fontsize=40)
plt.show()
i += 1

In [ ]:
cmap = plt.cm.inferno
# extract all colors from the inferno map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_sizes = len(beam_sizes_str)
size_colors = cmap_array[0::int(cmap.N/num_sizes)]
    
i = 0

color_cycler = cycle(size_colors)
# each_wv = sorted(unique_wvs)[i]
#color_cycler = cycle(depth_colors)
plt.figure(figsize=[20,20])
plt.xlabel('Depth (mm)', size=40)
ax1 = plt.gca()
ax1.set_ylim([0.001,2])
ax1.set_xlim([0,5])
ax1.tick_params(labelsize=45)
ax1.set_ylabel('Relative Fluence Rate', size=40)
#ax1.set_yscale('log')
ax1.grid(True)
ax1.grid(True, which='minor', axis='y', ls=':')
for each_beam_size in beam_sizes_str:
    cur_color = next(color_cycler)

#     plt.plot(numpy.linspace(10,5000,500)/1000,
#              beam_size_fluence_crop[each_beam_size][:, 0]/beam_size_fluence_crop['5.0mm'][:, 0],
#              lw =8, color=cur_color,
#              label='{}'.format(each_beam_size))
#     plt.plot(numpy.linspace(10,5000,500)/1000,
#              beam_size_fluence_crop[each_beam_size][:, 1]/beam_size_fluence_crop['5.0mm'][:, 1],
#              lw =8, color=cur_color)
    plt.plot(numpy.linspace(10,5000,500)/1000,
             beam_size_fluence_crop[each_beam_size][:, 2]/beam_size_fluence_crop['5.0mm'][:, 2],
             lw =8, color=cur_color,
             label='{}'.format(each_beam_size))
    plt.text(1.2,1.75, 'Ratio of 25mm:5.0mm fluence rate\nfor 630nm light', size=30)

plt.legend(loc='best', fontsize=30)
plt.show()
i += 1

# 25mm radius source

In [ ]:
cur_beam_size= '25.0mm'
zip_filemname = '25mm_source.zip'

cur_compressed_data_file = os.path.join(input_dir, zip_filemname)

new_dir = os.path.splitext(os.path.basename(cur_compressed_data_file))[0]
if not os.path.isdir(cur_compressed_data_file):
    if not os.path.isdir(os.path.join(input_dir,new_dir)):
        try:
            zo = zipfile.ZipFile(cur_compressed_data_file)
            os.mkdir(os.path.join(input_dir, new_dir))
            zo.extractall(path=os.path.join(input_dir, new_dir))
        except (zipfile.BadZipfile):
            print("BAD ZIP: " + cur_compressed_data_file)
            try:
                os.remove(zip_filemname)
            except OSError as e:
                if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
                    raise # re-raise exception if a different error occured  

In [ ]:
# Read GAMOS output fluence files, one for each input wavelength
all_fluence = {}
search_term = 'fluence.csv'
search_dir = os.path.join(input_dir, new_dir)
all_sub_dirs = os.listdir(search_dir)
for each_sub_dir in all_sub_dirs:
    if ('PDT' in each_sub_dir) & ('.zip' not in each_sub_dir):
        all_files = os.listdir(os.path.join(search_dir, each_sub_dir))
        # Return file names that match current category
        fluence_files = [s for s in all_files if search_term in s]
        for each_file in fluence_files:
            if '.new' not in each_file:
                wvnm = each_file.split('_')[0]
                print('Reading {}nm file: {} ...'.format(wvnm, each_file))
                all_fluence[wvnm] = read_gamos_fluence_file(os.path.join(search_dir, each_sub_dir, each_file)) 
        

In [ ]:
# Combine all fluence values into single array
num_wv = len(all_fluence.keys())
fluence_maps = numpy.zeros([2000, num_wv])
i=0
for each_wvnm in sorted(all_fluence.keys()):
    for each_voxel in all_fluence[each_wvnm]['voxel']:
        fluence_maps[1999-int(each_voxel),i] = (1/beam_scale_factor[each_beam_size])*scale_fluence_per_cm2*all_fluence[each_wvnm][all_fluence[each_wvnm]['voxel']==each_voxel]['fluence']
    i +=1
fluence_maps = numpy.fliplr(fluence_maps)    

In [ ]:
# Crop to the first 5mm of skin
fluence_map_crop = fluence_maps[0:500, :]

In [ ]:
#Display the fluence in a 3D plot
color_map = plt.get_cmap('inferno')
im_shape = numpy.shape(fluence_map_crop)
ny = im_shape[0]
nx = im_shape[1]
y = range(ny)
x = range(nx)
X, Y = numpy.meshgrid(x, y)
fig = plt.figure(figsize=(30,20))
ax = plt.gca(projection='3d')
#)
ax.plot_wireframe(X,Y,fluence_map_crop)
surf = ax.plot_surface(X,Y,fluence_map_crop, cmap=color_map,norm=matplotlib.colors.LogNorm() )
ax.xaxis.set_ticks([0,10,20,30,40,50])
ax.xaxis.set_ticklabels(['900', '800', '700', '600', '500', '400'], fontsize=40)
ax.xaxis.set_label_text('\n\nWavelength (nm)', fontsize=35)
ax.yaxis.set_ticks([0, 200, 400, 600, 800, 1000])
ax.yaxis.set_ticklabels(['0', '2', '4'], fontsize=40)
ax.yaxis.set_label_text('\n\nDepth (mm)', fontsize=35)
ax.tick_params(labelsize=30)

ax.zaxis.set_label_text('\n\nRelative Fluence', fontsize=35)
ax.view_init(45, 45)
# Add a color bar which maps values to colors.
cb = fig.colorbar(surf, shrink=0.6, aspect=6)
cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=40)
plt.savefig(os.path.join(output_dir, 'fluence_surface_plot_not_norm.png'),bbox_inches='tight' )
plt.show()

In [ ]:
# Convert number of photons into watts
power_watts = []
total_photons_1w = []
photons_1w_fluence_map_crop = numpy.zeros(numpy.shape(fluence_map_crop))
watts_fluence_map_crop = numpy.zeros(numpy.shape(fluence_map_crop))
i = 0 
for wv in numpy.linspace(350,900,56):
    cur_input_watts = get_watts_from_photons(10**7,wv)
    power_watts.append(cur_input_watts)
    scale_factor = 1/power_watts[i]
    total_photons_1w.append(scale_factor*10**7)
    photons_1w_fluence_map_crop[:,i] = total_photons_1w[i]*fluence_map_crop[:,i]
    watts_fluence_map_crop[:,i] = get_watts_from_photons(photons_1w_fluence_map_crop[:,i], wv)
    i +=1
#plt.figure(figsize=(5,5))
#plt.plot(numpy.linspace(350,900,56), total_photons_1w)
#plt.show()

In [ ]:
# Read PpIX absorption data
# Data from http://omlc.org/spectra/PhotochemCAD/html/149.html
ppix_file = os.path.join('../data/', 'PpIX_OMLC.txt')
ppix_abs = pandas.read_csv(ppix_file, header=0, delimiter='\t')
# Normalize Molar Extinction values
ppix_abs['spectrum'] = ppix_abs['spectrum'].values/numpy.nanmax(ppix_abs['spectrum'].values)
plt.plot(ppix_abs['wavelength'], ppix_abs['spectrum'])
plt.show()

In [ ]:
# Sub-sample PpIX at wavelengths used in simulations
ppix_wavelengths = ppix_abs['wavelength'].values
sample_points = numpy.linspace(350,900,56)
sampled_ppix = []
for each_point in sample_points:
    if each_point in ppix_wavelengths:
        sampled_ppix.append(ppix_abs[ppix_abs['wavelength']==each_point]['spectrum'].values[0])
        
    else:
        sampled_ppix.append(0)

sampled_ppix = numpy.array(sampled_ppix).clip(0)


In [ ]:
# Sun measurement from Apogee Spectroradiometer
sun_wavelengths = numpy.array([340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820])
sun_spectrum = numpy.array([0.027724879, 0.031817207, 0.031174873, 0.032650799, 0.031278356, 0.03091923, 0.030966295, 0.033347277, 0.033664562, 0.034438562, 0.037946459, 0.036596592, 0.040604768, 0.042604761, 0.043512284, 0.046448761, 0.041288809, 0.039568189, 0.040684499, 0.041657937, 0.043350947, 0.045585113, 0.047779749, 0.04832504, 0.053668762, 0.054341345, 0.057371709, 0.060692719, 0.062433924, 0.062107487, 0.062727039, 0.062240654, 0.058049176, 0.05634784, 0.056733393, 0.056075557, 0.05897765, 0.06493948, 0.067581531, 0.068748993, 0.068568255, 0.064201308, 0.059451523, 0.053066066, 0.053783743, 0.056568679, 0.058637209, 0.0597557, 0.063528508, 0.067421341, 0.073218991, 0.072343985, 0.067272348, 0.062429229, 0.06460007, 0.063763541, 0.065027185, 0.070229267, 0.07897529, 0.089531739, 0.105482207, 0.108602038, 0.11137408, 0.111926389, 0.112150571, 0.110654353, 0.108858251, 0.110124636, 0.111758036, 0.111730398, 0.112537992, 0.114736572, 0.117539401, 0.119468603, 0.119354842, 0.121009961, 0.121418471, 0.121171238, 0.117096515, 0.117718097, 0.119185945, 0.119877332, 0.119549273, 0.119662236, 0.117004348, 0.116849511, 0.116310644, 0.115068295, 0.112688056, 0.107799048, 0.101925239, 0.106665801, 0.113272239, 0.116766192, 0.120426581, 0.123316207, 0.128008548, 0.125099922, 0.125181384, 0.125167768, 0.128374733, 0.132594001, 0.136423912, 0.139603206, 0.139358934, 0.139110462, 0.140767111, 0.143298166, 0.146552993, 0.149498122, 0.152962143, 0.152230602, 0.150267796, 0.148024811, 0.147267257, 0.149793209, 0.151028991, 0.151494772, 0.150500166, 0.151490978, 0.152695425, 0.154374534, 0.156443441, 0.156306933, 0.152422209, 0.150221759, 0.150093591, 0.149904346, 0.150185822, 0.150066944, 0.149766524, 0.150734097, 0.152238503, 0.153522225, 0.154071434, 0.154078934, 0.154750931, 0.155751424, 0.15681209, 0.15704516, 0.15801784, 0.158235963, 0.157914288, 0.156710154, 0.153865838, 0.147370492, 0.137290594, 0.14041007, 0.145605767, 0.149506243, 0.15136645, 0.150326271, 0.149705017, 0.149535082, 0.152016654, 0.153188947, 0.153029486, 0.151866484, 0.150929138, 0.149420107, 0.147915462, 0.14589862, 0.144483446, 0.145999958, 0.147371677, 0.149458006, 0.150546677, 0.14974658, 0.148900834, 0.148682767, 0.149258706, 0.148450903, 0.147803995, 0.145463629, 0.144609694, 0.14317834, 0.139074679, 0.135125064, 0.134642209, 0.138372651, 0.142162848, 0.143566559, 0.144125949, 0.145147672, 0.146420829, 0.146905905, 0.139384924, 0.140090223, 0.144327844, 0.147484699, 0.148960477, 0.148300796, 0.146462031, 0.14501906, 0.144513403, 0.146533946, 0.146417394, 0.145524448, 0.144868891, 0.143359048, 0.141606578, 0.141266841, 0.142199429, 0.14402267, 0.14368553, 0.144100259, 0.14362828, 0.143182341, 0.143462716, 0.143686197, 0.144120418, 0.143866446, 0.143771169, 0.14381966, 0.143723247, 0.143413507, 0.141965585, 0.14001959, 0.138762077, 0.138324479, 0.139095985, 0.139930931, 0.140678691, 0.140352775, 0.139122291, 0.138518504, 0.138529378, 0.138621789, 0.138507105, 0.137488494, 0.136158924, 0.136464271, 0.13761553, 0.139321579, 0.138085552, 0.137127432, 0.13703801, 0.136723267, 0.136346754, 0.136242223, 0.137414838, 0.138545004, 0.139384545, 0.140133991, 0.13814509, 0.137283548, 0.136591855, 0.135098488, 0.130953206, 0.128055118, 0.128187438, 0.131309408, 0.133638567, 0.13350157, 0.133328321, 0.132920612, 0.133517075, 0.133245301, 0.132927058, 0.132642099, 0.132557764, 0.13194886, 0.132056528, 0.133278152, 0.134262006, 0.134531495, 0.13410411, 0.132582903, 0.132030028, 0.131309072, 0.131177463, 0.130794279, 0.130624148, 0.129659866, 0.12928047, 0.128526801, 0.127744634, 0.127743073, 0.128733615, 0.129989516, 0.130489492, 0.129811661, 0.128747768, 0.128096023, 0.127054418, 0.126775972, 0.126935781, 0.126613227, 0.12625312, 0.125586518, 0.125711141, 0.125306709, 0.126021587, 0.12657216, 0.126554856, 0.12686553, 0.127230342, 0.127611341, 0.127960905, 0.127529323, 0.126534642, 0.12623821, 0.126375478, 0.126533443, 0.126557216, 0.126456169, 0.125496527, 0.123891748, 0.122594232, 0.122878293, 0.123302754, 0.124324386, 0.124925665, 0.124720318, 0.123294072, 0.11782331, 0.113498435, 0.113527708, 0.11895288, 0.123025415, 0.124620732, 0.125680145, 0.125432346, 0.125360222, 0.125372243, 0.125572055, 0.125268315, 0.125146197, 0.125165898, 0.125413991, 0.12486676, 0.124288119, 0.123821936, 0.124041862, 0.123923424, 0.12378219, 0.123776159, 0.123914665, 0.12396378, 0.123444432, 0.123135093, 0.122866514, 0.122669714, 0.121894595, 0.121071917, 0.118176693, 0.111214476, 0.104709686, 0.102117782, 0.104979772, 0.107642698, 0.110398656, 0.112324928, 0.113002859, 0.112882412, 0.113277832, 0.114158783, 0.115553807, 0.11454491, 0.113691449, 0.112631924, 0.111989513, 0.112344112, 0.113251491, 0.114055712, 0.114478336, 0.114733397, 0.115067686, 0.11495552, 0.11503771, 0.115198511, 0.115270568, 0.114685437, 0.114076356, 0.113396539, 0.112016975, 0.108521583, 0.102738315, 0.09658367, 0.091519166, 0.094613599, 0.099178988, 0.100848189, 0.098921368, 0.096371608, 0.096044598, 0.096546224, 0.096758138, 0.096584139, 0.096696971, 0.097642932, 0.099591958, 0.102222716, 0.105222119, 0.107322435, 0.107910671, 0.108197089, 0.108146054, 0.108077546, 0.107982798, 0.10788053, 0.10766728, 0.107718748, 0.108142488, 0.108719951, 0.109132743, 0.108945601, 0.109158752, 0.108486437, 0.107705391, 0.107638674, 0.107481841, 0.107452992, 0.107635392, 0.107835963, 0.107647248, 0.107279845, 0.10531009, 0.096943834, 0.080183507, 0.061237921, 0.050227727, 0.049746502, 0.054087537, 0.059350211, 0.06723244, 0.076874918, 0.08629937, 0.093326192, 0.098198425, 0.100507157, 0.101450607, 0.101975643, 0.102127171, 0.102318676, 0.102224754, 0.102120742, 0.102090152, 0.101914348, 0.101806723, 0.101541314, 0.101692341, 0.101708472, 0.10143401, 0.101051473, 0.10048329, 0.099592014, 0.098346836, 0.097689685, 0.09733467, 0.096962738, 0.096594597, 0.09560491, 0.094941579, 0.094677301, 0.094848156, 0.096024049, 0.096149273, 0.09621305, 0.096216459, 0.095530505, 0.095131273, 0.094697779, 0.094651901, 0.09415254, 0.094140603, 0.094497063, 0.094031348, 0.093344445, 0.092454356, 0.091711977, 0.090530882, 0.088421029, 0.085934576, 0.082321187, 0.078860549, 0.076553088, 0.075962439, 0.075741367, 0.076976657, 0.078561714])
plt.plot(sun_wavelengths,sun_spectrum)
plt.show()

In [ ]:
# Sub-sample Sun at wavelengths used in simulations
sampled_sun = []
wavelengths = numpy.linspace(350,900,56)
for each_point in sample_points:
    if each_point in sun_wavelengths:
        sampled_sun.append(sun_spectrum[numpy.where(sun_wavelengths==each_point)[0][0]])
    else:
        sampled_sun.append(0)
# Reduce irradiance to match simulation (35mW/cm^2)
sampled_sun = 0.85*numpy.array(sampled_sun).clip(0)
plt.plot(wavelengths,sampled_sun)
plt.show()

In [ ]:
# CFL and Halogen measurement from Apogee Spectroradiometer
input_wavelengths = [346.205, 346.421, 346.638, 346.854, 347.07, 347.286, 347.502, 347.719, 347.935, 348.151, 348.367, 348.583, 348.799, 349.015, 349.231, 349.447, 349.663, 349.879, 350.095, 350.311, 350.527, 350.743, 350.959, 351.175, 351.391, 351.607, 351.823, 352.039, 352.255, 352.471, 352.687, 352.903, 353.119, 353.335, 353.55, 353.766, 353.982, 354.198, 354.414, 354.629, 354.845, 355.061, 355.277, 355.492, 355.708, 355.924, 356.14, 356.355, 356.571, 356.787, 357.002, 357.218, 357.434, 357.649, 357.865, 358.081, 358.296, 358.512, 358.727, 358.943, 359.158, 359.374, 359.59, 359.805, 360.021, 360.236, 360.452, 360.667, 360.882, 361.098, 361.313, 361.529, 361.744, 361.96, 362.175, 362.39, 362.606, 362.821, 363.037, 363.252, 363.467, 363.683, 363.898, 364.113, 364.328, 364.544, 364.759, 364.974, 365.189, 365.405, 365.62, 365.835, 366.05, 366.265, 366.481, 366.696, 366.911, 367.126, 367.341, 367.556, 367.771, 367.986, 368.202, 368.417, 368.632, 368.847, 369.062, 369.277, 369.492, 369.707, 369.922, 370.137, 370.352, 370.567, 370.782, 370.997, 371.211, 371.426, 371.641, 371.856, 372.071, 372.286, 372.501, 372.716, 372.93, 373.145, 373.36, 373.575, 373.79, 374.004, 374.219, 374.434, 374.649, 374.863, 375.078, 375.293, 375.507, 375.722, 375.937, 376.151, 376.366, 376.581, 376.795, 377.01, 377.224, 377.439, 377.654, 377.868, 378.083, 378.297, 378.512, 378.726, 378.941, 379.155, 379.37, 379.584, 379.799, 380.013, 380.228, 380.442, 380.656, 380.871, 381.085, 381.3, 381.514, 381.728, 381.943, 382.157, 382.371, 382.586, 382.8, 383.014, 383.228, 383.443, 383.657, 383.871, 384.085, 384.3, 384.514, 384.728, 384.942, 385.156, 385.371, 385.585, 385.799, 386.013, 386.227, 386.441, 386.655, 386.869, 387.083, 387.297, 387.511, 387.726, 387.94, 388.154, 388.368, 388.582, 388.795, 389.009, 389.223, 389.437, 389.651, 389.865, 390.079, 390.293, 390.507, 390.721, 390.935, 391.148, 391.362, 391.576, 391.79, 392.004, 392.217, 392.431, 392.645, 392.859, 393.072, 393.286, 393.5, 393.714, 393.927, 394.141, 394.355, 394.568, 394.782, 394.996, 395.209, 395.423, 395.636, 395.85, 396.064, 396.277, 396.491, 396.704, 396.918, 397.131, 397.345, 397.558, 397.772, 397.985, 398.199, 398.412, 398.626, 398.839, 399.052, 399.266, 399.479, 399.692, 399.906, 400.119, 400.333, 400.546, 400.759, 400.972, 401.186, 401.399, 401.612, 401.826, 402.039, 402.252, 402.465, 402.678, 402.892, 403.105, 403.318, 403.531, 403.744, 403.957, 404.171, 404.384, 404.597, 404.81, 405.023, 405.236, 405.449, 405.662, 405.875, 406.088, 406.301, 406.514, 406.727, 406.94, 407.153, 407.366, 407.579, 407.792, 408.005, 408.218, 408.431, 408.644, 408.856, 409.069, 409.282, 409.495, 409.708, 409.921, 410.133, 410.346, 410.559, 410.772, 410.984, 411.197, 411.41, 411.623, 411.835, 412.048, 412.261, 412.473, 412.686, 412.899, 413.111, 413.324, 413.536, 413.749, 413.962, 414.174, 414.387, 414.599, 414.812, 415.024, 415.237, 415.449, 415.662, 415.874, 416.087, 416.299, 416.512, 416.724, 416.936, 417.149, 417.361, 417.574, 417.786, 417.998, 418.211, 418.423, 418.635, 418.848, 419.06, 419.272, 419.484, 419.697, 419.909, 420.121, 420.333, 420.546, 420.758, 420.97, 421.182, 421.394, 421.606, 421.819, 422.031, 422.243, 422.455, 422.667, 422.879, 423.091, 423.303, 423.515, 423.727, 423.939, 424.151, 424.363, 424.575, 424.787, 424.999, 425.211, 425.423, 425.635, 425.847, 426.059, 426.271, 426.482, 426.694, 426.906, 427.118, 427.33, 427.542, 427.753, 427.965, 428.177, 428.389, 428.601, 428.812, 429.024, 429.236, 429.447, 429.659, 429.871, 430.082, 430.294, 430.506, 430.717, 430.929, 431.141, 431.352, 431.564, 431.775, 431.987, 432.198, 432.41, 432.621, 432.833, 433.044, 433.256, 433.467, 433.679, 433.89, 434.102, 434.313, 434.525, 434.736, 434.947, 435.159, 435.37, 435.581, 435.793, 436.004, 436.215, 436.427, 436.638, 436.849, 437.061, 437.272, 437.483, 437.694, 437.905, 438.117, 438.328, 438.539, 438.75, 438.961, 439.173, 439.384, 439.595, 439.806, 440.017, 440.228, 440.439, 440.65, 440.861, 441.072, 441.283, 441.494, 441.705, 441.916, 442.127, 442.338, 442.549, 442.76, 442.971, 443.182, 443.393, 443.604, 443.815, 444.025, 444.236, 444.447, 444.658, 444.869, 445.079, 445.29, 445.501, 445.712, 445.923, 446.133, 446.344, 446.555, 446.765, 446.976, 447.187, 447.397, 447.608, 447.819, 448.029, 448.24, 448.451, 448.661, 448.872, 449.082, 449.293, 449.503, 449.714, 449.924, 450.135, 450.345, 450.556, 450.766, 450.977, 451.187, 451.398, 451.608, 451.819, 452.029, 452.239, 452.45, 452.66, 452.87, 453.081, 453.291, 453.501, 453.712, 453.922, 454.132, 454.342, 454.553, 454.763, 454.973, 455.183, 455.393, 455.604, 455.814, 456.024, 456.234, 456.444, 456.654, 456.864, 457.075, 457.285, 457.495, 457.705, 457.915, 458.125, 458.335, 458.545, 458.755, 458.965, 459.175, 459.385, 459.595, 459.805, 460.015, 460.224, 460.434, 460.644, 460.854, 461.064, 461.274, 461.484, 461.693, 461.903, 462.113, 462.323, 462.533, 462.742, 462.952, 463.162, 463.372, 463.581, 463.791, 464.001, 464.21, 464.42, 464.63, 464.839, 465.049, 465.258, 465.468, 465.678, 465.887, 466.097, 466.306, 466.516, 466.725, 466.935, 467.144, 467.354, 467.563, 467.773, 467.982, 468.192, 468.401, 468.61, 468.82, 469.029, 469.239, 469.448, 469.657, 469.867, 470.076, 470.285, 470.495, 470.704, 470.913, 471.122, 471.332, 471.541, 471.75, 471.959, 472.168, 472.378, 472.587, 472.796, 473.005, 473.214, 473.423, 473.632, 473.841, 474.051, 474.26, 474.469, 474.678, 474.887, 475.096, 475.305, 475.514, 475.723, 475.932, 476.141, 476.35, 476.558, 476.767, 476.976, 477.185, 477.394, 477.603, 477.812, 478.021, 478.229, 478.438, 478.647, 478.856, 479.065, 479.273, 479.482, 479.691, 479.9, 480.108, 480.317, 480.526, 480.734, 480.943, 481.152, 481.36, 481.569, 481.778, 481.986, 482.195, 482.403, 482.612, 482.82, 483.029, 483.237, 483.446, 483.654, 483.863, 484.071, 484.28, 484.488, 484.697, 484.905, 485.114, 485.322, 485.53, 485.739, 485.947, 486.155, 486.364, 486.572, 486.78, 486.989, 487.197, 487.405, 487.613, 487.822, 488.03, 488.238, 488.446, 488.654, 488.863, 489.071, 489.279, 489.487, 489.695, 489.903, 490.111, 490.319, 490.528, 490.736, 490.944, 491.152, 491.36, 491.568, 491.776, 491.984, 492.192, 492.4, 492.608, 492.815, 493.023, 493.231, 493.439, 493.647, 493.855, 494.063, 494.271, 494.478, 494.686, 494.894, 495.102, 495.31, 495.517, 495.725, 495.933, 496.141, 496.348, 496.556, 496.764, 496.971, 497.179, 497.387, 497.594, 497.802, 498.01, 498.217, 498.425, 498.632, 498.84, 499.047, 499.255, 499.462, 499.67, 499.877, 500.085, 500.292, 500.5, 500.707, 500.915, 501.122, 501.33, 501.537, 501.744, 501.952, 502.159, 502.366, 502.574, 502.781, 502.988, 503.196, 503.403, 503.61, 503.817, 504.025, 504.232, 504.439, 504.646, 504.853, 505.061, 505.268, 505.475, 505.682, 505.889, 506.096, 506.303, 506.51, 506.717, 506.924, 507.132, 507.339, 507.546, 507.753, 507.96, 508.167, 508.373, 508.58, 508.787, 508.994, 509.201, 509.408, 509.615, 509.822, 510.029, 510.236, 510.442, 510.649, 510.856, 511.063, 511.27, 511.476, 511.683, 511.89, 512.097, 512.303, 512.51, 512.717, 512.923, 513.13, 513.337, 513.543, 513.75, 513.956, 514.163, 514.37, 514.576, 514.783, 514.989, 515.196, 515.402, 515.609, 515.815, 516.022, 516.228, 516.435, 516.641, 516.848, 517.054, 517.26, 517.467, 517.673, 517.879, 518.086, 518.292, 518.498, 518.705, 518.911, 519.117, 519.324, 519.53, 519.736, 519.942, 520.149, 520.355, 520.561, 520.767, 520.973, 521.179, 521.386, 521.592, 521.798, 522.004, 522.21, 522.416, 522.622, 522.828, 523.034, 523.24, 523.446, 523.652, 523.858, 524.064, 524.27, 524.476, 524.682, 524.888, 525.094, 525.3, 525.505, 525.711, 525.917, 526.123, 526.329, 526.535, 526.74, 526.946, 527.152, 527.358, 527.563, 527.769, 527.975, 528.181, 528.386, 528.592, 528.798, 529.003, 529.209, 529.414, 529.62, 529.826, 530.031, 530.237, 530.442, 530.648, 530.853, 531.059, 531.264, 531.47, 531.675, 531.881, 532.086, 532.292, 532.497, 532.702, 532.908, 533.113, 533.319, 533.524, 533.729, 533.935, 534.14, 534.345, 534.551, 534.756, 534.961, 535.166, 535.372, 535.577, 535.782, 535.987, 536.192, 536.397, 536.603, 536.808, 537.013, 537.218, 537.423, 537.628, 537.833, 538.038, 538.243, 538.448, 538.653, 538.858, 539.063, 539.268, 539.473, 539.678, 539.883, 540.088, 540.293, 540.498, 540.703, 540.908, 541.113, 541.317, 541.522, 541.727, 541.932, 542.137, 542.341, 542.546, 542.751, 542.956, 543.16, 543.365, 543.57, 543.774, 543.979, 544.184, 544.388, 544.593, 544.798, 545.002, 545.207, 545.411, 545.616, 545.821, 546.025, 546.23, 546.434, 546.639, 546.843, 547.048, 547.252, 547.456, 547.661, 547.865, 548.07, 548.274, 548.478, 548.683, 548.887, 549.091, 549.296, 549.5, 549.704, 549.909, 550.113, 550.317, 550.521, 550.726, 550.93, 551.134, 551.338, 551.542, 551.747, 551.951, 552.155, 552.359, 552.563, 552.767, 552.971, 553.175, 553.379, 553.583, 553.787, 553.991, 554.195, 554.399, 554.603, 554.807, 555.011, 555.215, 555.419, 555.623, 555.827, 556.031, 556.234, 556.438, 556.642, 556.846, 557.05, 557.254, 557.457, 557.661, 557.865, 558.069, 558.272, 558.476, 558.68, 558.883, 559.087, 559.291, 559.494, 559.698, 559.902, 560.105, 560.309, 560.512, 560.716, 560.919, 561.123, 561.326, 561.53, 561.733, 561.937, 562.14, 562.344, 562.547, 562.751, 562.954, 563.157, 563.361, 563.564, 563.768, 563.971, 564.174, 564.378, 564.581, 564.784, 564.987, 565.191, 565.394, 565.597, 565.8, 566.004, 566.207, 566.41, 566.613, 566.816, 567.019, 567.222, 567.426, 567.629, 567.832, 568.035, 568.238, 568.441, 568.644, 568.847, 569.05, 569.253, 569.456, 569.659, 569.862, 570.065, 570.268, 570.47, 570.673, 570.876, 571.079, 571.282, 571.485, 571.688, 571.89, 572.093, 572.296, 572.499, 572.701, 572.904, 573.107, 573.31, 573.512, 573.715, 573.918, 574.12, 574.323, 574.526, 574.728, 574.931, 575.133, 575.336, 575.538, 575.741, 575.943, 576.146, 576.348, 576.551, 576.753, 576.956, 577.158, 577.361, 577.563, 577.766, 577.968, 578.17, 578.373, 578.575, 578.777, 578.98, 579.182, 579.384, 579.587, 579.789, 579.991, 580.193, 580.396, 580.598, 580.8, 581.002, 581.204, 581.406, 581.609, 581.811, 582.013, 582.215, 582.417, 582.619, 582.821, 583.023, 583.225, 583.427, 583.629, 583.831, 584.033, 584.235, 584.437, 584.639, 584.841, 585.043, 585.245, 585.446, 585.648, 585.85, 586.052, 586.254, 586.456, 586.657, 586.859, 587.061, 587.263, 587.464, 587.666, 587.868, 588.07, 588.271, 588.473, 588.675, 588.876, 589.078, 589.279, 589.481, 589.683, 589.884, 590.086, 590.287, 590.489, 590.69, 590.892, 591.093, 591.295, 591.496, 591.698, 591.899, 592.1, 592.302, 592.503, 592.705, 592.906, 593.107, 593.309, 593.51, 593.711, 593.912, 594.114, 594.315, 594.516, 594.717, 594.919, 595.12, 595.321, 595.522, 595.723, 595.924, 596.126, 596.327, 596.528, 596.729, 596.93, 597.131, 597.332, 597.533, 597.734, 597.935, 598.136, 598.337, 598.538, 598.739, 598.94, 599.141, 599.342, 599.543, 599.744, 599.944, 600.145, 600.346, 600.547, 600.748, 600.948, 601.149, 601.35, 601.551, 601.751, 601.952, 602.153, 602.354, 602.554, 602.755, 602.956, 603.156, 603.357, 603.557, 603.758, 603.959, 604.159, 604.36, 604.56, 604.761, 604.961, 605.162, 605.362, 605.563, 605.763, 605.964, 606.164, 606.364, 606.565, 606.765, 606.966, 607.166, 607.366, 607.567, 607.767, 607.967, 608.167, 608.368, 608.568, 608.768, 608.968, 609.169, 609.369, 609.569, 609.769, 609.969, 610.169, 610.37, 610.57, 610.77, 610.97, 611.17, 611.37, 611.57, 611.77, 611.97, 612.17, 612.37, 612.57, 612.77, 612.97, 613.17, 613.37, 613.57, 613.769, 613.969, 614.169, 614.369, 614.569, 614.769, 614.968, 615.168, 615.368, 615.568, 615.767, 615.967, 616.167, 616.367, 616.566, 616.766, 616.966, 617.165, 617.365, 617.564, 617.764, 617.964, 618.163, 618.363, 618.562, 618.762, 618.961, 619.161, 619.36, 619.56, 619.759, 619.959, 620.158, 620.357, 620.557, 620.756, 620.956, 621.155, 621.354, 621.554, 621.753, 621.952, 622.151, 622.351, 622.55, 622.749, 622.948, 623.148, 623.347, 623.546, 623.745, 623.944, 624.143, 624.343, 624.542, 624.741, 624.94, 625.139, 625.338, 625.537, 625.736, 625.935, 626.134, 626.333, 626.532, 626.731, 626.93, 627.129, 627.328, 627.526, 627.725, 627.924, 628.123, 628.322, 628.521, 628.719, 628.918, 629.117, 629.316, 629.515, 629.713, 629.912, 630.111, 630.309, 630.508, 630.707, 630.905, 631.104, 631.303, 631.501, 631.7, 631.898, 632.097, 632.295, 632.494, 632.692, 632.891, 633.089, 633.288, 633.486, 633.685, 633.883, 634.082, 634.28, 634.478, 634.677, 634.875, 635.073, 635.272, 635.47, 635.668, 635.867, 636.065, 636.263, 636.461, 636.66, 636.858, 637.056, 637.254, 637.452, 637.65, 637.849, 638.047, 638.245, 638.443, 638.641, 638.839, 639.037, 639.235, 639.433, 639.631, 639.829, 640.027, 640.225, 640.423, 640.621, 640.819, 641.017, 641.215, 641.412, 641.61, 641.808, 642.006, 642.204, 642.402, 642.599, 642.797, 642.995, 643.193, 643.39, 643.588, 643.786, 643.983, 644.181, 644.379, 644.576, 644.774, 644.972, 645.169, 645.367, 645.564, 645.762, 645.959, 646.157, 646.354, 646.552, 646.749, 646.947, 647.144, 647.342, 647.539, 647.736, 647.934, 648.131, 648.329, 648.526, 648.723, 648.921, 649.118, 649.315, 649.512, 649.71, 649.907, 650.104, 650.301, 650.499, 650.696, 650.893, 651.09, 651.287, 651.484, 651.681, 651.878, 652.076, 652.273, 652.47, 652.667, 652.864, 653.061, 653.258, 653.455, 653.652, 653.849, 654.045, 654.242, 654.439, 654.636, 654.833, 655.03, 655.227, 655.423, 655.62, 655.817, 656.014, 656.211, 656.407, 656.604, 656.801, 656.998, 657.194, 657.391, 657.588, 657.784, 657.981, 658.177, 658.374, 658.571, 658.767, 658.964, 659.16, 659.357, 659.553, 659.75, 659.946, 660.143, 660.339, 660.536, 660.732, 660.928, 661.125, 661.321, 661.518, 661.714, 661.91, 662.107, 662.303, 662.499, 662.695, 662.892, 663.088, 663.284, 663.48, 663.677, 663.873, 664.069, 664.265, 664.461, 664.657, 664.854, 665.05, 665.246, 665.442, 665.638, 665.834, 666.03, 666.226, 666.422, 666.618, 666.814, 667.01, 667.206, 667.402, 667.597, 667.793, 667.989, 668.185, 668.381, 668.577, 668.773, 668.968, 669.164, 669.36, 669.556, 669.751, 669.947, 670.143, 670.338, 670.534, 670.73, 670.925, 671.121, 671.317, 671.512, 671.708, 671.903, 672.099, 672.295, 672.49, 672.686, 672.881, 673.077, 673.272, 673.467, 673.663, 673.858, 674.054, 674.249, 674.444, 674.64, 674.835, 675.031, 675.226, 675.421, 675.616, 675.812, 676.007, 676.202, 676.397, 676.593, 676.788, 676.983, 677.178, 677.373, 677.568, 677.764, 677.959, 678.154, 678.349, 678.544, 678.739, 678.934, 679.129, 679.324, 679.519, 679.714, 679.909, 680.104, 680.299, 680.494, 680.688, 680.883, 681.078, 681.273, 681.468, 681.663, 681.858, 682.052, 682.247, 682.442, 682.637, 682.831, 683.026, 683.221, 683.415, 683.61, 683.805, 683.999, 684.194, 684.389, 684.583, 684.778, 684.972, 685.167, 685.361, 685.556, 685.75, 685.945, 686.139, 686.334, 686.528, 686.723, 686.917, 687.111, 687.306, 687.5, 687.694, 687.889, 688.083, 688.277, 688.472, 688.666, 688.86, 689.054, 689.249, 689.443, 689.637, 689.831, 690.025, 690.22, 690.414, 690.608, 690.802, 690.996, 691.19, 691.384, 691.578, 691.772, 691.966, 692.16, 692.354, 692.548, 692.742, 692.936, 693.13, 693.324, 693.518, 693.712, 693.905, 694.099, 694.293, 694.487, 694.681, 694.875, 695.068, 695.262, 695.456, 695.649, 695.843, 696.037, 696.231, 696.424, 696.618, 696.811, 697.005, 697.199, 697.392, 697.586, 697.779, 697.973, 698.166, 698.36, 698.553, 698.747, 698.94, 699.134, 699.327, 699.521, 699.714, 699.907, 700.101, 700.294, 700.487, 700.681, 700.874, 701.067, 701.261, 701.454, 701.647, 701.84, 702.034, 702.227, 702.42, 702.613, 702.806, 702.999, 703.193, 703.386, 703.579, 703.772, 703.965, 704.158, 704.351, 704.544, 704.737, 704.93, 705.123, 705.316, 705.509, 705.702, 705.895, 706.087, 706.28, 706.473, 706.666, 706.859, 707.052, 707.244, 707.437, 707.63, 707.823, 708.016, 708.208, 708.401, 708.594, 708.786, 708.979, 709.172, 709.364, 709.557, 709.749, 709.942, 710.135, 710.327, 710.52, 710.712, 710.905, 711.097, 711.29, 711.482, 711.675, 711.867, 712.059, 712.252, 712.444, 712.636, 712.829, 713.021, 713.213, 713.406, 713.598, 713.79, 713.983, 714.175, 714.367, 714.559, 714.751, 714.944, 715.136, 715.328, 715.52, 715.712, 715.904, 716.096, 716.288, 716.48, 716.672, 716.865, 717.057, 717.249, 717.44, 717.632, 717.824, 718.016, 718.208, 718.4, 718.592, 718.784, 718.976, 719.168, 719.359, 719.551, 719.743, 719.935, 720.126, 720.318, 720.51, 720.702, 720.893, 721.085, 721.277, 721.468, 721.66, 721.852, 722.043, 722.235, 722.426, 722.618, 722.809, 723.001, 723.192, 723.384, 723.575, 723.767, 723.958, 724.15, 724.341, 724.532, 724.724, 724.915, 725.107, 725.298, 725.489, 725.681, 725.872, 726.063, 726.254, 726.446, 726.637, 726.828, 727.019, 727.21, 727.402, 727.593, 727.784, 727.975, 728.166, 728.357, 728.548, 728.739, 728.93, 729.121, 729.312, 729.503, 729.694, 729.885, 730.076, 730.267, 730.458, 730.649, 730.84, 731.031, 731.221, 731.412, 731.603, 731.794, 731.985, 732.175, 732.366, 732.557, 732.748, 732.938, 733.129, 733.32, 733.51, 733.701, 733.892, 734.082, 734.273, 734.463, 734.654, 734.844, 735.035, 735.225, 735.416, 735.606, 735.797, 735.987, 736.178, 736.368, 736.559, 736.749, 736.939, 737.13, 737.32, 737.51, 737.701, 737.891, 738.081, 738.271, 738.462, 738.652, 738.842, 739.032, 739.223, 739.413, 739.603, 739.793, 739.983, 740.173, 740.363, 740.553, 740.743, 740.933, 741.123, 741.313, 741.503, 741.693, 741.883, 742.073, 742.263, 742.453, 742.643, 742.833, 743.023, 743.213, 743.402, 743.592, 743.782, 743.972, 744.162, 744.351, 744.541, 744.731, 744.921, 745.11, 745.3, 745.49, 745.679, 745.869, 746.058, 746.248, 746.438, 746.627, 746.817, 747.006, 747.196, 747.385, 747.575, 747.764, 747.954, 748.143, 748.332, 748.522, 748.711, 748.901, 749.09, 749.279, 749.469, 749.658, 749.847, 750.036, 750.226, 750.415, 750.604, 750.793, 750.983, 751.172, 751.361, 751.55, 751.739, 751.928, 752.117, 752.306, 752.496, 752.685, 752.874, 753.063, 753.252, 753.441, 753.63, 753.819, 754.007, 754.196, 754.385, 754.574, 754.763, 754.952, 755.141, 755.33, 755.518, 755.707, 755.896, 756.085, 756.273, 756.462, 756.651, 756.84, 757.028, 757.217, 757.406, 757.594, 757.783, 757.971, 758.16, 758.348, 758.537, 758.726, 758.914, 759.103, 759.291, 759.48, 759.668, 759.856, 760.045, 760.233, 760.422, 760.61, 760.798, 760.987, 761.175, 761.363, 761.552, 761.74, 761.928, 762.116, 762.305, 762.493, 762.681, 762.869, 763.057, 763.245, 763.434, 763.622, 763.81, 763.998, 764.186, 764.374, 764.562, 764.75, 764.938, 765.126, 765.314, 765.502, 765.69, 765.878, 766.066, 766.254, 766.441, 766.629, 766.817, 767.005, 767.193, 767.381, 767.568, 767.756, 767.944, 768.132, 768.319, 768.507, 768.695, 768.882, 769.07, 769.258, 769.445, 769.633, 769.82, 770.008, 770.195, 770.383, 770.57, 770.758, 770.945, 771.133, 771.32, 771.508, 771.695, 771.883, 772.07, 772.257, 772.445, 772.632, 772.819, 773.007, 773.194, 773.381, 773.568, 773.756, 773.943, 774.13, 774.317, 774.504, 774.692, 774.879, 775.066, 775.253, 775.44, 775.627, 775.814, 776.001, 776.188, 776.375, 776.562, 776.749, 776.936, 777.123, 777.31, 777.497, 777.684, 777.871, 778.058, 778.244, 778.431, 778.618, 778.805, 778.992, 779.178, 779.365, 779.552, 779.739, 779.925, 780.112, 780.299, 780.485, 780.672, 780.858, 781.045, 781.232, 781.418, 781.605, 781.791, 781.978, 782.164, 782.351, 782.537, 782.724, 782.91, 783.097, 783.283, 783.469, 783.656, 783.842, 784.028, 784.215, 784.401, 784.587, 784.774, 784.96, 785.146, 785.332, 785.518, 785.705, 785.891, 786.077, 786.263, 786.449, 786.635, 786.821, 787.007, 787.194, 787.38, 787.566, 787.752, 787.938, 788.124, 788.309, 788.495, 788.681, 788.867, 789.053, 789.239, 789.425, 789.611, 789.797, 789.982, 790.168, 790.354, 790.54, 790.725, 790.911, 791.097, 791.282, 791.468, 791.654, 791.839, 792.025, 792.211, 792.396, 792.582, 792.767, 792.953, 793.138, 793.324, 793.509, 793.695, 793.88, 794.066, 794.251, 794.437, 794.622, 794.807, 794.993, 795.178, 795.363, 795.549, 795.734, 795.919, 796.105, 796.29, 796.475, 796.66, 796.845, 797.031, 797.216, 797.401, 797.586, 797.771, 797.956, 798.141, 798.326, 798.511, 798.696, 798.881, 799.066, 799.251, 799.436, 799.621, 799.806, 799.991, 800.176, 800.361, 800.546, 800.731, 800.915, 801.1, 801.285, 801.47, 801.655, 801.839, 802.024, 802.209, 802.394, 802.578, 802.763, 802.947, 803.132, 803.317, 803.501, 803.686, 803.87, 804.055, 804.24, 804.424, 804.609, 804.793, 804.977, 805.162, 805.346, 805.531, 805.715, 805.899, 806.084, 806.268, 806.453, 806.637, 806.821, 807.005, 807.19, 807.374, 807.558, 807.742, 807.927, 808.111, 808.295, 808.479, 808.663, 808.847, 809.031, 809.215, 809.399, 809.583, 809.767, 809.951, 810.135, 810.319, 810.503, 810.687, 810.871, 811.055, 811.239, 811.423, 811.607, 811.791, 811.974, 812.158, 812.342, 812.526, 812.71, 812.893, 813.077, 813.261, 813.444, 813.628, 813.812, 813.995, 814.179, 814.363, 814.546, 814.73, 814.913, 815.097, 815.28, 815.464, 815.647, 815.831, 816.014, 816.198, 816.381, 816.565, 816.748, 816.931, 817.115, 817.298, 817.481, 817.665, 817.848, 818.031, 818.214, 818.398, 818.581, 818.764, 818.947, 819.13, 819.313, 819.497, 819.68, 819.863, 820.046, 820.229, 820.412, 820.595, 820.778, 820.961, 821.144, 821.327, 821.51, 821.693, 821.876, 822.059, 822.242, 822.424, 822.607, 822.79, 822.973, 823.156, 823.338, 823.521, 823.704, 823.887, 824.069, 824.252, 824.435, 824.617, 824.8, 824.983, 825.165, 825.348, 825.53, 825.713, 825.895, 826.078, 826.26, 826.443, 826.625, 826.808, 826.99, 827.173, 827.355, 827.538, 827.72, 827.902, 828.085, 828.267, 828.449, 828.631, 828.814, 828.996, 829.178, 829.36, 829.543, 829.725, 829.907, 830.089, 830.271, 830.453, 830.635, 830.818, 831, 831.182, 831.364, 831.546, 831.728, 831.91, 832.092, 832.274, 832.455, 832.637, 832.819, 833.001, 833.183, 833.365, 833.547, 833.728, 833.91, 834.092, 834.274, 834.455, 834.637, 834.819, 835.001, 835.182, 835.364, 835.545, 835.727, 835.909, 836.09, 836.272, 836.453, 836.635, 836.816, 836.998, 837.179, 837.361, 837.542, 837.724, 837.905, 838.087, 838.268, 838.449, 838.631, 838.812, 838.993, 839.175, 839.356, 839.537, 839.718, 839.899, 840.081, 840.262, 840.443, 840.624, 840.805, 840.986, 841.168, 841.349, 841.53, 841.711, 841.892, 842.073, 842.254, 842.435, 842.616, 842.797, 842.978, 843.158, 843.339, 843.52, 843.701, 843.882, 844.063, 844.244, 844.424, 844.605, 844.786, 844.967, 845.147, 845.328, 845.509, 845.689, 845.87, 846.051, 846.231, 846.412, 846.592, 846.773, 846.954, 847.134, 847.315, 847.495, 847.676, 847.856, 848.037, 848.217, 848.397, 848.578, 848.758, 848.938, 849.119, 849.299, 849.479, 849.66, 849.84, 850.02, 850.2, 850.381, 850.561, 850.741, 850.921, 851.101, 851.282, 851.462, 851.642, 851.822, 852.002, 852.182, 852.362, 852.542, 852.722, 852.902, 853.082, 853.262, 853.442, 853.622, 853.802, 853.981, 854.161, 854.341, 854.521, 854.701, 854.881, 855.06, 855.24, 855.42, 855.599, 855.779, 855.959, 856.139, 856.318, 856.498, 856.677, 856.857, 857.037, 857.216, 857.396, 857.575, 857.755, 857.934, 858.114, 858.293, 858.473, 858.652, 858.831, 859.011, 859.19, 859.369, 859.549, 859.728, 859.907, 860.087, 860.266, 860.445, 860.624, 860.804, 860.983, 861.162, 861.341, 861.52, 861.699, 861.878, 862.058, 862.237, 862.416, 862.595, 862.774, 862.953, 863.132, 863.311, 863.49, 863.669, 863.847, 864.026, 864.205, 864.384, 864.563, 864.742, 864.921, 865.099, 865.278, 865.457, 865.636, 865.814, 865.993, 866.172, 866.35, 866.529, 866.708, 866.886, 867.065, 867.243, 867.422, 867.601, 867.779, 867.958, 868.136, 868.315, 868.493, 868.671, 868.85, 869.028, 869.207, 869.385, 869.563, 869.742, 869.92, 870.098, 870.277, 870.455, 870.633, 870.811, 870.99, 871.168, 871.346, 871.524, 871.702, 871.88, 872.058, 872.236, 872.415, 872.593, 872.771, 872.949, 873.127, 873.305, 873.483, 873.661, 873.838, 874.016, 874.194, 874.372, 874.55, 874.728, 874.906, 875.083, 875.261, 875.439, 875.617, 875.794, 875.972, 876.15, 876.328, 876.505, 876.683, 876.861, 877.038, 877.216, 877.393, 877.571, 877.748, 877.926, 878.103, 878.281, 878.458, 878.636, 878.813, 878.991, 879.168, 879.345, 879.523, 879.7, 879.877, 880.055, 880.232, 880.409, 880.587, 880.764, 880.941, 881.118, 881.295, 881.473, 881.65, 881.827, 882.004, 882.181, 882.358, 882.535, 882.712, 882.889, 883.066, 883.243, 883.42, 883.597, 883.774, 883.951, 884.128, 884.305, 884.482, 884.658, 884.835, 885.012, 885.189, 885.366, 885.542, 885.719, 885.896, 886.073, 886.249, 886.426, 886.603, 886.779, 886.956, 887.132, 887.309, 887.486, 887.662, 887.839, 888.015, 888.192, 888.368, 888.545, 888.721, 888.897, 889.074, 889.25, 889.427, 889.603, 889.779, 889.956, 890.132, 890.308, 890.484, 890.661, 890.837, 891.013, 891.189, 891.365, 891.542, 891.718, 891.894, 892.07, 892.246, 892.422, 892.598, 892.774, 892.95, 893.126, 893.302, 893.478, 893.654, 893.83, 894.006, 894.182, 894.357, 894.533, 894.709, 894.885, 895.061, 895.237, 895.412, 895.588, 895.764, 895.939, 896.115, 896.291, 896.466, 896.642, 896.818, 896.993, 897.169, 897.344, 897.52, 897.695, 897.871, 898.046, 898.222, 898.397, 898.573, 898.748, 898.924, 899.099, 899.274, 899.45, 899.625, 899.8, 899.976, 900.151, 900.326, 900.501, 900.677, 900.852, 901.027, 901.202, 901.377, 901.552, 901.727, 901.903, 902.078, 902.253, 902.428, 902.603, 902.778, 902.953, 903.128, 903.303, 903.478, 903.652, 903.827, 904.002, 904.177, 904.352, 904.527, 904.701, 904.876, 905.051, 905.226, 905.401, 905.575, 905.75, 905.925, 906.099, 906.274, 906.448, 906.623, 906.798, 906.972, 907.147, 907.321, 907.496, 907.67, 907.845, 908.019, 908.194, 908.368, 908.542, 908.717, 908.891, 909.066, 909.24, 909.414, 909.589, 909.763, 909.937, 910.111, 910.286, 910.46, 910.634, 910.808, 910.982, 911.156, 911.33, 911.505, 911.679, 911.853, 912.027, 912.201, 912.375, 912.549, 912.723, 912.897, 913.071, 913.244, 913.418, 913.592, 913.766, 913.94, 914.114, 914.288, 914.461, 914.635, 914.809, 914.983, 915.156, 915.33, 915.504, 915.677, 915.851, 916.025, 916.198, 916.372, 916.545, 916.719, 916.892, 917.066, 917.239, 917.413, 917.586, 917.76, 917.933, 918.107, 918.28, 918.453, 918.627, 918.8, 918.973, 919.147, 919.32, 919.493, 919.666, 919.84, 920.013, 920.186, 920.359, 920.532, 920.705, 920.878, 921.052, 921.225, 921.398, 921.571, 921.744, 921.917, 922.09, 922.263, 922.436, 922.609, 922.781, 922.954, 923.127, 923.3, 923.473, 923.646, 923.818, 923.991, 924.164, 924.337, 924.509, 924.682, 924.855, 925.028, 925.2, 925.373, 925.545, 925.718, 925.891, 926.063, 926.236, 926.408, 926.581, 926.753, 926.926, 927.098, 927.271, 927.443, 927.615, 927.788, 927.96, 928.132, 928.305, 928.477, 928.649, 928.822, 928.994, 929.166, 929.338, 929.51, 929.683, 929.855, 930.027, 930.199, 930.371, 930.543, 930.715, 930.887, 931.059, 931.231, 931.403, 931.575, 931.747, 931.919, 932.091, 932.263, 932.435, 932.607, 932.779, 932.95, 933.122, 933.294, 933.466, 933.637, 933.809, 933.981, 934.153, 934.324, 934.496, 934.668, 934.839, 935.011, 935.182, 935.354, 935.525, 935.697, 935.868, 936.04, 936.211, 936.383, 936.554, 936.726, 936.897, 937.068, 937.24, 937.411, 937.582, 937.754, 937.925, 938.096, 938.268, 938.439, 938.61, 938.781, 938.952, 939.124, 939.295, 939.466, 939.637, 939.808, 939.979, 940.15, 940.321, 940.492, 940.663, 940.834, 941.005, 941.176, 941.347, 941.518, 941.689, 941.859, 942.03, 942.201, 942.372, 942.543, 942.713, 942.884, 943.055, 943.226, 943.396, 943.567, 943.738, 943.908, 944.079, 944.249, 944.42, 944.591, 944.761, 944.932, 945.102, 945.273, 945.443, 945.614, 945.784, 945.954, 946.125, 946.295, 946.465, 946.636, 946.806, 946.976, 947.147, 947.317, 947.487, 947.657, 947.828, 947.998, 948.168, 948.338, 948.508, 948.678, 948.848, 949.019, 949.189, 949.359, 949.529, 949.699, 949.869, 950.039, 950.209, 950.378, 950.548, 950.718, 950.888, 951.058, 951.228, 951.398, 951.567, 951.737, 951.907, 952.077, 952.246, 952.416, 952.586, 952.755, 952.925, 953.095, 953.264, 953.434, 953.603, 953.773, 953.942, 954.112, 954.281, 954.451, 954.62, 954.79, 954.959, 955.129, 955.298, 955.467, 955.637, 955.806, 955.975, 956.145, 956.314, 956.483, 956.652, 956.822, 956.991, 957.16, 957.329, 957.498, 957.667, 957.836, 958.006, 958.175, 958.344, 958.513, 958.682, 958.851, 959.02, 959.189, 959.357, 959.526, 959.695, 959.864, 960.033, 960.202, 960.371, 960.539, 960.708, 960.877, 961.046, 961.214, 961.383, 961.552, 961.72, 961.889, 962.058, 962.226, 962.395, 962.563, 962.732, 962.901, 963.069, 963.238, 963.406, 963.574, 963.743, 963.911, 964.08, 964.248, 964.416, 964.585, 964.753, 964.921, 965.09, 965.258, 965.426, 965.594, 965.763, 965.931, 966.099, 966.267, 966.435, 966.603, 966.771, 966.94, 967.108, 967.276, 967.444, 967.612, 967.78, 967.948, 968.116, 968.283, 968.451, 968.619, 968.787, 968.955, 969.123, 969.291, 969.458, 969.626, 969.794, 969.962, 970.129, 970.297, 970.465, 970.632, 970.8, 970.968, 971.135, 971.303, 971.47, 971.638, 971.805, 971.973, 972.14, 972.308, 972.475, 972.643, 972.81, 972.978, 973.145, 973.312, 973.48, 973.647, 973.814, 973.982, 974.149, 974.316, 974.483, 974.65, 974.818, 974.985, 975.152, 975.319, 975.486, 975.653, 975.82, 975.987, 976.154, 976.321, 976.488, 976.655, 976.822, 976.989, 977.156, 977.323, 977.49, 977.657, 977.824, 977.99, 978.157, 978.324, 978.491, 978.658, 978.824, 978.991, 979.158, 979.324, 979.491, 979.658, 979.824, 979.991, 980.157, 980.324, 980.49, 980.657, 980.823, 980.99, 981.156, 981.323, 981.489, 981.656, 981.822, 981.988, 982.155, 982.321, 982.487, 982.654, 982.82, 982.986, 983.152, 983.319, 983.485, 983.651, 983.817, 983.983, 984.149, 984.315, 984.481, 984.647, 984.814, 984.98, 985.146, 985.311, 985.477, 985.643, 985.809, 985.975, 986.141, 986.307, 986.473, 986.639, 986.804, 986.97, 987.136, 987.302, 987.467, 987.633, 987.799, 987.964, 988.13, 988.296, 988.461, 988.627, 988.792, 988.958, 989.124, 989.289, 989.455, 989.62, 989.786, 989.951, 990.116, 990.282, 990.447, 990.612, 990.778, 990.943, 991.108, 991.274, 991.439, 991.604, 991.769, 991.935, 992.1, 992.265, 992.43, 992.595, 992.76, 992.925, 993.091, 993.256, 993.421, 993.586, 993.751, 993.916, 994.081, 994.245, 994.41, 994.575, 994.74, 994.905, 995.07, 995.235, 995.399, 995.564, 995.729, 995.894, 996.058, 996.223, 996.388, 996.552, 996.717, 996.882, 997.046, 997.211, 997.375, 997.54, 997.705, 997.869, 998.034, 998.198, 998.362, 998.527, 998.691, 998.856, 999.02, 999.184, 999.349, 999.513, 999.677, 999.842, 1000.006, 1000.17, 1000.334, 1000.498, 1000.663, 1000.827, 1000.991, 1001.155, 1001.319, 1001.483, 1001.647, 1001.811, 1001.975, 1002.139, 1002.303, 1002.467, 1002.631, 1002.795, 1002.959, 1003.123, 1003.287, 1003.451, 1003.614, 1003.778, 1003.942, 1004.106, 1004.27, 1004.433, 1004.597, 1004.761, 1004.924, 1005.088, 1005.252, 1005.415, 1005.579, 1005.742, 1005.906, 1006.069, 1006.233, 1006.396, 1006.56, 1006.723, 1006.887, 1007.05, 1007.214, 1007.377, 1007.54, 1007.704, 1007.867, 1008.03, 1008.194, 1008.357, 1008.52, 1008.683, 1008.847, 1009.01, 1009.173, 1009.336, 1009.499, 1009.662, 1009.825, 1009.988, 1010.151, 1010.314, 1010.477, 1010.64, 1010.803, 1010.966, 1011.129, 1011.292, 1011.455, 1011.618, 1011.781, 1011.944, 1012.106, 1012.269, 1012.432, 1012.595, 1012.757, 1012.92, 1013.083, 1013.245, 1013.408, 1013.571, 1013.733, 1013.896, 1014.058, 1014.221, 1014.384, 1014.546, 1014.709, 1014.871, 1015.034, 1015.196, 1015.358, 1015.521, 1015.683, 1015.845, 1016.008, 1016.17, 1016.332, 1016.495, 1016.657, 1016.819, 1016.981, 1017.144, 1017.306, 1017.468, 1017.63, 1017.792, 1017.954, 1018.116, 1018.278, 1018.44, 1018.602, 1018.764, 1018.926, 1019.088, 1019.25, 1019.412, 1019.574, 1019.736, 1019.898, 1020.06, 1020.222, 1020.383, 1020.545, 1020.707, 1020.869, 1021.03, 1021.192, 1021.354, 1021.515, 1021.677, 1021.839, 1022, 1022.162, 1022.323, 1022.485, 1022.647, 1022.808, 1022.97, 1023.131, 1023.292, 1023.454, 1023.615, 1023.777, 1023.938, 1024.099, 1024.261, 1024.422, 1024.583, 1024.745, 1024.906, 1025.067, 1025.228, 1025.389, 1025.551, 1025.712, 1025.873, 1026.034, 1026.195, 1026.356, 1026.517, 1026.678, 1026.839, 1027, 1027.161, 1027.322, 1027.483, 1027.644, 1027.805, 1027.966, 1028.127, 1028.287, 1028.448, 1028.609, 1028.77, 1028.931, 1029.091, 1029.252, 1029.413, 1029.573, 1029.734, 1029.895, 1030.055, 1030.216, 1030.376, 1030.537, 1030.697, 1030.858, 1031.018, 1031.179, 1031.339, 1031.5, 1031.66, 1031.821, 1031.981, 1032.141, 1032.302, 1032.462, 1032.622, 1032.783, 1032.943, 1033.103, 1033.263, 1033.423, 1033.584, 1033.744, 1033.904, 1034.064, 1034.224, 1034.384, 1034.544, 1034.704, 1034.864, 1035.024, 1035.184, 1035.344, 1035.504, 1035.664, 1035.824, 1035.984, 1036.144, 1036.304, 1036.463, 1036.623, 1036.783, 1036.943, 1037.102, 1037.262, 1037.422, 1037.581, 1037.741, 1037.901, 1038.06, 1038.22, 1038.38, 1038.539, 1038.699, 1038.858, 1039.018, 1039.177, 1039.337, 1039.496, 1039.655, 1039.815, 1039.974, 1040.134, 1040.293, 1040.452]
input_cfl_mwcmsq = [0, 0.000105671, 3.46317E-05, 1.47153E-05, 3.4674E-06, 3.72534E-05, 3.55197E-05, 0, 8.13992E-05, 0, 3.20523E-05, 0, 1.12479E-05, 0, 2.33838E-05, 8.13992E-05, 0.00036285, 0.000105671, 0.000225169, 0.000210454, 0.000161953, 0.000181869, 0.000178402, 5.0235E-05, 0.000135102, 0.000180135, 0.000207832, 0.000164532, 0.000226015, 3.55197E-05, 0.000223436, 0.000157597, 0.000155018, 0.000198318, 0.000261535, 0.000128167, 0.000160219, 9.95819E-05, 8.8334E-05, 0.000150663, 0.000288386, 0.00024073, 7.53524E-05, 0.000118653, 0.000242464, 0.000271937, 0.000200052, 0.000199164, 0.000106517, 0.000157597, 0.000212188, 0.000128167, 0.000130746, 0.000185337, 0.000233838, 0.00019743, 0.000128167, 0.000126433, 0.000258067, 0.000222548, 0.000232104, 0.00033342, 0.000360271, 0.0001299, 0.000351602, 0.000331686, 0.000554234, 0.000636521, 0.000809722, 0.000977721, 0.00108077, 0.001301584, 0.001395965, 0.001417615, 0.001741521, 0.001609887, 0.001746722, 0.001782199, 0.002119932, 0.002061071, 0.001922502, 0.002064538, 0.002268015, 0.002232538, 0.002280997, 0.0021667, 0.002282731, 0.002255922, 0.002371953, 0.002216089, 0.002347681, 0.002247253, 0.002379733, 0.002273216, 0.002328652, 0.0023832, 0.002237739, 0.002131222, 0.00206365, 0.001936372, 0.001872267, 0.001666168, 0.001415882, 0.001199381, 0.001157814, 0.000973366, 0.00066075, 0.00051698, 0.000581085, 0.000244198, 0.000375832, 0.000274516, 0.000272783, 0.000223436, 0.000310882, 0.000172313, 0.000345513, 0.000224281, 0.00026238, 0.000271049, 0.000428646, 0.000204365, 0.000337733, 0.000246819, 0.000308302, 0.000229482, 0.000354182, 0.00019743, 0.0002979, 0.000351602, 0.000174934, 0.000323018, 0.00031177, 0.000349869, 0.000315237, 0.000309148, 0.000313503, 0.000188804, 0.000349869, 0.000421754, 0.000323018, 0.000321284, 0.000295321, 0.000442516, 0.000337733, 0.000308302, 0.000474568, 0.000348981, 0.00031955, 0.000381033, 0.000431268, 0.000357649, 0.000317816, 0.000373252, 0.000395748, 0.000347247, 0.0004659, 0.000445138, 0.000404417, 0.000403571, 0.000453764, 0.000374986, 0.000481503, 0.000359383, 0.000339467, 0.000348981, 0.000289232, 0.000443404, 0.00037672, 0.000494485, 0.000347247, 0.000255488, 0.000383655, 0.000200052, 0.000451184, 0.000274516, 0.000362005, 0.000321284, 0.000387968, 0.00040873, 0.000279717, 0.00040873, 0.000362005, 0.000348135, 0.000152396, 0.000335999, 0.000339467, 0.000249399, 0.000337733, 0.000234684, 0.000320396, 0.000289232, 0.000285764, 0.000239885, 0.000411352, 0.000306569, 0.00029012, 0.000213033, 0.000388813, 0.000222548, 0.00027625, 0.000389701, 0.000305681, 0.000229482, 0.000258913, 0.000212188, 0.000362005, 0.000370631, 0.000340355, 0.000375832, 0.000411352, 0.000236417, 0.000357649, 0.000265848, 0.000314349, 0.000366318, 0.000250287, 0.000271937, 0.000325597, 0.0003845, 0.00035507, 0.000261535, 0.000316083, 0.000324751, 0.000315237, 0.000369785, 0.000299634, 0.000320396, 0.000419132, 0.000349869, 0.000272783, 0.000331686, 0.000416553, 0.000351602, 0.000389701, 0.000160219, 0.000406996, 0.000413931, 0.000453764, 0.000404417, 0.000356804, 0.00051698, 0.000516135, 0.00044425, 0.000544719, 0.00051698, 0.000822703, 0.000985502, 0.001327548, 0.001906899, 0.002670698, 0.003814685, 0.004995883, 0.005665302, 0.006499253, 0.006919273, 0.007399888, 0.007605987, 0.008245975, 0.008687603, 0.009140521, 0.009326703, 0.009965804, 0.010097438, 0.010472424, 0.010554669, 0.011043952, 0.011286459, 0.01148909, 0.011635439, 0.011872745, 0.011942854, 0.01199829, 0.01243996, 0.01219314, 0.012277161, 0.01236892, 0.012257203, 0.011986154, 0.012187939, 0.01198442, 0.011648421, 0.011376526, 0.010677677, 0.010368487, 0.009852352, 0.009078192, 0.008339468, 0.007218907, 0.005941552, 0.004610537, 0.003510738, 0.002724401, 0.002644735, 0.002500119, 0.002555555, 0.00244295, 0.002638688, 0.002634333, 0.002558981, 0.00257285, 0.00253733, 0.002512255, 0.002332966, 0.002366751, 0.002257655, 0.002262814, 0.002159765, 0.002246365, 0.002292287, 0.002445571, 0.002411785, 0.002544265, 0.002459399, 0.002556401, 0.002565915, 0.002625664, 0.002748672, 0.002812735, 0.002649936, 0.002895868, 0.002981622, 0.003088985, 0.003069914, 0.003298551, 0.003178164, 0.003201548, 0.003266498, 0.003308065, 0.003373903, 0.003445788, 0.0036129, 0.003597339, 0.003771385, 0.003906487, 0.003766184, 0.003914267, 0.004008648, 0.00406235, 0.004135123, 0.004257201, 0.004216523, 0.004359405, 0.004385368, 0.004407906, 0.004434715, 0.004606182, 0.004728302, 0.004923153, 0.004931821, 0.005001972, 0.005117115, 0.005084217, 0.0051024, 0.005498191, 0.005493835, 0.005692153, 0.005517219, 0.005874868, 0.005789156, 0.006046335, 0.00600739, 0.006244653, 0.006311337, 0.006234251, 0.006470668, 0.006467201, 0.006579807, 0.006848234, 0.007157424, 0.007111502, 0.007154803, 0.007124484, 0.007462217, 0.007399, 0.007567888, 0.007881391, 0.007864054, 0.008011292, 0.007906509, 0.008224325, 0.008374988, 0.008521337, 0.008655551, 0.008758642, 0.00881577, 0.009150035, 0.009140521, 0.009392542, 0.009504259, 0.009647141, 0.009971893, 0.010418722, 0.011119305, 0.012068441, 0.013470495, 0.016153329, 0.019545415, 0.023937718, 0.027194702, 0.030126935, 0.032591535, 0.034449087, 0.035625084, 0.037697403, 0.039594787, 0.041361425, 0.042227429, 0.043498694, 0.044336959, 0.045628141, 0.045986678, 0.047368774, 0.048371612, 0.048948342, 0.048031258, 0.049455808, 0.050244725, 0.050645717, 0.051031951, 0.05067946, 0.051513411, 0.051197328, 0.050379826, 0.050378981, 0.049663682, 0.048899883, 0.047552376, 0.047677076, 0.045858511, 0.04573119, 0.04449629, 0.04182039, 0.03942332, 0.035784457, 0.032212236, 0.028772537, 0.025030582, 0.019653666, 0.017062591, 0.016170624, 0.016208723, 0.016402728, 0.016184494, 0.015811241, 0.016180138, 0.016010405, 0.015916024, 0.016306614, 0.016154175, 0.016361162, 0.016510979, 0.016430425, 0.016704941, 0.016945714, 0.016620963, 0.016774247, 0.017106779, 0.01659149, 0.01707646, 0.017061745, 0.017134476, 0.017372627, 0.017193379, 0.017037473, 0.017360491, 0.017215029, 0.017254862, 0.017407258, 0.017402945, 0.017575258, 0.017499948, 0.017449713, 0.017862798, 0.01784026, 0.017798693, 0.017863644, 0.017657545, 0.018336478, 0.017651456, 0.017711247, 0.017774464, 0.018288865, 0.017901743, 0.017997012, 0.017944197, 0.018014306, 0.018120823, 0.018356395, 0.017778777, 0.018439527, 0.017749347, 0.018190974, 0.018079257, 0.018091392, 0.018017774, 0.017960646, 0.01830616, 0.018045513, 0.01845771, 0.017859331, 0.018358128, 0.018203998, 0.017963226, 0.018229073, 0.018255079, 0.017936374, 0.018012615, 0.017795226, 0.017825545, 0.01837111, 0.018240364, 0.018598013, 0.018113043, 0.01801604, 0.018100907, 0.01790521, 0.017784824, 0.018169324, 0.018149408, 0.01824641, 0.018023863, 0.01809486, 0.017964114, 0.01813038, 0.017677461, 0.01837889, 0.018089659, 0.018017774, 0.017976207, 0.017977941, 0.017985764, 0.017915612, 0.017706892, 0.018036844, 0.018074098, 0.017953711, 0.017510308, 0.018338212, 0.017846307, 0.018020395, 0.017508574, 0.018003946, 0.017544939, 0.017932907, 0.017669681, 0.017750192, 0.017796114, 0.01747483, 0.018047247, 0.017841994, 0.017819498, 0.017925127, 0.017827278, 0.018024708, 0.017935529, 0.01767226, 0.017654923, 0.017780511, 0.018034223, 0.017799539, 0.017389076, 0.017595174, 0.017512041, 0.017597796, 0.017610778, 0.017681775, 0.017720762, 0.017833325, 0.017844573, 0.017898275, 0.017561431, 0.017365692, 0.017578725, 0.017625493, 0.017703425, 0.018156343, 0.018035111, 0.017632428, 0.017706046, 0.017933795, 0.017644563, 0.018025596, 0.017765795, 0.01734493, 0.017591707, 0.017441044, 0.017874046, 0.017802161, 0.018056761, 0.018071476, 0.017832479, 0.017831591, 0.017408146, 0.017960646, 0.018060228, 0.017690443, 0.017511196, 0.017953711, 0.017622025, 0.017766641, 0.01789743, 0.01794073, 0.018145095, 0.01827064, 0.01818146, 0.018132113, 0.017918192, 0.017946777, 0.018552979, 0.018251612, 0.018358128, 0.018396227, 0.018584143, 0.01898856, 0.01877726, 0.019188612, 0.019025813, 0.018910628, 0.019140111, 0.019367859, 0.019187724, 0.020161978, 0.020155043, 0.020743908, 0.020992461, 0.021638496, 0.021578747, 0.022313116, 0.022546066, 0.022883799, 0.024282343, 0.025360534, 0.025664481, 0.026361597, 0.026999852, 0.027534169, 0.028583733, 0.030105285, 0.03098427, 0.032291901, 0.03342722, 0.035096856, 0.036331756, 0.037211587, 0.037665393, 0.039333252, 0.040681604, 0.041882761, 0.042615354, 0.043652824, 0.044988195, 0.045978897, 0.04647334, 0.047525525, 0.048435674, 0.049073929, 0.050325278, 0.05118088, 0.051698748, 0.052173316, 0.052659978, 0.052143843, 0.053222034, 0.052130016, 0.053193449, 0.053679265, 0.052899017, 0.05280028, 0.053568435, 0.052935382, 0.053240217, 0.052908531, 0.050840525, 0.050576411, 0.052167227, 0.050404944, 0.049362315, 0.048030412, 0.04797671, 0.047366194, 0.04676088, 0.046291513, 0.045409906, 0.045191672, 0.043336742, 0.043205108, 0.042055074, 0.041821278, 0.04062879, 0.041470522, 0.040280655, 0.039115906, 0.03863009, 0.03915836, 0.037965026, 0.03731379, 0.037250574, 0.036556037, 0.035979308, 0.034527907, 0.034186706, 0.034352972, 0.033702582, 0.033070416, 0.032585488, 0.031109815, 0.03132805, 0.030746119, 0.030733983, 0.030036868, 0.029364869, 0.029190781, 0.028379368, 0.02845472, 0.027612946, 0.027140112, 0.027218932, 0.026629179, 0.025741567, 0.026207467, 0.025408148, 0.024991595, 0.024626165, 0.024366364, 0.024227795, 0.023636351, 0.02325443, 0.023528946, 0.022583277, 0.022295779, 0.022506233, 0.022236876, 0.021454048, 0.021512909, 0.021279113, 0.021242748, 0.02086865, 0.02094396, 0.020683313, 0.020558614, 0.02073266, 0.020794143, 0.020303126, 0.020344692, 0.020368964, 0.020159398, 0.020291878, 0.020181048, 0.019779211, 0.020462499, 0.020500598, 0.020408797, 0.020335178, 0.020415732, 0.020420045, 0.020618363, 0.020500598, 0.020339491, 0.020740483, 0.021192513, 0.020474593, 0.020923198, 0.020932712, 0.021068659, 0.020710164, 0.021040074, 0.020928399, 0.021733765, 0.02143151, 0.021736345, 0.021483478, 0.021557097, 0.022088793, 0.021463562, 0.02200308, 0.022184061, 0.021887895, 0.021997033, 0.022042913, 0.022364197, 0.021876647, 0.02232263, 0.022446484, 0.022367664, 0.022159832, 0.022355528, 0.021946799, 0.02262311, 0.022261148, 0.022055049, 0.022212646, 0.022180594, 0.021939864, 0.022527883, 0.021927728, 0.021890517, 0.022141649, 0.02196236, 0.021900877, 0.021717316, 0.021624627, 0.021534559, 0.021434977, 0.02113103, 0.021402079, 0.021124095, 0.020942226, 0.020902393, 0.020672911, 0.020264181, 0.020493664, 0.020178427, 0.019955879, 0.020284943, 0.019702125, 0.019638908, 0.019255296, 0.019348831, 0.019211108, 0.019015411, 0.018960863, 0.018938325, 0.018540843, 0.018465491, 0.018283664, 0.018035956, 0.018283664, 0.017921659, 0.017799539, 0.017847195, 0.017516397, 0.017422862, 0.017041829, 0.017159593, 0.016868628, 0.016923176, 0.016515292, 0.016510979, 0.016684179, 0.016212191, 0.015943763, 0.016069309, 0.016126478, 0.015613811, 0.015879659, 0.015525477, 0.015518542, 0.015325425, 0.015300308, 0.015124528, 0.014881176, 0.015036194, 0.014811024, 0.014868194, 0.014709709, 0.014353793, 0.014372822, 0.014507078, 0.014304404, 0.014094838, 0.014242921, 0.014530461, 0.013955423, 0.014281908, 0.014029888, 0.014442127, 0.014172812, 0.014682858, 0.014257679, 0.014564205, 0.014579808, 0.01457114, 0.014668988, 0.014997207, 0.014925322, 0.015122794, 0.014967776, 0.015539347, 0.015351388, 0.015631106, 0.015970572, 0.016191429, 0.016730947, 0.016952607, 0.017568323, 0.018080145, 0.018526973, 0.019542794, 0.019783524, 0.02049971, 0.021460983, 0.022732248, 0.023454482, 0.025008932, 0.026455978, 0.027954147, 0.02924017, 0.03242269, 0.034232586, 0.037405592, 0.040679025, 0.044885991, 0.049152749, 0.053752884, 0.059370531, 0.065157107, 0.070635339, 0.07709476, 0.082881294, 0.089424735, 0.094674373, 0.101997175, 0.107044181, 0.113929669, 0.11950317, 0.125985086, 0.132523326, 0.136833342, 0.142129747, 0.149630951, 0.154036235, 0.159663438, 0.167284183, 0.173269881, 0.185702906, 0.192869844, 0.203201965, 0.210863388, 0.220434332, 0.226282349, 0.232015181, 0.234225223, 0.243424605, 0.247460147, 0.247561462, 0.251977994, 0.257567098, 0.256311436, 0.257423328, 0.257108134, 0.256129567, 0.258997695, 0.252813679, 0.249787065, 0.246944858, 0.245250992, 0.243120658, 0.238428722, 0.233008505, 0.227482617, 0.226968217, 0.222450326, 0.219705164, 0.214426053, 0.209075988, 0.203489463, 0.197101928, 0.19240479, 0.186387928, 0.179278117, 0.172305184, 0.166625166, 0.155385475, 0.149999002, 0.138644126, 0.127811473, 0.112473912, 0.102075107, 0.092460906, 0.086537536, 0.080295462, 0.075929165, 0.071025929, 0.067324653, 0.064068556, 0.060024388, 0.057716498, 0.053189094, 0.051050133, 0.048239132, 0.044900706, 0.042657809, 0.040296258, 0.037832504, 0.03579135, 0.033749349, 0.031651067, 0.030016951, 0.028601915, 0.026810202, 0.024898948, 0.024288433, 0.022987694, 0.02168353, 0.020737016, 0.019690877, 0.018589344, 0.017832479, 0.01674913, 0.016180138, 0.015266522, 0.014709709, 0.014074922, 0.01312494, 0.012833086, 0.012233861, 0.011750625, 0.01136697, 0.010680256, 0.01050621, 0.010215202, 0.009643674, 0.009730274, 0.00927909, 0.008895435, 0.008821859, 0.008755175, 0.008404418, 0.008215657, 0.007990487, 0.007812974, 0.007638885, 0.007496891, 0.007503783, 0.007291638, 0.00707945, 0.006969508, 0.006822271, 0.0068032, 0.00672789, 0.006393625, 0.006332987, 0.006246387, 0.006122533, 0.0060394, 0.005984007, 0.005971871, 0.005879224, 0.005697355, 0.005703401, 0.005483433, 0.005374337, 0.005377804, 0.005541491, 0.005231455, 0.005214118, 0.005057366, 0.005065189, 0.00496992, 0.004893722, 0.004744751, 0.004776803, 0.004878119, 0.004671132, 0.004516156, 0.004555989, 0.004391415, 0.004555989, 0.004219102, 0.004271071, 0.00430917, 0.004073598, 0.004115165, 0.00406844, 0.004145483, 0.004036387, 0.004043322, 0.003956722, 0.003737599, 0.003970549, 0.003851939, 0.003925515, 0.003856252, 0.004267603, 0.004634767, 0.005575235, 0.006397938, 0.007354854, 0.00810914, 0.008786339, 0.009602953, 0.01039796, 0.011088986, 0.011770541, 0.012684157, 0.01401864, 0.015489957, 0.016876408, 0.018140739, 0.019557509, 0.020525716, 0.022053315, 0.022639559, 0.024511868, 0.025257485, 0.026297535, 0.027759296, 0.028285833, 0.029513798, 0.03046378, 0.031486535, 0.03233609, 0.032918866, 0.034323499, 0.034763436, 0.035729021, 0.036305751, 0.037154459, 0.037939909, 0.038325255, 0.039674453, 0.039950703, 0.040692895, 0.041382187, 0.041448026, 0.042109622, 0.041401258, 0.040828842, 0.040118702, 0.03963204, 0.03935837, 0.039788792, 0.039718641, 0.039800886, 0.040588069, 0.041023693, 0.040935359, 0.041079974, 0.03984592, 0.040556905, 0.040018275, 0.040963055, 0.04084787, 0.041197739, 0.041469676, 0.042019554, 0.041762375, 0.041867158, 0.041869737, 0.042639626, 0.042184974, 0.042785975, 0.042180661, 0.041748505, 0.041664527, 0.041724276, 0.041086021, 0.040297104, 0.04000876, 0.039193838, 0.039459686, 0.03806372, 0.038522685, 0.037272224, 0.037449738, 0.036711901, 0.036834021, 0.036884256, 0.036274586, 0.036261605, 0.036231286, 0.035434588, 0.035129754, 0.035505586, 0.034867373, 0.034223072, 0.033924284, 0.033473099, 0.032297103, 0.03153935, 0.030585055, 0.030042915, 0.029845484, 0.029085152, 0.028573331, 0.028645216, 0.028423514, 0.027864967, 0.0277368, 0.027825134, 0.027260498, 0.027104635, 0.026772949, 0.026828385, 0.026240365, 0.02619013, 0.026251613, 0.025874935, 0.025633317, 0.025524179, 0.024972567, 0.024653017, 0.024601894, 0.023967994, 0.024077979, 0.023134931, 0.02260146, 0.022320897, 0.021288628, 0.020952628, 0.020100495, 0.019756715, 0.019500381, 0.019201593, 0.018971265, 0.018539997, 0.018565115, 0.018197063, 0.018403162, 0.018302692, 0.017622913, 0.01784026, 0.017413347, 0.017573524, 0.017184711, 0.017113714, 0.016687647, 0.01687903, 0.016392326, 0.016540409, 0.016155909, 0.01571944, 0.015684808, 0.01544061, 0.015090742, 0.014802356, 0.014392738, 0.014184906, 0.013999611, 0.013506861, 0.013480855, 0.013342328, 0.013156992, 0.013492991, 0.013403811, 0.013724207, 0.014315694, 0.01469326, 0.015408558, 0.016258113, 0.017483456, 0.019031014, 0.021362246, 0.024215702, 0.02869025, 0.033856754, 0.040249491, 0.046840545, 0.053096447, 0.060664334, 0.065999684, 0.073550277, 0.078573012, 0.085770269, 0.091928322, 0.098126208, 0.103036379, 0.108661848, 0.113691518, 0.120004589, 0.122470034, 0.128052204, 0.132857592, 0.137761716, 0.139528312, 0.143194914, 0.148436771, 0.149612768, 0.151249506, 0.154694406, 0.156001191, 0.157394577, 0.158141928, 0.15937594, 0.158336779, 0.158822595, 0.156587477, 0.156411655, 0.151431375, 0.153172853, 0.147953534, 0.145043797, 0.140609082, 0.137973861, 0.131535245, 0.126160908, 0.117519185, 0.11069518, 0.100872259, 0.092086807, 0.082452648, 0.077584043, 0.070994722, 0.065897523, 0.062045584, 0.057603935, 0.054960933, 0.051592231, 0.049047966, 0.04621616, 0.044581156, 0.042450822, 0.040548236, 0.038545223, 0.03656644, 0.03508214, 0.033588285, 0.032804569, 0.0313497, 0.030679435, 0.029489569, 0.0296056, 0.028604537, 0.028468548, 0.028624454, 0.028364653, 0.028061551, 0.028246, 0.028382835, 0.028401864, 0.02803728, 0.028895502, 0.029183, 0.028821884, 0.029350154, 0.029770132, 0.029798717, 0.030123468, 0.030147698, 0.030067186, 0.03020918, 0.030152053, 0.030416167, 0.030459467, 0.030207447, 0.030389316, 0.030363353, 0.030497566, 0.030296669, 0.03002562, 0.029974497, 0.03005505, 0.029828147, 0.030343436, 0.030604971, 0.030893315, 0.031296886, 0.031212865, 0.031422431, 0.031079539, 0.031584383, 0.031180855, 0.03123887, 0.031339298, 0.030921054, 0.031011121, 0.030497566, 0.03012258, 0.029717318, 0.029650634, 0.028905862, 0.028578532, 0.028291034, 0.027289929, 0.027056979, 0.026539999, 0.025985765, 0.025451448, 0.024595847, 0.024181916, 0.02374718, 0.023054378, 0.022394515, 0.021589995, 0.020895459, 0.020747376, 0.019732443, 0.019179098, 0.018592812, 0.017745879, 0.016806257, 0.016313506, 0.015122794, 0.014472446, 0.013414171, 0.012489307, 0.011724619, 0.010605792, 0.00960122, 0.008394058, 0.007939407, 0.007366102, 0.006621373, 0.006227316, 0.005898252, 0.005714649, 0.005296405, 0.005283424, 0.005079904, 0.004996771, 0.004968187, 0.004827038, 0.004741283, 0.004608804, 0.004536073, 0.00446072, 0.004195718, 0.004327353, 0.004288365, 0.004262402, 0.004148951, 0.004200919, 0.004061505, 0.004262402, 0.003940273, 0.003961035, 0.004038967, 0.003945432, 0.003998288, 0.004073598, 0.003936805, 0.003956722, 0.004021672, 0.00394632, 0.003826821, 0.003986152, 0.003963657, 0.004055416, 0.003868388, 0.004157619, 0.003935072, 0.00382162, 0.004095248, 0.0039541, 0.004098716, 0.004112585, 0.003978372, 0.003957567, 0.00418705, 0.00418447, 0.004283207, 0.004403551, 0.004478015, 0.004610537, 0.004730924, 0.005105867, 0.0050158, 0.005315434, 0.00541159, 0.005549271, 0.005745856, 0.005719005, 0.00600997, 0.006085322, 0.006137291, 0.006410919, 0.006416966, 0.006656005, 0.006743451, 0.006788485, 0.006963419, 0.006988537, 0.006981602, 0.007043085, 0.007283857, 0.007298573, 0.007134886, 0.007449235, 0.007386907, 0.007485601, 0.007439721, 0.007437987, 0.007501204, 0.007518541, 0.007389486, 0.007487334, 0.007328003, 0.007508139, 0.007211084, 0.007161737, 0.007148756, 0.00709074, 0.006863837, 0.006943503, 0.006634355, 0.006260257, 0.006274972, 0.005917323, 0.00586362, 0.005486055, 0.005383005, 0.005317167, 0.005227099, 0.005183799, 0.00511369, 0.004927466, 0.004850422, 0.004723101, 0.004618318, 0.00452825, 0.004558569, 0.004377587, 0.004415687, 0.004397504, 0.004387102, 0.004544699, 0.004548166, 0.004462454, 0.004436449, 0.00462872, 0.004602757, 0.004457253, 0.004635655, 0.004536073, 0.004598401, 0.004697983, 0.004575906, 0.004842599, 0.004597556, 0.004700605, 0.004614005, 0.004738704, 0.004577639, 0.004603602, 0.004598401, 0.004701451, 0.004418266, 0.004568971, 0.004475436, 0.004626986, 0.004452052, 0.004467655, 0.004260669, 0.004436449, 0.004302235, 0.004329932, 0.004510067, 0.004312637, 0.004334287, 0.004272804, 0.004253734, 0.004219102, 0.004027719, 0.004161087, 0.003936805, 0.003980951, 0.003702968, 0.003748889, 0.003533234, 0.003376483, 0.003300284, 0.003362655, 0.003127084, 0.003151356, 0.00288462, 0.002933967, 0.002964285, 0.002946948, 0.002915784, 0.00309592, 0.002988515, 0.002847366, 0.002730448, 0.002838698, 0.002856035, 0.002807534, 0.002821403, 0.002910583, 0.00269497, 0.00278157, 0.002612683, 0.002854301, 0.00265167, 0.002689769, 0.002634333, 0.002614417, 0.002526083, 0.002589299, 0.002494918, 0.002590187, 0.002512255, 0.002524349, 0.002393603, 0.002438637, 0.002386668, 0.002438637, 0.002422188, 0.0024646, 0.002393603, 0.002365018, 0.002277572, 0.002307002, 0.00228882, 0.002340788, 0.002341634, 0.00226717, 0.002403117, 0.002372798, 0.002436015, 0.002336433, 0.002331232, 0.002444683, 0.002336433, 0.002467221, 0.002390981, 0.002295754, 0.002390135, 0.002348569, 0.00239707, 0.002513101, 0.002517414, 0.002446417, 0.0024646, 0.002523503, 0.002512255, 0.002500965, 0.002351148, 0.002426501, 0.0025512, 0.002542532, 0.002568537, 0.002645581, 0.002513101, 0.002668119, 0.002449884, 0.002678521, 0.002555555, 0.002643847, 0.002577206, 0.002674166, 0.002809267, 0.002844787, 0.002787617, 0.002856035, 0.003023189, 0.003121883, 0.003201548, 0.003231021, 0.003215418, 0.003137486, 0.003310687, 0.003442321, 0.003420671, 0.003539281, 0.003540169, 0.003575689, 0.003640639, 0.003662289, 0.003641485, 0.00363197, 0.003759249, 0.003630237, 0.003659667, 0.003942006, 0.003876168, 0.003715103, 0.003770539, 0.003936805, 0.003659667, 0.003744534, 0.003741067, 0.003579156, 0.003566132, 0.003714215, 0.003572221, 0.003670915, 0.003403334, 0.00367785, 0.003546216, 0.003518519, 0.003573067, 0.003386039, 0.003405067, 0.003274321, 0.003129705, 0.003088139, 0.002847366, 0.002921831, 0.002680255, 0.002579785, 0.002473268, 0.002448151, 0.002453352, 0.002377999, 0.002370219, 0.002359817, 0.002119086, 0.002141582, 0.002184882, 0.002265436, 0.002058449, 0.002069697, 0.002138115, 0.002029864, 0.00193375, 0.00188787, 0.001976204, 0.001957134, 0.00184715, 0.001785667, 0.001775264, 0.001800382, 0.00148172, 0.001628915, 0.001411568, 0.001495547, 0.001253083, 0.001324968, 0.001129272, 0.00119418, 0.000966431, 0.000872938, 0.000912771, 0.000942201, 0.000912771, 0.001008885, 0.000913617, 0.001008885, 0.000941313, 0.001102421, 0.001021867, 0.001096331, 0.001057387, 0.001188133, 0.001214984, 0.001347464, 0.001291182, 0.001440999, 0.00157868, 0.001683505, 0.001978784, 0.002119932, 0.002352882, 0.002623931, 0.002881152, 0.003085517, 0.003364389, 0.00358867, 0.003812952, 0.004103917, 0.004388835, 0.004851268, 0.005114536, 0.005436666, 0.005859307, 0.006184016, 0.006468935, 0.006887221, 0.007255272, 0.00736957, 0.007483867, 0.007908242, 0.00811692, 0.008267625, 0.008409619, 0.008849556, 0.00892402, 0.009176886, 0.00917862, 0.009725073, 0.009776154, 0.010128602, 0.010401427, 0.010632643, 0.01040574, 0.010848256, 0.010762544, 0.010700173, 0.01086906, 0.010957352, 0.010669854, 0.010535641, 0.010630909, 0.010310471, 0.009902587, 0.009777042, 0.009499058, 0.009295539, 0.00859669, 0.008720501, 0.008488439, 0.008081401, 0.007828535, 0.007454436, 0.007162583, 0.006952171, 0.006481071, 0.00622905, 0.005858419, 0.005517219, 0.005254839, 0.004961252, 0.004639968, 0.004683268, 0.004466767, 0.004227771, 0.00417669, 0.003800816, 0.003605119, 0.003348786, 0.0032336, 0.003001539, 0.002645581, 0.002386668, 0.002014303, 0.001778732, 0.001582147, 0.001337104, 0.001136164, 0.001020133, 0.000948248, 0.000780249, 0.000710097, 0.000744771, 0.000592333, 0.000564636, 0.0006391, 0.000610515, 0.000593221, 0.00044425, 0.000584552, 0.00052827, 0.000410464, 0.000554234, 0.000436469, 0.000433847, 0.000448563, 0.000471101, 0.000508354, 0.000531738, 0.000451184, 0.000558547, 0.000413931, 0.000438203, 0.000294433, 0.00048755, 0.000369785, 0.000554234, 0.000413931, 0.000471101, 0.000485816, 0.000494485, 0.000400104, 0.000423487, 0.000445138, 0.000469367, 0.000449451, 0.000400949, 0.000438203, 0.00043038, 0.000456385, 0.000431268, 0.0005525, 0.000536897, 0.000428646, 0.000559435, 0.000550766, 0.000584552, 0.000581085, 0.000723121, 0.000657283, 0.000623497, 0.000678933, 0.000564636, 0.000603581, 0.00078545, 0.000628698, 0.0007638, 0.000648614, 0.000807988, 0.000786338, 0.0007205, 0.000792385, 0.000832217, 0.000827904, 0.000781137, 0.000775936, 0.000765533, 0.000776781, 0.000776781, 0.000770734, 0.000642568, 0.000712719, 0.000817502, 0.000735215, 0.000792385, 0.000733481, 0.000758599, 0.000715298, 0.000835685, 0.000539518, 0.000839152, 0.000669419, 0.000589753, 0.000573304, 0.000670265, 0.000492751, 0.000621763, 0.000587132, 0.000345513, 0.000568103, 0.000694536, 0.000468521, 0.000395748, 0.000443404, 0.0005525, 0.000474568, 0.000409618, 0.000417398, 0.000401837, 0.000324751, 0.00040615, 0.000455497, 0.00045203, 0.000357649, 0.000392281, 0.000348135, 0.000407884, 0.00046332, 0.000476302, 0.000428646, 0.000422599, 0.000462432, 0.000457231, 0.000394015, 0.000476302, 0.000368897, 0.000481503, 0.00038708, 0.000613137, 0.000433002, 0.000500531, 0.000447717, 0.00052827, 0.000411352, 0.000481503, 0.000412197, 0.00046332, 0.00045203, 0.000413085, 0.000387968, 0.000489283, 0.000395748, 0.000471946, 0.000434735, 0.00044167, 0.000458119, 0.00056028, 0.000344668, 0.000471101, 0.000361117, 0.000524803, 0.000467633, 0.00047368, 0.00047368, 0.000483237, 0.000426913, 0.000351602, 0.000346401, 0.000355916, 0.000501419, 0.000417398, 0.000267581, 0.000347247, 0.000344668, 0.000517868, 0.000314349, 0.000433002, 0.000428646, 0.000368897, 0.000338621, 0.000390547, 0.00040615, 0.000405262, 0.000388813, 0.000386234, 0.000395748, 0.000553388, 0.000356804, 0.000431268, 0.000429534, 0.000364584, 0.00031955, 0.000390547, 0.000439048, 0.000423487, 0.000363696, 0.00046332, 0.000482349, 0.000446871, 0.0004659, 0.000449451, 0.000420866, 0.000375832, 0.000350714, 0.000340355, 0.00036285, 0.00043038, 0.00037672, 0.000336887, 0.000274516, 0.000245931, 0.000257179, 0.000248553, 0.000239885, 0.000278829, 0.000281451, 0.000201786, 0.000179247, 0.000200898, 0.000115185, 0.000157597, 0.000166266, 0.000231216, 0.000118653, 0.000164532, 0.000191383, 0.000291853, 0.000149817, 0.000167154, 0.000216501, 0.000164532, 0.000305681, 0.000182715, 0.000106517, 0.000131634, 0.000168888, 0.000221702, 0.000209566, 0.000187916, 0.00019743, 0.000477148, 0.000363696, 0.000510933, 0.00026238, 0.000294433, 0.000358537, 0.000418286, 0.000368897, 0.000436469, 0.00038708, 0.000478881, 0.000401837, 0.000403571, 0.000562014, 0.000414819, 0.000443404, 0.000549921, 0.000489283, 0.000545565, 0.000530004, 0.000580197, 0.000520448, 0.000633899, 0.000499686, 0.000640834, 0.000546453, 0.000586286, 0.00058802, 0.000623497, 0.000540364, 0.000546453, 0.000459853, 0.000523915, 0.000564636, 0.000472834, 0.000447717, 0.000459853, 0.000400949, 0.000426067, 0.000404417, 0.000458965, 0.000396636, 0.000284918, 0.000245086, 0.000260647, 0.000245086, 0.000320396, 0.000252866, 0.000215613, 0.000174046, 0.000272783, 0.00024073, 0.000200052, 0.000212188, 0.000249399, 0.000286652, 0.000159331, 0.000221702, 0.000289232, 0.000187916, 0.000307414, 0.000195696, 0.000287498, 0.000267581, 0.000339467, 0.000295321, 0.000315237, 0.000239885, 0.000337733, 0.000274516, 0.000213879, 0.00023295, 0.000363696, 0.000244198, 0.000310036, 0.000251132, 0.000329952, 0.000267581, 0.000342046, 0.000289232, 0.000362005, 0.00031955, 0.000204365, 0.000205253, 0.000321284, 0.000303101, 0.000222548, 0.00014377, 0.000271937, 0.000157597, 6.23708E-05, 0.000184449, 0.000155864, 0.000109984, 0.000106517, 7.70861E-05, 4.15665E-05, 9.87362E-05, 0.000128167, 0.000181869, 0.000122966, 7.61981E-05, 0.000116031, 5.19687E-05, 0.000128167, 3.98328E-05, 0.000131634, 0.000171467, 0.000107362, 5.10807E-05, 6.40623E-05, 7.44644E-05, 0.0001299, 0.000147237, 0.000151551, 0.00015413, 0.000170579, 1.6449E-05, 8.40209E-05, 9.00677E-05, 8.91797E-05, 4.67676E-05, 7.27307E-05, 4.76133E-05, 8.48666E-05, 0.000104783, 0.000194851, 9.70025E-05, 0.000150663, 1.7337E-05, 0.000140303, 7.01513E-05, 0.000224281, 3.80991E-05, 0.000126433, 7.53524E-05, 8.66003E-05, 0.000109096, 2.42295E-05, 0.000124699, 0.000225169, 3.80991E-05, 0.000121232, 5.97492E-05, 0.000142882, 7.79318E-05, 0.000125587, 0, 3.55197E-05, 0.000116919, 7.96655E-05, 0, 9.43808E-05, 0.00010047, 5.45481E-05, 7.36187E-05, 0.000137681, 6.84176E-05, 0.000151551, 5.10807E-05, 0.000148083, 0.000128167, 9.26471E-05, 0.00011083, 0, 0.00010047, 0.000168888, 0.000170579, 5.0235E-05, 0.000158485, 0.000158485, 9.70025E-05, 0.000216501, 0.000142882, 0.000223436, 0.000133368, 0.000152396, 0.000203519, 0.0002546, 0.000182715, 0.000186182, 0.000190538, 0.000158485, 0.000176668, 0.000210454, 0.000232104, 0.000122966, 0.000344668, 0.000404417, 0.00019743, 0.000357649, 0.000213879, 0.000255488, 0.000270203, 0.000284918, 0.000258913, 0.00023295, 0.000142036, 0.000204365, 0.000163686, 0.000243352, 0.000252866, 0.000249399, 0.000399216, 0.000318704, 0.000258913, 0.000351602, 0.000313503, 0.000329952, 0.000193117, 0.000258067, 0.000287498, 0.000360271, 0.000213033, 0.000395748, 0.000468521, 0.00019743, 0.00038708, 0.000407884, 0.000335999, 0.0003412, 0.000346401, 0.000295321, 0.000331686, 0.000326485, 0.000273671, 0.000212188, 0.000309148, 0.000397482, 0.000382767, 0.0003845, 0.000383655, 0.000329064, 0.000427801, 0.000323863, 0.000381921, 0.000251132, 0.00023037, 0.000393169, 0.000298746, 0.000264114, 0.000378454, 0.000433847, 0.000354182, 0.000372365, 0.000257179, 0.000573304, 0.000413085, 0.000534317, 0.00038708, 0.00049533, 0.000403571, 0.00053085, 0.000563748, 0.000619184, 0.00073957, 0.000846933, 0.000774202, 0.000866003, 0.000918818, 0.001110201, 0.000970786, 0.001058232, 0.00121325, 0.001137052, 0.001156969, 0.00108335, 0.001167371, 0.00133537, 0.0013596, 0.001322346, 0.001333637, 0.001449668, 0.001229699, 0.001414148, 0.001437532, 0.001462649, 0.001400321, 0.001436686, 0.001247882, 0.001524132, 0.001344885, 0.0014029, 0.001393386, 0.0014462, 0.001327548, 0.00129465, 0.001389918, 0.001272999, 0.00119418, 0.001124071, 0.001119715, 0.001011465, 0.000891966, 0.00092922, 0.000712719, 0.000839998, 0.00067462, 0.000691915, 0.000502265, 0.000563748, 0.000435581, 0.000439048, 0.000349869, 0.000259801, 0.000339467, 0.000246819, 0.000193117, 0.000188804, 5.37024E-05, 0.000201786, 9.35351E-05, 0.000191383, 0.000116919, 0.00016542, 0.000144616, 0.000176668, 0.000174934, 0.000229482, 0.000177514, 0.000126433, 5.10807E-05, 0.000160219, 7.53524E-05, 0.000135102, 4.15665E-05, 0.00016542, 0.000134214, 0.000190538, 0.000180981, 0.000183603, 5.62818E-05, 0.000180981, 0.000174934, 5.80155E-05, 9.43808E-05, 5.97492E-05, 9.35351E-05, 0.000288386, 4.85013E-05, 0.00015413, 0.000139415, 0.000130746, 9.18014E-05, 0.000160219, 0.000109096, 0.000227749, 0.000302213, 0.000138569, 9.70025E-05, 0.000216501, 0.000155864, 0.000161065, 0.000160219, 0.000173201, 0.000126433, 0.000141148, 0.000206099, 0.000173201, 0.000109096, 0.000183603, 0.000200052, 0.000162798, 0.000124699, 0.000142036, 0.000252866, 0.000204365, 0.000137681, 0.000146349, 6.14829E-05, 0.00023295, 0.000125587, 0.000204365, 0.000234684, 0.000155018, 0.000245931, 0.000193963, 0.000171467, 0.000187916, 0.000146349, 0.000222548, 0.00010825, 0.000168888, 0.000109096, 7.0997E-05, 0.000105671, 0.000196584, 3.46317E-05, 0.000217347, 1.21359E-05, 0.000121232, 0.000151551, 8.66003E-05, 4.76133E-05, 0.000183603, 6.49502E-05, 0.000142882, 7.36187E-05, 0.000188804, 0.000111718, 0.000136835, 0.000161065, 0.000173201, 5.0235E-05, 0.000173201, 5.45481E-05, 0.00010825, 3.46317E-05, 0.000213879, 1.7337E-05, 0.000163686, 0.000111718, 0.00019743, 2.94306E-05, 0.000129013, 0.000120386, 0.000226015, 0.000162798, 0.000263268, 0.000171467, 0.000173201, 0.000231216, 0.000228637, 0.000212188, 0.000271049, 0.000155864, 0.000223436, 0.000248553, 0.000339467, 0.000361117, 0.00038708, 0.000389701, 0.000447717, 0.000405262, 0.000352448, 0.00044425, 0.000478036, 0.000411352, 0.000485816, 0.000327331, 0.000478036, 0.000332532, 0.000497952, 0.000483237, 0.000613137, 0.000481503, 0.000575038, 0.000484082, 0.000550766, 0.000581931, 0.000601847, 0.000489283, 0.000612249, 0.0005092, 0.00058802, 0.000541252, 0.000602735, 0.000436469, 0.000510933, 0.000512667, 0.000537785, 0.000387968, 0.000426067, 0.000480615, 0.000347247, 0.000270203, 0.000433002, 0.000310036, 0.000356804, 0.00034378, 0.000367163, 0.000216501, 0.000253754, 0.00013248, 0.000284918, 0.000131634, 0.000200898, 0.000146349, 0.000155018, 8.7446E-05, 0.000173201, 8.91797E-05, 0.000260647, 0.000274516, 0.00023037, 0.000104783, 0.000164532, 0.00015413, 0.000138569, 0.000222548, 0.00023295, 0.000172313, 0.000283185, 0.000214767, 0.000275404, 0.000134214, 0.000213879, 0.000103937, 0.000179247, 0.000237263, 0.000164532, 0.000280563, 0.000245931, 0.000219968, 0.000290965, 0.000232104, 0.000270203, 0.000202631, 0.000214767, 0.000258913, 0.000243352, 0.00018965, 0.000264114, 9.09134E-05, 0.000156752, 0.000201786, 0.000244198, 0.000225169, 0.000203519, 0.000138569, 0.000179247, 0.00019743, 0.000171467, 0.000180981, 0.000168888, 0.000126433, 0.000140303, 0.000134214, 0.000220814, 7.70861E-05, 0.000114297, 0.000134214, 0.000192229, 0.000121232, 0.000125587, 0.000155864, 0.000216501, 0.000136835, 0.000177514, 0.000116919, 0.000200898, 9.43808E-05, 9.00677E-05, 3.72534E-05, 0.000213879, 0.0001299, 0.000192229, 0.000218235, 0.00020872, 0.000176668, 7.88198E-05, 4.33002E-05, 0.00018707, 0.000245086, 0.000149817, 0.000115185, 0.000225169, 5.45481E-05, 0.0002546, 0.000123854, 0.000173201, 9.87362E-05, 0.0002113, 0.000141148, 0.000164532, 0.00016542, 0.000148083, 0.000117765, 0.000158485, 9.78482E-05, 0.000138569, 0.000114297, 0.0002546, 0.00036543, 0.000239885, 0.000144616, 0.000171467, 0.000138569, 0.000269315, 0.000129013, 9.43808E-05, 0.000274516, 0.000152396, 0.000223436, 0.000152396, 0.000265848, 0.000281451, 0.000247665, 0.000321284, 0.000231216, 0.000380187, 0.000245931, 0.000471946, 0.000356804, 0.000375832, 0.000492751, 0.000320396, 0.00046332, 0.000500531, 0.000555967, 0.000553388, 0.000590599, 0.000643413, 0.000582819, 0.000632165, 0.00053863, 0.000593221, 0.000619184, 0.000600113, 0.000683246, 0.00068498, 0.000620918, 0.000612249, 0.000690181, 0.000700583, 0.000650348, 0.000662484, 0.000627852, 0.000632165, 0.000548187, 0.000657283, 0.000572416, 0.000620918, 0.000576729, 0.000618296, 0.0006391, 0.000671153, 0.000604469, 0.00064168, 0.000564636, 0.000514401, 0.000406996, 0.000445983, 0.000390547, 0.000464166, 0.000158485, 0.000435581, 0.000217347, 0.000250287, 0.000192229, 0.000243352, 0.00010047, 7.53524E-05, 0.000222548, 9.35351E-05, 0.000198318, 0.00018965, 0.000234684, 9.70025E-05, 4.85013E-05, 0.000134214, 5.62818E-05, 0.000117765, 0.000141148, 0.000218235, 9.43808E-05, 7.0997E-05, 0, 0.000135102, 0.000114297, 0.000138569, 2.68512E-05, 0.000107362, 0.00012212, 0.000131634, 9.52688E-05, 0.000162798, 0.000173201, 0.000140303, 0.00014377, 9.09134E-05, 7.1885E-05, 2.5794E-06, 7.1885E-05, 0.000109984, 3.72534E-05, 0.000109984, 0.000116031, 0.000171467, 4.9347E-05, 0.000123854, 3.2898E-05, 8.05535E-05, 8.48666E-05, 0.000113452, 0.00016542, 6.49502E-05, 7.70861E-05, 0.000102204, 0.00013248, 9.43808E-05, 0.000109984, 0.000117765, 0.000157597, 0.000150663, 5.62818E-05, 0.000141148, 0.000101316, 0.000135947, 0.000174934, 0.000157597, 5.20109E-06, 7.27307E-05, 0.000220814, 6.5796E-05, 0.000112564, 9.09134E-05, 8.57123E-05, 9.87362E-05, 0, 0.000164532, 0, 8.91797E-05, 0.000121232, 5.80155E-05, 6.93479E-06, 0.000113452, 6.75297E-05, 8.66849E-06, 9.70025E-05, 0.000115185, 0.000107362, 6.5796E-05, 0.000180981, 0.000176668, 0.00012212, 0.000113452, 0.000209566, 0.000133368, 0.000125587, 9.52688E-05, 0.000174046, 0.000147237, 3.98328E-05, 0.00013248, 0, 0.000168888, 4.9347E-05, 0.0001299, 6.06371E-05, 0.000148971, 7.70861E-05, 3.89871E-05, 9.61145E-05, 0.000173201, 2.16501E-05, 1.99164E-05, 0, 0.000126433, 0.000120386, 0.000155018, 0.000141148, 0.000102204, 0.000135102, 0.000142036, 0.000159331, 0.000123854, 9.78482E-05, 0.000103049, 0.000128167, 0.000166266, 0.000160219, 6.66839E-05, 0, 0.000163686, 0, 0.000180981, 0.000117765, 5.97492E-05, 7.0997E-05, 0.000217347, 4.24545E-05, 0.000148083, 0, 0.000247665, 0.000107362, 6.66839E-05, 2.16501E-05, 0, 5.97492E-05, 0.000138569, 4.85013E-05, 0.000107362, 9.00677E-05, 8.31329E-05, 3.46317E-05, 0.000106517, 4.58796E-05, 0.000128167, 2.51175E-05, 0.000103937, 4.07208E-05, 9.52688E-05, 8.40209E-05, 8.7446E-05, 0.000210454, 0.000140303, 0.000167154, 0.000151551, 6.49502E-05, 9.78482E-05, 4.50339E-05, 5.71697E-05, 8.7446E-05, 0.000161953, 0.000127321, 0.000111718, 0.000149817, 0.00010047, 1.29816E-05, 8.31329E-05, 0.000161065, 7.88198E-05, 0.00011083, 8.57123E-05, 0, 0.000113452, 0.000158485, 0.000166266, 0.000235529, 0.000156752, 0.000184449, 0.000201786, 8.40209E-05, 0.000150663, 0.000183603, 0.000267581, 9.18014E-05, 0.00016542, 0.000205253, 0.000176668, 0.000215613, 0.000248553, 0.000224281, 0.000221702, 0.000241618, 0.000185337, 0.000315237, 0.000303947, 0.000245086, 0.000225169, 0.000206099, 0.00021908, 0.000137681, 0.000315237, 0.000163686, 0.000308302, 0.0002979, 0.000265848, 0.000238997, 0.000223436, 0.000264114, 0.000271049, 0.000301367, 0.00027625, 0.000171467, 0.000266736, 0.00018707, 0.000206099, 0.000258913, 5.62818E-05, 0.000183603, 0.000166266, 0.000139415, 0.000120386, 0.00014377, 9.43808E-05, 0.000160219, 0.000241618, 0.000109096, 0.000119498, 8.48666E-05, 1.12479E-05, 0, 0.00016542, 8.05535E-05, 0.000160219, 8.48666E-05, 6.14829E-05, 0.000180981, 0.00018707, 4.9347E-05, 0.0002113, 0.000131634, 4.9347E-05, 0.000166266, 0.000130746, 2.94306E-05, 0.00010047, 1.90707E-05, 0.000158485, 3.3786E-05, 0.000155018, 1.56033E-05, 0.000109984, 4.58796E-05, 0.000174934, 7.88198E-05, 0.000121232, 6.84176E-05, 0.000155018, 0.000119498, 9.70025E-05, 0, 5.62818E-05, 0, 8.31329E-05, 9.43808E-05, 8.05535E-05, 0.000148971, 6.40623E-05, 0.000107362, 0.000149817, 4.85013E-05, 2.76969E-05, 6.40623E-05, 0, 7.44644E-05, 8.05535E-05, 5.0235E-05, 3.98328E-05, 9.70025E-05, 0.000213879, 3.20523E-05, 0.000137681, 6.49502E-05, 5.10807E-05, 4.15665E-05, 0.000126433, 0.000140303, 0.00017578, 5.54361E-05, 0, 1.47153E-05, 3.80991E-05, 7.1885E-05, 2.42295E-05, 0, 0.000116919, 0, 1.81827E-05, 0, 0.000111718, 5.28144E-05, 0.000125587, 4.50339E-05, 0.000114297, 8.40209E-05, 0.000127321, 6.23708E-05, 6.14829E-05, 2.76969E-05, 3.46317E-05, 3.11643E-05, 3.72534E-05, 2.59632E-05, 0.00011083, 4.3131E-06, 6.75297E-05, 0, 1.99164E-05, 0.000101316, 7.27307E-05, 3.89871E-05, 0, 0.000172313, 5.97492E-05, 8.40209E-05, 8.48666E-05, 6.14829E-05, 0.000109096, 0.00012212, 8.48666E-05, 2.16501E-05, 4.07208E-05, 1.38696E-05, 0.000144616, 0, 9.5142E-06, 3.11643E-05, 0.000142036, 0.000244198, 2.59632E-05, 1.38696E-05, 8.7446E-05, 0, 2.76969E-05, 3.03186E-05, 4.3131E-06, 0, 3.89871E-05, 2.51175E-05, 1.04022E-05, 2.59632E-05, 8.31329E-05, 9.35351E-05, 0.000103049, 7.27307E-05, 0.0001299, 0.000116031, 0.000142036, 7.36187E-05, 3.63654E-05, 6.84176E-05, 1.56033E-05, 0, 0, 0.000125587, 6.32165E-05, 0.000130746, 0.000138569, 8.7446E-05, 1.56033E-05, 0, 0, 9.87362E-05, 0.000151551, 1.99164E-05, 8.13992E-05, 6.93479E-06, 5.62818E-05, 0.00011083, 7.7805E-06, 1.12479E-05, 8.31329E-05, 0.000106517, 6.14829E-05, 0.000104783, 0.000184449, 1.99164E-05, 0.000116031, 0, 0.000123854, 0.000128167, 5.10807E-05, 4.41459E-05, 1.47153E-05, 6.66839E-05, 0.00014377, 9.09134E-05, 8.13992E-05, 5.19687E-05, 0.000135102, 4.07208E-05, 9.87362E-05, 1.04022E-05, 8.40209E-05, 0.000128167, 0, 3.3786E-05, 9.43808E-05, 0, 5.62818E-05, 0, 0.000120386, 0.000119498, 0.000196584, 9.18014E-05, 0.000103937, 9.35351E-05, 0.000109096, 0.00012212, 1.56033E-05, 0.000142036, 0.000229482, 0, 0.00016542, 5.10807E-05, 7.0997E-05, 0.000103049, 5.37024E-05, 5.45481E-05, 0.000113452, 0.000116031, 9.26471E-05, 1.81827E-05, 5.0235E-05, 0, 8.66003E-05, 2.08044E-05, 0.000135947, 5.54361E-05, 5.28144E-05, 8.7446E-05, 0, 0, 5.97492E-05, 8.91797E-05, 2.16501E-05, 3.89871E-05, 0.000160219, 0, 0.000114297, 4.50339E-05, 0.000174046, 7.1885E-05, 0.00015413, 5.37024E-05, 0.000148971, 0, 9.70025E-05, 0, 2.94306E-05, 5.89034E-05, 0, 9.35351E-05, 0, 9.26471E-05, 0, 0.000118653, 7.36187E-05, 5.54361E-05, 3.55197E-05, 7.7805E-06, 6.06371E-05, 2.16501E-05, 0.000103937, 0.000116031, 0.000114297, 3.89871E-05, 0.000111718, 0, 0.000183603, 0.000135947, 0, 3.98328E-05, 7.70861E-05, 0, 0.00012212, 0.000109096, 9.61145E-05, 2.68512E-05, 0.00010047, 4.76133E-05, 7.0997E-05, 5.54361E-05, 8.66003E-05, 6.66839E-05, 0.000102204, 0.000101316, 0.00010047, 0, 6.23708E-05, 0, 8.66003E-05, 9.00677E-05, 0.000127321, 6.5796E-05, 0.000170579, 0, 9.26471E-05, 0.000136835, 4.41459E-05, 3.55197E-05, 0.000160219, 1.7337E-05, 0.000125587, 7.0997E-05, 7.1885E-05, 0.000137681, 3.3786E-05, 8.66003E-05, 7.61981E-05, 0.000106517, 0, 7.96655E-05, 0.000160219, 8.66003E-05, 0.000229482, 4.9347E-05, 9.52688E-05, 0.000124699, 2.85849E-05, 0.000187916, 2.68512E-05, 0.000117765, 0.000103049, 0.000116031, 0.00012212, 0.000122966, 0.000155864, 0.000103049, 6.49502E-05, 0.000123854, 0.000193117, 0, 0.000187916, 0, 6.40623E-05, 0, 8.13992E-05, 6.06371E-05, 8.8334E-05, 0.000113452, 0.000173201, 0, 5.19687E-05, 0.000140303, 0.000148971, 0, 9.52688E-05, 0, 0.000173201, 1.6449E-05, 0.000114297, 0, 9.09134E-05, 8.40209E-05, 2.85849E-05, 6.66839E-05, 9.78482E-05, 0.000135102, 7.79318E-05, 0.000194851, 0.000121232, 2.94306E-05, 7.01513E-05, 8.91797E-05, 3.3786E-05, 0.0001299, 8.91797E-05, 1.21359E-05, 8.40209E-05, 8.7446E-05, 0.000105671, 9.52688E-05, 0.000191383, 9.61145E-05, 0.000229482, 0.000144616, 9.18014E-05, 0.000144616, 7.1885E-05, 2.5794E-06, 6.14829E-05, 2.94306E-05, 0.000210454, 3.20523E-05, 9.70025E-05, 0.000124699, 9.52688E-05, 8.40209E-05, 0.000199164, 6.75297E-05, 0.000129013, 0.000155018, 9.09134E-05, 0.000156752, 0.000116919, 8.57123E-05, 6.75297E-05, 9.52688E-05, 6.92633E-05, 9.5142E-06, 5.54361E-05, 0.000149817, 7.96655E-05, 2.5794E-06, 0.00017578, 0.000112564, 3.89871E-05, 7.88198E-05, 4.15665E-05, 4.3131E-06, 2.16501E-05, 5.71697E-05, 8.8334E-05, 0, 0.000128167, 0.000103049, 0.000137681, 7.96655E-05, 2.08044E-05, 0.000112564, 0.000155864, 6.66839E-05, 0.000109984, 4.41459E-05, 1.7337E-06, 5.97492E-05, 0.000184449, 0, 0.000223436, 9.78482E-05, 0.000173201, 3.03186E-05, 5.19687E-05, 1.7337E-06, 0.000136835, 3.4674E-06, 0.000142882, 6.84176E-05, 0.000127321, 0.000116919, 0.000226015, 1.29816E-05, 3.46317E-05, 4.15665E-05, 5.0235E-05, 4.76133E-05, 6.66839E-05, 2.76969E-05, 0.000116919, 0, 0.000139415, 1.56033E-05, 0.000113452, 0.000131634, 0.00011083, 0, 0.00023295, 0, 0.000116919, 9.18014E-05, 6.40623E-05, 1.90707E-05, 0.000122966, 0.000105671, 0.000116919, 4.3131E-06, 6.32165E-05, 6.32165E-05, 0.000101316, 8.48666E-05, 7.27307E-05, 6.75297E-05, 0.00010047, 0.000128167, 8.31329E-05, 0, 0, 0, 0.00010825, 1.6449E-05, 0, 0.000123854, 0.000206099, 8.8334E-05, 9.26471E-05, 4.58796E-05, 7.53524E-05, 0.000141148, 1.81827E-05, 9.18014E-05, 8.57123E-05, 2.42295E-05, 8.31329E-05, 8.05535E-05, 4.41459E-05, 4.07208E-05, 3.11643E-05, 0.000103937, 7.44644E-05, 9.5142E-06, 0.000161953, 1.47153E-05, 8.40209E-05, 4.9347E-05, 2.33838E-05, 1.38696E-05, 4.41459E-05, 7.1885E-05, 0.000117765, 0.000125587, 2.68512E-05, 7.7805E-06, 0, 8.40209E-05, 4.9347E-05, 0.000151551, 3.3786E-05, 0, 6.75297E-05, 8.8334E-05, 0.000145504, 0, 4.24545E-05, 2.68512E-05, 0.000181869, 8.48666E-05, 6.0468E-06, 0, 0.000118653, 0, 6.32165E-05, 0.000112564, 9.78482E-05, 0, 4.9347E-05, 7.88198E-05, 8.48666E-05, 3.11643E-05, 4.24545E-05, 7.7805E-06, 8.40209E-05, 0, 0, 3.80991E-05, 0.000116031, 8.31329E-05, 3.03186E-05, 0.000118653, 2.85849E-05, 8.7446E-05, 8.05535E-05, 5.89034E-05, 0, 0, 0.000199164, 9.78482E-05, 0, 0, 0, 3.72534E-05, 4.76133E-05, 8.57123E-05, 8.40209E-05, 6.66839E-05, 0.000140303, 0.000172313, 7.0997E-05, 0, 0, 0.000163686, 3.3786E-05, 0, 9.61145E-05, 0, 0.000114297, 3.20523E-05, 0.00017578, 0.00010047, 0.000203519, 0.000127321, 4.33002E-05, 0.000173201, 0.000127321, 0.000206987, 5.80155E-05, 5.80155E-05, 4.58796E-05, 0.000171467, 0.000248553, 0.000167154, 0.000232104, 0.000188804, 0.000269315, 0.000155018, 0.00014377, 0.000193963, 0.000201786, 0.000155864, 0.000221702, 7.27307E-05, 0.000219968, 0.000257179, 0.000193963, 0.000285764, 0.000258067, 0.000168, 0.000148971, 0.000169733, 0.000299634, 0.000323863, 0.000273671, 0.000193117, 0.00028403, 0.00028403, 0.000255488, 0.000161953, 0.000202631, 0.000193963, 0.000260647, 0.000187916, 0.000255488, 0.000164532, 0.000242464, 0.000184449, 0.000200898, 0.000130746, 0.000171467, 0.000145504, 0.000128167, 9.26471E-05, 0.000109984, 3.63654E-05, 8.8334E-05, 3.89871E-05, 4.24545E-05, 2.59632E-05, 0, 2.16501E-05, 5.62818E-05, 4.33002E-05, 0, 3.3786E-05, 8.22872E-05, 1.56033E-05, 0.000180135, 0, 4.85013E-05, 8.05535E-05, 0.000190538, 1.7337E-06, 0, 4.24545E-05, 1.6449E-05, 5.54361E-05, 7.01513E-05, 0, 6.92633E-05, 2.59632E-05, 0, 4.15665E-05, 0.000113452, 4.58796E-05, 0, 4.85013E-05, 4.50339E-05, 5.62818E-05, 3.63654E-05, 9.26471E-05, 6.5796E-05, 1.81827E-05, 8.7446E-05, 0.000123854, 0, 0.000102204, 0, 5.10807E-05, 1.38696E-05, 0, 7.70861E-05, 0, 0.000129013, 4.9347E-05, 0.000114297, 0, 0.000119498, 0.000115185, 0, 0, 0.000133368, 1.7337E-06, 0.00012212, 4.41459E-05, 0, 5.0235E-05, 8.66849E-06, 5.37024E-05, 2.59632E-05, 0, 0.000161953, 0, 0, 0, 6.93479E-06, 2.94306E-05, 6.0468E-06, 3.03186E-05, 0.000104783, 1.12479E-05, 3.46317E-05, 0, 3.55197E-05, 1.56033E-05, 0.000102204, 3.89871E-05, 7.1885E-05, 0.000187916, 0.000156752, 5.0235E-05, 0.000115185, 0, 0.00014377, 0, 8.31329E-05, 5.19687E-05, 4.3131E-06, 0.000107362, 0, 0, 0.000228637, 5.80155E-05, 6.40623E-05, 0, 4.3131E-06, 0, 2.59632E-05, 0, 1.47153E-05, 0, 2.33838E-05, 0, 6.23708E-05, 0, 3.03186E-05, 6.66839E-05, 8.13992E-05, 0, 0, 5.0235E-05, 8.05535E-05, 0, 0.00019743, 0, 0, 5.10807E-05, 3.98328E-05, 3.98328E-05, 3.98328E-05, 3.98328E-05, 3.98328E-05, 3.98328E-05]
input_halogen_mwcmsq = [0, 0, 5.58986E-06, 9.80677E-08, 1.05455E-05, 0, 3.85079E-06, 0, 4.2496E-06, 0, 4.923E-06, 0, 0, 0, 2.43339E-05, 4.34832E-05, 3.93317E-05, 3.07606E-05, 2.56741E-05, 4.05346E-05, 4.81708E-05, 4.1208E-05, 3.51801E-05, 3.87956E-05, 4.094E-05, 4.05346E-05, 5.24531E-05, 4.21429E-05, 4.44181E-05, 3.59843E-05, 4.16068E-05, 3.81287E-05, 2.71451E-05, 4.2411E-05, 4.08027E-05, 2.78185E-05, 2.92895E-05, 3.65204E-05, 4.70986E-05, 5.47283E-05, 5.15182E-05, 4.97726E-05, 4.5353E-05, 4.29471E-05, 4.50915E-05, 4.28098E-05, 4.65625E-05, 5.20478E-05, 5.75396E-05, 3.94624E-05, 5.312E-05, 3.94624E-05, 4.22737E-05, 4.96418E-05, 4.72294E-05, 5.15182E-05, 4.87004E-05, 4.84389E-05, 6.82551E-05, 3.65204E-05, 6.41036E-05, 4.99099E-05, 7.49498E-05, 5.60685E-05, 5.71408E-05, 4.96418E-05, 6.16911E-05, 6.42343E-05, 7.1864E-05, 5.78076E-05, 5.66047E-05, 3.85275E-05, 7.38776E-05, 7.45445E-05, 6.62414E-05, 4.61572E-05, 6.70456E-05, 5.39241E-05, 7.11971E-05, 6.15603E-05, 9.23536E-05, 7.41457E-05, 6.35675E-05, 6.15603E-05, 7.49498E-05, 8.36517E-05, 7.24001E-05, 6.43716E-05, 7.02622E-05, 6.94581E-05, 8.60576E-05, 6.15603E-05, 6.27633E-05, 7.53486E-05, 8.49854E-05, 9.46287E-05, 6.47704E-05, 7.30735E-05, 9.36938E-05, 6.51692E-05, 8.84701E-05, 8.37825E-05, 8.49854E-05, 9.7172E-05, 9.78454E-05, 7.89641E-05, 9.78454E-05, 8.05724E-05, 8.96731E-05, 0.000100657, 0.000106547, 9.91791E-05, 9.32885E-05, 9.98525E-05, 0.000110967, 0.000105606, 0.000110163, 0.000112301, 0.000107619, 0.000102258, 0.000113778, 0.000111235, 0.00011538, 0.000127168, 0.000133457, 0.000124088, 0.000128769, 0.000122075, 0.000126631, 0.000136268, 0.000128501, 0.000144839, 0.000125023, 0.000119531, 0.000156222, 0.000134797, 0.00013734, 0.000155548, 0.000158092, 0.000142832, 0.000158765, 0.000147252, 0.000168134, 0.00015073, 0.000174292, 0.000164388, 0.000169075, 0.000186348, 0.000189826, 0.000176169, 0.000206432, 0.000176169, 0.000180856, 0.000186616, 0.000197724, 0.000196521, 0.000228386, 0.000201209, 0.000209512, 0.000203752, 0.00024713, 0.000210179, 0.000231603, 0.000221561, 0.000237356, 0.000227582, 0.000243652, 0.00024124, 0.000263599, 0.000255035, 0.000268692, 0.00025865, 0.000273909, 0.000278466, 0.000273105, 0.000266679, 0.000296942, 0.000284756, 0.000297073, 0.000291856, 0.000290646, 0.000302696, 0.00030618, 0.000316222, 0.000323584, 0.000327873, 0.00031823, 0.000329475, 0.000325996, 0.000338046, 0.000350631, 0.000339118, 0.000365361, 0.000360405, 0.000358797, 0.000356254, 0.000382765, 0.000371251, 0.000382098, 0.000395082, 0.000398697, 0.000399907, 0.000397364, 0.000399103, 0.000411819, 0.00041905, 0.000451052, 0.000421462, 0.000423875, 0.000435924, 0.000453059, 0.000440873, 0.000463102, 0.000441278, 0.000468325, 0.000472608, 0.000473412, 0.000495909, 0.000485461, 0.000487736, 0.000502067, 0.000505415, 0.000516529, 0.000525362, 0.000532194, 0.00052965, 0.000531925, 0.00057008, 0.000541294, 0.000541294, 0.000551604, 0.000554285, 0.000586817, 0.000585882, 0.000580927, 0.000590701, 0.000587353, 0.000622703, 0.000629261, 0.000620696, 0.000601416, 0.000645331, 0.000635688, 0.000628993, 0.000655236, 0.000656445, 0.000680681, 0.000680812, 0.000685231, 0.000686702, 0.000688978, 0.000706787, 0.000729283, 0.000706388, 0.000748295, 0.000707859, 0.000748563, 0.000741601, 0.000776415, 0.000741463, 0.000777082, 0.000791406, 0.000788196, 0.000803861, 0.000807339, 0.000813367, 0.00082488, 0.000828763, 0.000840008, 0.000832784, 0.000870403, 0.00086023, 0.000886604, 0.000892233, 0.000887676, 0.00092517, 0.000924902, 0.000933068, 0.000907224, 0.000956232, 0.000960782, 0.000961717, 0.000976584, 0.00094753, 0.00099198, 0.000994256, 0.001025585, 0.001049422, 0.001062007, 0.001058255, 0.001054508, 0.001048886, 0.00107352, 0.001077267, 0.001134309, 0.001123058, 0.001128419, 0.001138729, 0.001164972, 0.001178093, 0.001163632, 0.001207278, 0.001222544, 0.001198576, 0.001262445, 0.001244635, 0.001282529, 0.001272219, 0.001262843, 0.001281855, 0.001310648, 0.00134184, 0.001325103, 0.001339565, 0.001360185, 0.001389913, 0.001377458, 0.001384153, 0.001399419, 0.001441863, 0.00145284, 0.001462614, 0.001458331, 0.001485777, 0.001548972, 0.001529293, 0.001547233, 0.00155942, 0.00158901, 0.001590618, 0.001616456, 0.001620738, 0.001617528, 0.001646719, 0.001635736, 0.001666398, 0.001712725, 0.001758386, 0.001721427, 0.00173294, 0.001747271, 0.00175075, 0.001796541, 0.001812879, 0.001818769, 0.001821718, 0.001829478, 0.001880094, 0.001899374, 0.001882232, 0.00193258, 0.001946774, 0.001980646, 0.001934182, 0.002009302, 0.002001397, 0.002020141, 0.002017735, 0.002063657, 0.002055361, 0.002069417, 0.002086422, 0.002169034, 0.002154703, 0.002164615, 0.002171041, 0.00220679, 0.002164883, 0.002235844, 0.00229141, 0.002265971, 0.002271325, 0.002302929, 0.002316181, 0.002314037, 0.002315645, 0.002395446, 0.00235501, 0.002374159, 0.002376702, 0.002439361, 0.002474443, 0.00250778, 0.002497607, 0.002493592, 0.002524922, 0.002540586, 0.002543797, 0.002621858, 0.002619184, 0.002597492, 0.002656672, 0.002632306, 0.00268064, 0.00267796, 0.002730177, 0.002750131, 0.002731518, 0.002816941, 0.002808775, 0.002844923, 0.002856037, 0.00288228, 0.002842517, 0.00295351, 0.002976138, 0.00298618, 0.003001982, 0.002998235, 0.003036122, 0.003006264, 0.003035455, 0.003109902, 0.003137478, 0.003151404, 0.003047237, 0.003150064, 0.003179523, 0.003234552, 0.00327914, 0.003253434, 0.003297754, 0.003322388, 0.003307796, 0.003323997, 0.003338589, 0.003356666, 0.003343545, 0.003405935, 0.003379026, 0.003482657, 0.003521623, 0.003543041, 0.003531665, 0.003506755, 0.003509835, 0.003551076, 0.003692064, 0.003548402, 0.003609053, 0.003676936, 0.003701838, 0.003772401, 0.003676269, 0.003668364, 0.003710945, 0.003682159, 0.003750042, 0.003809758, 0.003777089, 0.003846442, 0.003875228, 0.003855412, 0.003882459, 0.003953696, 0.003870946, 0.003986228, 0.003987967, 0.003918077, 0.003973911, 0.004038178, 0.004050632, 0.004063616, 0.004053045, 0.004033627, 0.004083701, 0.004102183, 0.004127354, 0.004146765, 0.00405411, 0.004153329, 0.004217596, 0.004160154, 0.004242099, 0.004233796, 0.004270886, 0.004262988, 0.004257097, 0.004385899, 0.004315336, 0.004328863, 0.004314401, 0.004480698, 0.004382421, 0.004442935, 0.004471192, 0.004414149, 0.004471192, 0.004564376, 0.0044823, 0.00454858, 0.004542958, 0.004610572, 0.004595176, 0.004624897, 0.004651813, 0.004645249, 0.004700147, 0.004681534, 0.004762943, 0.004715145, 0.004764552, 0.004806459, 0.004723442, 0.004794541, 0.0048169, 0.00481556, 0.004824399, 0.004817436, 0.004841267, 0.005029387, 0.004992304, 0.005036219, 0.005023228, 0.004984132, 0.005019881, 0.004967663, 0.005044254, 0.005111195, 0.005193676, 0.005218579, 0.00514748, 0.005248038, 0.005197422, 0.005232642, 0.005238532, 0.005401076, 0.005388085, 0.005331853, 0.005341091, 0.005454366, 0.005490651, 0.005479406, 0.005423167, 0.005584646, 0.005604325, 0.00558545, 0.005573126, 0.005813333, 0.005770889, 0.005818955, 0.005676496, 0.005893265, 0.00575884, 0.005906654, 0.00592634, 0.005967979, 0.005996497, 0.005910269, 0.006185153, 0.006104417, 0.006095715, 0.006198412, 0.006247282, 0.006332836, 0.006347429, 0.006320388, 0.006455349, 0.006447183, 0.006627131, 0.006612408, 0.006552018, 0.006668104, 0.006671987, 0.006702382, 0.006746434, 0.006850601, 0.00692036, 0.007043409, 0.00700779, 0.007104864, 0.007029882, 0.00702252, 0.007293115, 0.00728642, 0.007396347, 0.007469722, 0.007558094, 0.007517657, 0.007554877, 0.007687837, 0.007679129, 0.00790983, 0.007837927, 0.007746076, 0.007896172, 0.007867118, 0.008125932, 0.00821042, 0.008330252, 0.008482891, 0.008345649, 0.00847553, 0.008490253, 0.00866686, 0.008797407, 0.00875416, 0.008812399, 0.009041092, 0.009017255, 0.009110851, 0.009189711, 0.009304862, 0.00948294, 0.009625131, 0.009722872, 0.009727291, 0.00968458, 0.009766924, 0.009976599, 0.010085323, 0.010257909, 0.010180253, 0.010415367, 0.010567738, 0.010614738, 0.010722652, 0.010765769, 0.010728275, 0.010759878, 0.01105176, 0.010940898, 0.011263985, 0.011260233, 0.011602731, 0.011724439, 0.011716273, 0.011625626, 0.011871855, 0.01191216, 0.01187226, 0.012170974, 0.012571312, 0.012484549, 0.012430193, 0.012484549, 0.012604387, 0.012740956, 0.012867751, 0.013069666, 0.013253634, 0.01335365, 0.013472946, 0.013638576, 0.01378318, 0.013574577, 0.013890296, 0.014018026, 0.014178432, 0.014186735, 0.014368291, 0.014529233, 0.014706507, 0.014660716, 0.01477144, 0.014932383, 0.014977638, 0.015263766, 0.015426317, 0.015537846, 0.015622602, 0.015856108, 0.015567972, 0.015915426, 0.015676696, 0.016034454, 0.016188701, 0.016120551, 0.016113451, 0.01659627, 0.016545929, 0.016759082, 0.017038117, 0.016583953, 0.016717175, 0.017511163, 0.017300148, 0.01725556, 0.017203342, 0.017461489, 0.017509156, 0.017754979, 0.017952605, 0.017928108, 0.018359644, 0.017849908, 0.01825092, 0.01820473, 0.018330584, 0.018180226, 0.018843262, 0.018686072, 0.018653802, 0.018671343, 0.019307069, 0.019141708, 0.019116269, 0.019372945, 0.019392225, 0.019506572, 0.019147605, 0.019343753, 0.019816531, 0.019783999, 0.01982724, 0.020223296, 0.019542589, 0.020182859, 0.020045754, 0.020378079, 0.020272971, 0.02040325, 0.020673047, 0.020391332, 0.020656709, 0.02049376, 0.020582668, 0.020978051, 0.021013532, 0.020632611, 0.021293639, 0.021140196, 0.02105183, 0.021179828, 0.021166171, 0.021503046, 0.021348269, 0.021553126, 0.021928961, 0.021514429, 0.021567313, 0.02186523, 0.022007827, 0.021633457, 0.021937394, 0.021941147, 0.022230754, 0.021972476, 0.022155373, 0.02238085, 0.022252447, 0.022540046, 0.022723753, 0.022482206, 0.022379778, 0.022917496, 0.022663899, 0.02301912, 0.02279699, 0.022630027, 0.023116991, 0.023350235, 0.023146451, 0.023219289, 0.023358807, 0.023264413, 0.023340193, 0.023443962, 0.023317298, 0.023595123, 0.02393923, 0.023492427, 0.023791814, 0.023786192, 0.023839214, 0.023661803, 0.02393789, 0.023832519, 0.024362601, 0.024145826, 0.024384156, 0.024323367, 0.024134312, 0.024526485, 0.024203934, 0.024606417, 0.024789182, 0.024685283, 0.024657968, 0.024563306, 0.024813549, 0.024640695, 0.025050676, 0.025203851, 0.025207467, 0.024948784, 0.025379517, 0.025033135, 0.025524794, 0.025290615, 0.025435082, 0.025417946, 0.025600306, 0.025378176, 0.025897948, 0.025475518, 0.025675825, 0.026047109, 0.025892326, 0.026051529, 0.026099994, 0.026013498, 0.026139626, 0.026330557, 0.026045638, 0.026562062, 0.026257451, 0.026566482, 0.026645609, 0.026573575, 0.026144183, 0.026861849, 0.026576655, 0.026611868, 0.026988107, 0.026607049, 0.02683641, 0.026879788, 0.027039658, 0.027041267, 0.027066032, 0.026927319, 0.02737586, 0.027322172, 0.02732967, 0.027205281, 0.027346407, 0.027756651, 0.027617408, 0.027780756, 0.027832306, 0.027861093, 0.027834712, 0.027693188, 0.027970484, 0.027756651, 0.028267858, 0.02792536, 0.028059921, 0.028350202, 0.02832811, 0.02789269, 0.028103169, 0.028747062, 0.028090179, 0.028886848, 0.02819328, 0.028722558, 0.028860069, 0.028837304, 0.028651864, 0.028574071, 0.028871046, 0.029005209, 0.029170028, 0.029238852, 0.029006412, 0.029334585, 0.029234295, 0.029423356, 0.029162529, 0.02950302, 0.029693147, 0.029196271, 0.029909654, 0.029256125, 0.029750052, 0.029573851, 0.030080233, 0.02981968, 0.029892113, 0.02965847, 0.029906169, 0.030227517, 0.030427549, 0.030375737, 0.030163381, 0.030300218, 0.030470261, 0.030312137, 0.030591303, 0.030807542, 0.030590904, 0.030918672, 0.030732023, 0.031223146, 0.030753716, 0.030682754, 0.031242021, 0.031119109, 0.031237065, 0.031281385, 0.031282457, 0.031250991, 0.031568586, 0.031565376, 0.031527751, 0.031325038, 0.03242711, 0.031462411, 0.032014585, 0.031800889, 0.032086756, 0.031915236, 0.031916308, 0.031972677, 0.032313168, 0.032075105, 0.032249431, 0.032426972, 0.032691415, 0.032424429, 0.032928536, 0.032632365, 0.032929477, 0.032847133, 0.03319445, 0.033323389, 0.033048237, 0.033085732, 0.033450452, 0.033117728, 0.03349504, 0.033497185, 0.033256442, 0.033737117, 0.033506959, 0.033735646, 0.033872483, 0.034088317, 0.034010392, 0.033997676, 0.033619429, 0.034253142, 0.03442667, 0.033930997, 0.034166647, 0.034699409, 0.034360794, 0.034276574, 0.034340174, 0.034325843, 0.034974155, 0.034342044, 0.034476076, 0.034502718, 0.034904128, 0.035140184, 0.034886993, 0.034751757, 0.034631657, 0.035095596, 0.034943225, 0.035322544, 0.035185707, 0.035197089, 0.035085822, 0.034963178, 0.035089169, 0.035292686, 0.035020352, 0.035004282, 0.035401142, 0.034896499, 0.035597029, 0.035160667, 0.035598232, 0.035058644, 0.035348924, 0.035110057, 0.035573329, 0.035249438, 0.035575467, 0.03525185, 0.035438898, 0.035632908, 0.035408373, 0.035860798, 0.035290012, 0.035961213, 0.03557212, 0.035607738, 0.035608137, 0.03569035, 0.035647769, 0.035642951, 0.035858922, 0.035739756, 0.035935238, 0.036107157, 0.035795054, 0.035227614, 0.036127647, 0.03610381, 0.035920109, 0.03617317, 0.035846872, 0.036172496, 0.036193921, 0.036051592, 0.0365083, 0.035949837, 0.036371332, 0.036285908, 0.036171293, 0.036590644, 0.036085602, 0.03638244, 0.036496518, 0.036177851, 0.036446981, 0.036579131, 0.036310007, 0.036665626, 0.036443366, 0.036307594, 0.036477506, 0.036700172, 0.036645411, 0.03654941, 0.036722532, 0.036632421, 0.036970368, 0.036478579, 0.036915607, 0.037038251, 0.036530129, 0.036780241, 0.036451394, 0.036906095, 0.037085114, 0.036876236, 0.036830177, 0.036929396, 0.036780372, 0.037277516, 0.036778097, 0.03701308, 0.037010805, 0.036894582, 0.037024462, 0.036961261, 0.036825228, 0.03699072, 0.037135056, 0.036888292, 0.037255693, 0.037159423, 0.037215524, 0.037041461, 0.037018297, 0.03729492, 0.037383828, 0.037142954, 0.037558957, 0.037169066, 0.037563913, 0.037305498, 0.037485858, 0.037078419, 0.037634077, 0.037140143, 0.03751397, 0.037639294, 0.037103858, 0.037782826, 0.037817777, 0.037766763, 0.037312598, 0.03739253, 0.037692185, 0.03745479, 0.037680803, 0.037487727, 0.038130953, 0.03774962, 0.037819111, 0.03774065, 0.037680136, 0.037891949, 0.037541821, 0.038084358, 0.037957556, 0.038282115, 0.037572752, 0.037825407, 0.03789731, 0.037889811, 0.037892086, 0.038052623, 0.038074447, 0.0378471, 0.03832108, 0.03768281, 0.038236729, 0.037827551, 0.037957426, 0.038409178, 0.03801152, 0.038082482, 0.038431812, 0.038105776, 0.038184106, 0.038215036, 0.038548832, 0.038277963, 0.038269529, 0.038129744, 0.038115419, 0.038301931, 0.038102697, 0.038569185, 0.038378652, 0.038295641, 0.038685401, 0.038441984, 0.038717267, 0.038434355, 0.0385021, 0.038580698, 0.038290686, 0.038271, 0.038354953, 0.038815949, 0.03831398, 0.038527944, 0.038568917, 0.038516294, 0.038901235, 0.038180092, 0.039034463, 0.038306886, 0.038838577, 0.038392578, 0.038746321, 0.038661166, 0.039057627, 0.038528879, 0.038662375, 0.03859837, 0.038885976, 0.038643494, 0.038754225, 0.038773237, 0.038675091, 0.038716999, 0.039103013, 0.038876869, 0.038789438, 0.038893468, 0.038454439, 0.039147601, 0.038700399, 0.039272788, 0.038699458, 0.039131936, 0.038888787, 0.039056418, 0.039031515, 0.038797205, 0.039267701, 0.039165005, 0.038788098, 0.038680047, 0.039219903, 0.039032182, 0.038847278, 0.038955329, 0.039250029, 0.038959344, 0.039079182, 0.039046513, 0.038922124, 0.039307203, 0.038846612, 0.039314832, 0.039427571, 0.039384591, 0.039084138, 0.039174916, 0.039394764, 0.039268643, 0.039197812, 0.039165142, 0.039660678, 0.039023349, 0.039348973, 0.039633095, 0.039455821, 0.039348705, 0.039642065, 0.039431853, 0.039236234, 0.039499206, 0.039195536, 0.039823627, 0.039511791, 0.039733385, 0.039711555, 0.039487288, 0.039844515, 0.039289256, 0.039464523, 0.039555575, 0.039908116, 0.039649433, 0.039548345, 0.039679291, 0.039477912, 0.039835009, 0.039970912, 0.039447786, 0.039680631, 0.039482064, 0.040099446, 0.039811172, 0.040222089, 0.039636847, 0.040176703, 0.03963792, 0.040194643, 0.039811578, 0.039948951, 0.040098641, 0.040063161, 0.039822823, 0.039790421, 0.039778235, 0.040024332, 0.040183398, 0.039542317, 0.039915209, 0.039977737, 0.039733653, 0.040141758, 0.040001836, 0.04003799, 0.040124754, 0.040306447, 0.040270024, 0.040061958, 0.040064363, 0.040358795, 0.040330545, 0.03998604, 0.040441675, 0.040339116, 0.04024231, 0.040460557, 0.040066776, 0.040638229, 0.039992061, 0.040473142, 0.040164654, 0.040404854, 0.040217002, 0.040524288, 0.040283283, 0.040533258, 0.04044516, 0.040639439, 0.040047228, 0.040408071, 0.04045734, 0.040593778, 0.040096765, 0.040346478, 0.040594981, 0.040366431, 0.040209504, 0.040505544, 0.04037888, 0.040468723, 0.040395348, 0.040611051, 0.040405796, 0.040629259, 0.040419715, 0.040690983, 0.039854688, 0.04066595, 0.040167197, 0.040402716, 0.040159293, 0.040530178, 0.040340051, 0.040514651, 0.040182188, 0.040571818, 0.040507819, 0.040296267, 0.039697898, 0.040494835, 0.04020817, 0.040173356, 0.040342594, 0.040069051, 0.040503667, 0.040209373, 0.04011819, 0.039797515, 0.040192767, 0.040140686, 0.040209504, 0.040323713, 0.039973723, 0.040132514, 0.04000398, 0.040327466, 0.040031563, 0.040116849, 0.039736464, 0.040141758, 0.039894994, 0.040118327, 0.040189419, 0.040060748, 0.039816527, 0.040019913, 0.04033282, 0.040061552, 0.039613278, 0.040206562, 0.040082578, 0.039837016, 0.040071464, 0.040065972, 0.039842103, 0.040068116, 0.040030092, 0.040020717, 0.039936902, 0.039861383, 0.039825503, 0.039966087, 0.039753868, 0.03980689, 0.039818939, 0.040153808, 0.039822954, 0.039674871, 0.03975936, 0.039539107, 0.039622922, 0.039785603, 0.039809969, 0.039817599, 0.039912398, 0.039534151, 0.039704998, 0.039452474, 0.039613416, 0.039338395, 0.039757216, 0.039792298, 0.039703121, 0.039701245, 0.039603106, 0.039531738, 0.039365442, 0.03978306, 0.039450198, 0.039545128, 0.039898204, 0.039170895, 0.03960083, 0.039482469, 0.039357812, 0.039592795, 0.039334786, 0.039490367, 0.039559054, 0.039469616, 0.039295552, 0.039339336, 0.039203702, 0.03959815, 0.03901451, 0.039352588, 0.039363971, 0.039722271, 0.038835229, 0.03924829, 0.039264086, 0.039221106, 0.038933238, 0.039120018, 0.039428774, 0.039092971, 0.03923798, 0.038900431, 0.039070611, 0.039488223, 0.038970863, 0.038961089, 0.038658086, 0.039132871, 0.039064054, 0.038766405, 0.038968588, 0.038814341, 0.039098057, 0.03910047, 0.038784214, 0.038912755, 0.038791308, 0.038813001, 0.038466083, 0.0389406, 0.038603724, 0.038765601, 0.038348931, 0.038841519, 0.038587125, 0.038548296, 0.038196423, 0.038243286, 0.038529147, 0.038278905, 0.03832429, 0.038410786, 0.03864269, 0.038356692, 0.038247039, 0.038253198, 0.038377717, 0.038285462, 0.038314248, 0.037978046, 0.038203654, 0.038153581, 0.038182497, 0.038369951, 0.038108724, 0.038117962, 0.038473987, 0.038289614, 0.0377926, 0.037753105, 0.038303807, 0.038049145, 0.03781322, 0.037996921, 0.037761271, 0.037914577, 0.037723914, 0.037738513, 0.037636352, 0.038105246, 0.037656704, 0.037821255, 0.037727797, 0.037650677, 0.037918728, 0.037647061, 0.037480497, 0.038029728, 0.03740418, 0.037692048, 0.037731014, 0.037684013, 0.037466041, 0.037453319, 0.037659647, 0.037003842, 0.037607566, 0.037507413, 0.036588637, 0.037459745, 0.037156344, 0.037319424, 0.037268814, 0.03659386, 0.037495494, 0.037172943, 0.03697331, 0.03713318, 0.03679992, 0.037206417, 0.036689189, 0.036722126, 0.03717884, 0.036848928, 0.036591984, 0.037062081, 0.036705259, 0.036200348, 0.037059675, 0.036562263, 0.036822548, 0.036402524, 0.036474695, 0.036538962, 0.036509372, 0.03625163, 0.036670713, 0.0364281, 0.036442692, 0.036198739, 0.036352581, 0.036184683, 0.036280685, 0.035893468, 0.036328352, 0.036141703, 0.035766535, 0.036285908, 0.035886237, 0.035833078, 0.035875253, 0.035655536, 0.035995896, 0.03570387, 0.035580828, 0.03604731, 0.035674417, 0.03556583, 0.035150226, 0.035832947, 0.03540141, 0.035494731, 0.03529925, 0.03540596, 0.035189858, 0.035310364, 0.035283716, 0.035130004, 0.034864901, 0.035027981, 0.034795541, 0.035030126, 0.034804649, 0.035106841, 0.034601936, 0.034876545, 0.034617196, 0.034512093, 0.034799026, 0.034444073, 0.034400826, 0.03467718, 0.034420374, 0.034376054, 0.034461078, 0.034218864, 0.034501247, 0.034128891, 0.034173341, 0.034015616, 0.03420896, 0.033980939, 0.034132905, 0.034240289, 0.034110683, 0.033455408, 0.033557568, 0.033884669, 0.033431309, 0.033985888, 0.033508162, 0.03374997, 0.033760012, 0.033279742, 0.033126698, 0.033455538, 0.03329393, 0.033193109, 0.033126162, 0.033216005, 0.033080103, 0.03325095, 0.03269369, 0.033183603, 0.032656202, 0.03277523, 0.032788221, 0.032641335, 0.032466206, 0.032667715, 0.032501687, 0.032198958, 0.032533552, 0.032580147, 0.032291345, 0.032467546, 0.032302858, 0.032220108, 0.032071758, 0.032251712, 0.032179005, 0.03235146, 0.031660704, 0.032152095, 0.031619601, 0.031790585, 0.032126251, 0.031407245, 0.031405369, 0.031716943, 0.031372836, 0.031244564, 0.03149414, 0.031406578, 0.031224349, 0.031412469, 0.031163429, 0.030980128, 0.03104667, 0.031123261, 0.030846906, 0.031159814, 0.030937017, 0.030824409, 0.030705381, 0.030753448, 0.030423398, 0.030855471, 0.030354175, 0.030611387, 0.029967364, 0.030346683, 0.030330345, 0.030216266, 0.030181589, 0.030556358, 0.029567692, 0.030031761, 0.029570235, 0.029986774, 0.029867609, 0.030020653, 0.029590856, 0.029458432, 0.02985101, 0.029200958, 0.029490434, 0.029259603, 0.029348642, 0.029342882, 0.029510787, 0.028848549, 0.029043095, 0.029169361, 0.028938928, 0.02904979, 0.028640345, 0.028913489, 0.028733136, 0.028941877, 0.028560282, 0.028319807, 0.028396392, 0.028189665, 0.028355694, 0.028445667, 0.02835502, 0.028004356, 0.028139186, 0.028087367, 0.027592904, 0.028011718, 0.028193411, 0.027601069, 0.027841812, 0.027798963, 0.027724517, 0.027417232, 0.027084508, 0.02767632, 0.027183726, 0.027192696, 0.027144761, 0.026925181, 0.026923304, 0.026813377, 0.026859436, 0.026946468, 0.026612404, 0.02675366, 0.026610795, 0.026490832, 0.0261731, 0.026317037, 0.026151414, 0.026329354, 0.026249423, 0.026019794, 0.026133605, 0.025827254, 0.025507384, 0.02594334, 0.025788426, 0.025529207, 0.025381929, 0.025395718, 0.025184702, 0.025428119, 0.025200367, 0.025335334, 0.024857738, 0.024869918, 0.024808331, 0.024953472, 0.024638021, 0.024955747, 0.024342249, 0.02454349, 0.024304891, 0.024386301, 0.024252543, 0.02437331, 0.024006308, 0.02423527, 0.023852067, 0.024025856, 0.023642928, 0.023732235, 0.023904553, 0.023608781, 0.023361618, 0.02365163, 0.023198002, 0.023213261, 0.02315529, 0.023279274, 0.022893659, 0.022889239, 0.022767668, 0.022901962, 0.022478859, 0.0226165, 0.022357418, 0.022548487, 0.02257486, 0.022254323, 0.022419815, 0.021904993, 0.021923737, 0.022170638, 0.02209767, 0.021907536, 0.021667336, 0.021711388, 0.021762128, 0.021524203, 0.021301138, 0.021492069, 0.021172597, 0.021437308, 0.021235125, 0.020934535, 0.020922218, 0.020964393, 0.020605963, 0.020858356, 0.020740524, 0.020819521, 0.020389325, 0.020570214, 0.020282614, 0.020376471, 0.020241373, 0.020367102, 0.020053789, 0.020093827, 0.019895129, 0.019866742, 0.019807293, 0.019728695, 0.019685853, 0.019858439, 0.019412577, 0.019298904, 0.019694018, 0.019407353, 0.01911011, 0.018922259, 0.019056284, 0.019071818, 0.018689152, 0.019033526, 0.018777524, 0.01863747, 0.018619393, 0.018361246, 0.018383475, 0.018295639, 0.018159469, 0.018281046, 0.018230705, 0.017954618, 0.017932253, 0.017861696, 0.017719766, 0.017772657, 0.017733025, 0.017481435, 0.017699682, 0.017268145, 0.017411814, 0.01747327, 0.017080698, 0.017078822, 0.017188213, 0.017082169, 0.016817596, 0.016840491, 0.016834601, 0.016930871, 0.016602161, 0.01661421, 0.016415649, 0.01639905, 0.01627319, 0.016318575, 0.016262474, 0.016079578, 0.016105422, 0.016032447, 0.015815939, 0.015986257, 0.015902572, 0.0157107, 0.015601583, 0.015650185, 0.015600374, 0.015388822, 0.015341161, 0.015183567, 0.015201774, 0.015294834, 0.01513737, 0.01506293, 0.014916718, 0.014903328, 0.014752566, 0.014737437, 0.01481563, 0.014717621, 0.014512228, 0.014388512, 0.014500042, 0.014340845, 0.014303488, 0.014184453, 0.014238953, 0.013916807, 0.014026061, 0.014017895, 0.013871546, 0.013805271, 0.013664819, 0.013619427, 0.013784651, 0.013617956, 0.013442826, 0.013417113, 0.013299288, 0.013297281, 0.013173696, 0.013245462, 0.013106082, 0.013042214, 0.012986916, 0.013037663, 0.012876185, 0.012859187, 0.01266517, 0.012634782, 0.012697846, 0.01256248, 0.012524449, 0.012581492, 0.012415058, 0.012385468, 0.012303659, 0.012207122, 0.012230285, 0.012224931, 0.012121698, 0.012093716, 0.012030384, 0.012024494, 0.011884309, 0.011861414, 0.011826201, 0.011738365, 0.011703283, 0.011760325, 0.011608759, 0.011639957, 0.01154489, 0.011577959, 0.011487854, 0.011318073, 0.011394258, 0.011396135, 0.011283266, 0.011131025, 0.01115379, 0.011247379, 0.011137184, 0.010901266, 0.0110768, 0.010974104, 0.010974908, 0.010886, 0.010824008, 0.010805938, 0.010740729, 0.010694265, 0.010764697, 0.010643119, 0.010619825, 0.01061447, 0.010405325, 0.010469461, 0.01049892, 0.010409613, 0.010356186, 0.010328472, 0.010344274, 0.010309323, 0.010164987, 0.010216937, 0.010189628, 0.010043416, 0.010055465, 0.010034845, 0.010002711, 0.010026947, 0.009960934, 0.009945806, 0.009803614, 0.009946878, 0.009733718, 0.009806824, 0.009755947, 0.009818069, 0.009661553, 0.009755679, 0.009601569, 0.0097289, 0.009621248, 0.009571037, 0.009558452, 0.009570102, 0.009436475, 0.00944451, 0.009419202, 0.009463123, 0.00935721, 0.009364173, 0.009372345, 0.00934811, 0.00933338, 0.009277011, 0.009147803, 0.009301645, 0.009090499, 0.009257325, 0.009138166, 0.009187841, 0.009163468, 0.009131066, 0.008994229, 0.009055554, 0.009010698, 0.009007213, 0.008893808, 0.00888015, 0.00888457, 0.00895848, 0.008819898, 0.008886446, 0.008806508, 0.008762057, 0.0087954, 0.008736481, 0.008697921, 0.008784554, 0.008608483, 0.008663911, 0.008667128, 0.008603397, 0.008577553, 0.008560816, 0.008552114, 0.008553853, 0.008534442, 0.008582907, 0.00844607, 0.008421566, 0.008469371, 0.008469103, 0.008432681, 0.008375507, 0.008313653, 0.008322485, 0.008328912, 0.008324362, 0.008255943, 0.008272275, 0.008215775, 0.008210956, 0.0081096, 0.008252995, 0.008244829, 0.008167571, 0.00815847, 0.008125932, 0.008123519, 0.008030603, 0.008007302, 0.00801962, 0.00801654, 0.007993109, 0.007935138, 0.007961381, 0.007953744, 0.007931522, 0.007839535, 0.007860554, 0.007829761, 0.007820124, 0.007789592, 0.00781811, 0.007710327, 0.007740722, 0.0076877, 0.007790128, 0.00767418, 0.007738041, 0.007652088, 0.007709661, 0.007649408, 0.007659718, 0.007656103, 0.007644452, 0.007564252, 0.007588756, 0.007498913, 0.007598262, 0.007544972, 0.007476685, 0.007440936, 0.007527026, 0.007534793, 0.007478293, 0.007426075, 0.007419112, 0.007415091, 0.007420721, 0.007389522, 0.007296998, 0.00731856, 0.007288166, 0.007273299, 0.00733382, 0.007203141, 0.007222421, 0.007249468, 0.007208228, 0.007193236, 0.007193504, 0.007202069, 0.007151191, 0.007117855, 0.007142084, 0.0071054, 0.00708933, 0.007060276, 0.007138201, 0.007090808, 0.007116645, 0.006974317, 0.00703591, 0.007051038, 0.006941784, 0.006956776, 0.007033497, 0.006888893, 0.006876981, 0.006902152, 0.006877648, 0.006913397, 0.006869345, 0.006880597, 0.006875504, 0.006851006, 0.006804006, 0.006845515, 0.006736123, 0.006818069, 0.006787942, 0.006771872, 0.00673987, 0.006707873, 0.00675728, 0.006740674, 0.006731037, 0.006657127, 0.006676276, 0.006558046, 0.006606517, 0.006587368, 0.006545996, 0.006601025, 0.006623921, 0.006550547, 0.0065069, 0.006534078, 0.006581477, 0.006471687, 0.006522695, 0.006460573, 0.006487751, 0.006429773, 0.006450125, 0.006398581, 0.006492706, 0.006325338, 0.006433925, 0.006276604, 0.006361623, 0.006307934, 0.006269642, 0.00630137, 0.006349305, 0.006258259, 0.006288386, 0.006330431, 0.006183414, 0.006247282, 0.006270577, 0.006184617, 0.006169887, 0.006253572, 0.006233625, 0.006137087, 0.006149005, 0.006081921, 0.006151549, 0.006066661, 0.006142579, 0.006054874, 0.006123965, 0.006092904, 0.006091695, 0.006062909, 0.006017784, 0.006081653, 0.005975609, 0.005971196, 0.006025283, 0.005936512, 0.005982441, 0.006000917, 0.00593665, 0.005972399, 0.005947496, 0.00593544, 0.00589246, 0.005963291, 0.005864348, 0.005947895, 0.005878672, 0.005887374, 0.005927948, 0.005808246, 0.005897285, 0.005874789, 0.005876528, 0.005885766, 0.005874122, 0.005882686, 0.0058211, 0.005839177, 0.005798067, 0.00581668, 0.005870774, 0.005809717, 0.005829534, 0.005834757, 0.005850422, 0.005804631, 0.005858052, 0.005854842, 0.005846271, 0.005880143, 0.005851762, 0.005833417, 0.005809051, 0.005864884, 0.005886171, 0.005897553, 0.005899691, 0.005868231, 0.005814941, 0.005867159, 0.00585832, 0.005879476, 0.005917906, 0.00589233, 0.00586287, 0.005867022, 0.005888583, 0.005865949, 0.005922587, 0.00585832, 0.00588349, 0.005885635, 0.005948163, 0.00590264, 0.005912545, 0.005868898, 0.005869702, 0.005939592, 0.005904516, 0.005831678, 0.005943475, 0.005836895, 0.005946156, 0.005909603, 0.005924999, 0.005875992, 0.005911211, 0.005894206, 0.005948032, 0.005886707, 0.00593357, 0.005877469, 0.005924064, 0.005892192, 0.005965835, 0.005950843, 0.00585069, 0.005931295, 0.005911878, 0.005918036, 0.005918703, 0.00594883, 0.005903712, 0.005874122, 0.005903307, 0.005863544, 0.005911074, 0.00586729, 0.005912009, 0.005873984, 0.005843459, 0.005858725, 0.005867963, 0.005853632, 0.005844126, 0.005779859, 0.00586408, 0.005804363, 0.005873984, 0.005802356, 0.00583462, 0.005824179, 0.005734872, 0.005811326, 0.005838772, 0.00573059, 0.005747588, 0.005788025, 0.005792582, 0.005709165, 0.00578843, 0.005709433, 0.005796465, 0.00572925, 0.005789234, 0.00566699, 0.00568493, 0.005665381, 0.005669128, 0.005677032, 0.005705949, 0.005691356, 0.005632038, 0.005724026, 0.005660563, 0.00565547, 0.005584378, 0.005653731, 0.005660158, 0.005586117, 0.00560352, 0.005556258, 0.005607541, 0.005563757, 0.005553983, 0.005543268, 0.005458786, 0.005497078, 0.005515423, 0.005470567, 0.005502033, 0.005529349, 0.005473247, 0.005479537, 0.005446599, 0.005381129, 0.005470698, 0.005467487, 0.005461329, 0.005375232, 0.005432137, 0.005376036, 0.005364791, 0.005367066, 0.005374297, 0.00536519, 0.005360371, 0.005315921, 0.005307219, 0.005296104, 0.005259682, 0.005312037, 0.00520479, 0.005194343, 0.005299184, 0.005250444, 0.005264239, 0.005133155, 0.005278165, 0.005208805, 0.005191669, 0.005153377, 0.00515913, 0.005133024, 0.005167832, 0.005091117, 0.005180417, 0.005125924, 0.00502296, 0.005121642, 0.005121773, 0.005048935, 0.005089241, 0.004992435, 0.005043044, 0.005069424, 0.00506996, 0.004970742, 0.005057506, 0.004965257, 0.005003948, 0.004997789, 0.004960301, 0.004922408, 0.004960837, 0.004975162, 0.004954143, 0.004858945, 0.004921205, 0.004899244, 0.004930574, 0.004912902, 0.004828681, 0.004840064, 0.004873936, 0.004821987, 0.004850374, 0.004833238, 0.004817436, 0.004785839, 0.004793469, 0.004726789, 0.004816364, 0.004730006, 0.004677651, 0.0047533, 0.00475009, 0.004718493, 0.004741918, 0.004721572, 0.004648864, 0.004733085, 0.004749822, 0.004670825, 0.004612181, 0.00463641, 0.004615927, 0.004629048, 0.004590351, 0.004595843, 0.004633331, 0.004559826, 0.004596784, 0.004600262, 0.004596248, 0.004536126, 0.004581649, 0.004587003, 0.00451444, 0.00456063, 0.004516846, 0.004544161, 0.004516983, 0.004438522, 0.00446891, 0.004442805, 0.004488458, 0.004464092, 0.004464092, 0.004444949, 0.004455926, 0.004400897, 0.004479358, 0.004416156, 0.004362468, 0.004361395, 0.004406918, 0.004375053, 0.004367155, 0.004295926, 0.004371039, 0.004326719, 0.004365815, 0.004327922, 0.004314938, 0.004311989, 0.004308779, 0.00434292, 0.0043057, 0.004275436, 0.004245179, 0.004235941, 0.004250801, 0.004168588, 0.004275168, 0.004229377, 0.004244375, 0.004171805, 0.004237948, 0.004164175, 0.00420822, 0.004177427, 0.004156944, 0.004146229, 0.004153061, 0.004184259, 0.004155068, 0.004090127, 0.004108472, 0.004059733, 0.00414596, 0.004014615, 0.004122398, 0.004060674, 0.004131368, 0.004052103, 0.004048756, 0.004003769, 0.004059066, 0.004034163, 0.004057994, 0.004044604, 0.004065362, 0.004013942, 0.004051704, 0.003945791, 0.004010326, 0.003942712, 0.00396347, 0.003947936, 0.003923837, 0.003937756, 0.003983286, 0.003955833, 0.003863316, 0.003903348, 0.003885676, 0.003885539, 0.003868802, 0.00388969, 0.003903753, 0.003890095, 0.003909912, 0.003830915, 0.003844304, 0.003787667, 0.003787262, 0.003827567, 0.003787667, 0.003838682, 0.003814982, 0.003751781, 0.003781371, 0.003749904, 0.003783515, 0.003770394, 0.003775749, 0.00375352, 0.003754056, 0.003725675, 0.003721654, 0.00370291, 0.003746557, 0.003683094, 0.003656851, 0.003673052, 0.003705055, 0.003678544, 0.003620834, 0.003579731, 0.003656315, 0.003633557, 0.003641455, 0.003588302, 0.003589505, 0.003595932, 0.003599142, 0.003583477, 0.003559241, 0.003580267, 0.003634224, 0.00356487, 0.003596599, 0.003507828, 0.003500466, 0.003557371, 0.003529919, 0.003470072, 0.003531795, 0.003491091, 0.003502342, 0.003488816, 0.003486403, 0.003445705, 0.003460964, 0.003416651, 0.0034524, 0.003402588, 0.003419057, 0.003417985, 0.003444365, 0.003410754, 0.003420796, 0.003372331, 0.003380229, 0.003375678, 0.003383713, 0.003324663, 0.003332162, 0.003323461, 0.003391879, 0.003299355, 0.003338452, 0.003286502, 0.00330418, 0.003290255, 0.003302435, 0.003244595, 0.003283959, 0.003264816, 0.003215808, 0.003231878, 0.003253296, 0.003258657, 0.003249282, 0.003242855, 0.00322893, 0.003203354, 0.003191573, 0.003199071, 0.003200144, 0.003157027, 0.003227995, 0.003141094, 0.003168677, 0.003134935, 0.003187022, 0.003093701, 0.003129581, 0.003136413, 0.003149397, 0.003061162, 0.003131725, 0.003099454, 0.003107091, 0.003060358, 0.003080312, 0.003058482, 0.003059286, 0.003033847, 0.00309731, 0.003002649, 0.00303184, 0.003014835, 0.003040947, 0.003013227, 0.002990867, 0.003012292, 0.003014567, 0.002938787, 0.00297279, 0.002946417, 0.002942265, 0.002896075, 0.00293557, 0.002891656, 0.002925129, 0.002894598, 0.002893793, 0.002881881, 0.002895539, 0.002863536, 0.002898749, 0.002863536, 0.002838098, 0.00281413, 0.002836221, 0.002834881, 0.002833946, 0.002793509, 0.002865007, 0.002821629, 0.002834214, 0.002794444, 0.002757492, 0.002704072, 0.002770078, 0.002734728, 0.002733525, 0.002755217, 0.002735133, 0.002707419, 0.002751733, 0.002697776, 0.002709158, 0.002668323, 0.002699384, 0.002664034, 0.00267475, 0.002619315, 0.002675416, 0.002643015, 0.002644218, 0.002604049, 0.002611149, 0.002604991, 0.002617975, 0.00258303, 0.002579415, 0.002573655, 0.002585979, 0.002530544, 0.002574459, 0.002553976, 0.002567902, 0.002537507, 0.002508048, 0.00251046, 0.002516083, 0.002512873, 0.00251943, 0.002529341, 0.002481406, 0.002439361, 0.002465068, 0.002450207, 0.002428384, 0.0024245, 0.002438158, 0.002431862, 0.002414864, 0.002378977, 0.002426776, 0.002379782, 0.002387817, 0.002346308, 0.002404952, 0.002355546, 0.002360364, 0.002330107, 0.002335998, 0.002310559, 0.002321673, 0.002290337, 0.002299177, 0.002271731, 0.002322477, 0.002275078, 0.002282577, 0.002223527, 0.002258204, 0.002216166, 0.002251777, 0.002203175, 0.002204378, 0.00222473, 0.002205182, 0.002183091, 0.002216832, 0.002198219, 0.00220679, 0.00215337, 0.00218885, 0.002143595, 0.002168629, 0.002157652, 0.002173722, 0.002167563, 0.002158051, 0.002106774, 0.002139712, 0.002131409, 0.002145335, 0.002089233, 0.002130873, 0.002110926, 0.00209887, 0.002080126, 0.002082539, 0.002088828, 0.002069953, 0.002086422, 0.002098739, 0.002066469, 0.002049333, 0.002042638, 0.002080394, 0.002074105, 0.002083742, 0.002061114, 0.002073431, 0.002028444, 0.002059643, 0.00203795, 0.002070084, 0.002021351, 0.002047456, 0.002002339, 0.002013179, 0.002035813, 0.002011309, 0.002029118, 0.002004882, 0.002016395, 0.002014924, 0.002020141, 0.002000325, 0.002006085, 0.002004614, 0.002008628, 0.001997115, 0.001939674, 0.002022554, 0.001982255, 0.001989485, 0.001976763, 0.001989616, 0.001977031, 0.00199484, 0.001992833, 0.002012381, 0.001943021, 0.002005954, 0.001965917, 0.001983588, 0.001954939, 0.001995507, 0.001960961, 0.001969264, 0.001949716, 0.001976763, 0.001961229, 0.002007556, 0.00196659, 0.001972611, 0.001931508, 0.001972212, 0.00195186, 0.001949585, 0.00195373, 0.001994036, 0.001951187, 0.001955874, 0.00195373, 0.001928023, 0.001938065, 0.001975154, 0.001902186, 0.001944093, 0.001933384, 0.001927487, 0.001930037, 0.001933652, 0.001944093, 0.00194597, 0.001949853, 0.001939007, 0.001930834, 0.00198145, 0.001910221, 0.001940478, 0.001935254, 0.001940746, 0.001903258, 0.001904729, 0.001929363, 0.001946774, 0.001918785, 0.001920792, 0.001886521, 0.001930037, 0.00190178, 0.001914235, 0.001893614, 0.001885311, 0.001885985, 0.001911156, 0.001902316, 0.001890672, 0.001894687, 0.001869784, 0.001860814, 0.001886521, 0.001903657, 0.001897498, 0.001876211, 0.001887724, 0.001870451, 0.001889731, 0.001877015, 0.001879153, 0.001870856, 0.001871654, 0.001856395, 0.001861749, 0.001819972, 0.00187661, 0.001819037, 0.001878885, 0.001817834, 0.001835774, 0.001812741, 0.001821044, 0.001850099, 0.001823189, 0.001821581, 0.001811806, 0.001814487, 0.001811002, 0.001755843, 0.001813546, 0.001782752, 0.001805249, 0.001773514, 0.001772841, 0.001747938, 0.001774717, 0.001765349, 0.001748612, 0.001755574, 0.001743519, 0.001733346, 0.001736425, 0.001773377, 0.001716877, 0.001710581, 0.001751423, 0.001729194, 0.00172076, 0.001690765, 0.001735889, 0.001686084, 0.00168983, 0.001687823, 0.001642967, 0.001669615, 0.001680592, 0.001666536, 0.00167644, 0.001644307, 0.001644307, 0.001647118, 0.001650335, 0.001614717, 0.001654081, 0.001603066, 0.001617397, 0.001588068, 0.001609761, 0.001596241, 0.001606283, 0.001568121, 0.001595168, 0.001580439, 0.001592089, 0.001584322, 0.001585257, 0.001552457, 0.001547638, 0.001544154, 0.001538799, 0.001504391, 0.001525142, 0.001516839, 0.001527554, 0.001528888, 0.001508941, 0.001504659, 0.001497022, 0.001475604, 0.001469975, 0.001462483, 0.00151068, 0.001464758, 0.001480554, 0.001482966, 0.00146449, 0.001461273, 0.001449093, 0.001435436, 0.001439182, 0.001418562, 0.001451231, 0.001413076, 0.001415084, 0.001409324, 0.001416555, 0.001389377, 0.001392051, 0.001364873, 0.001386827, 0.00136112, 0.001390979, 0.001330464, 0.001391783, 0.001358309, 0.001364069, 0.001345593, 0.001344919, 0.00130007, 0.001316807, 0.001331798, 0.001322024, 0.001300874, 0.001321625, 0.001290695, 0.001313322, 0.001276102, 0.00129886, 0.001262843, 0.001293375, 0.001255612, 0.001252265, 0.001258829, 0.00125267, 0.001263379, 0.001240889, 0.001233122, 0.001234999, 0.001201656, 0.001207147, 0.001207546, 0.001222407, 0.001196569, 0.001206879, 0.00118907, 0.00120567, 0.001182506, 0.00120835, 0.001166037, 0.001164168, 0.001165508, 0.001174877, 0.001123463, 0.001158277, 0.001115964, 0.001176616, 0.001122796, 0.001135113, 0.00112989, 0.001105255, 0.001112617, 0.001100698, 0.001093873, 0.001127883, 0.001082222, 0.001095213, 0.001085171, 0.001094272, 0.00103938, 0.001086642, 0.001061334, 0.001074723, 0.001044865, 0.001061602, 0.001035359, 0.001058124, 0.001054508, 0.001050756, 0.001040315, 0.001026389, 0.001053972, 0.001022107, 0.001030005, 0.001040315, 0.001004298, 0.001000015, 0.000986358, 0.001014471, 0.000970157, 0.001005501, 0.000998806, 0.00100671, 0.000977388, 0.000993988, 0.000955695, 0.001004697, 0.000970961, 0.000974439, 0.000949131, 0.000959978, 0.000946052, 0.000951943, 0.000931061, 0.000954623, 0.000926641, 0.000931996, 0.000924634, 0.000946189, 0.000907629, 0.000934539, 0.00094753, 0.000913919, 0.000882857, 0.000909499, 0.000903072, 0.000915259, 0.000904681, 0.000891559, 0.000880046, 0.000886335, 0.000865316, 0.000886335, 0.000862773, 0.000889951, 0.000847507, 0.000852064, 0.000838805, 0.00086157, 0.000851927, 0.000862374, 0.000842826, 0.000841617, 0.000845232, 0.000867055, 0.000845905, 0.000842153, 0.000836262, 0.000843624, 0.000810954, 0.000821402, 0.000815374, 0.000817649, 0.000796898, 0.0008218, 0.000793818, 0.000815374, 0.000799846, 0.000810019, 0.000767706, 0.000798637, 0.000796767, 0.000817786, 0.000787392, 0.000788327, 0.000760743, 0.000772531, 0.000763561, 0.000759946, 0.000776277, 0.000771459, 0.000740797, 0.000769184, 0.000747092, 0.000753251, 0.000725531, 0.000753519, 0.00072995, 0.000729819, 0.000718836, 0.000745484, 0.000720176, 0.000740398, 0.000707859, 0.000737848, 0.000717365, 0.000724596, 0.00069273, 0.00070799, 0.000714684, 0.00072995, 0.000704511, 0.000734501, 0.000671443, 0.000741065, 0.000701968, 0.000718705, 0.000682152, 0.000680812, 0.00066769, 0.000683492, 0.000673045, 0.000680276, 0.00068563, 0.000673718, 0.000652961, 0.00068376, 0.000661931, 0.000679877, 0.000663944, 0.000670103, 0.000671842, 0.000669966, 0.000656177, 0.000661662, 0.00065149, 0.000630738, 0.000641853, 0.00063676, 0.000615074, 0.000643991, 0.000635021, 0.000647475, 0.000639839, 0.000637296, 0.000609981, 0.000610386, 0.000624312, 0.000644396, 0.000601952, 0.000609582, 0.000610922, 0.000613328, 0.000605699, 0.000605699, 0.000610118, 0.000622703, 0.000601815, 0.00060717, 0.000582934, 0.000592577, 0.000588831, 0.000590701, 0.000585882, 0.000578521, 0.00057263, 0.000582934, 0.000574101, 0.00058026, 0.000567942, 0.000572094, 0.000555893, 0.000584274, 0.000552945, 0.000564059, 0.000551872, 0.00056098, 0.000540222, 0.00055977, 0.000544511, 0.000575572, 0.000531788, 0.000553612, 0.00054491, 0.000545446, 0.000534599, 0.000543033, 0.000511436, 0.000561378, 0.000514116, 0.000531657, 0.000528977, 0.00053286, 0.000518, 0.000542903, 0.000516124, 0.000495771, 0.000525231, 0.000522419, 0.000522956, 0.000531788, 0.000518131, 0.000501799, 0.000491888, 0.000509697, 0.000502871, 0.00050006, 0.000495909, 0.000488004, 0.000488946, 0.000503806, 0.000485997, 0.00047944, 0.000494836, 0.000460689, 0.000452929, 0.000492424, 0.000467122, 0.000495503, 0.000470731, 0.00046792, 0.000469934, 0.000468992, 0.000469796, 0.000467652, 0.00044998, 0.000451857, 0.000447836, 0.000457747, 0.000428556, 0.000462435, 0.000445162, 0.000442213, 0.000449176, 0.000459623, 0.000440742, 0.00046471, 0.000445293, 0.000444756, 0.000434315, 0.000452929, 0.000409007, 0.000424404, 0.000423201, 0.000425346, 0.000433779, 0.000422534, 0.000416774, 0.000422534, 0.000404326, 0.000430837, 0.000411021, 0.000419854, 0.000413028, 0.00041584, 0.0004141, 0.000404725, 0.000377815, 0.000407935, 0.000390669, 0.000418115, 0.000394016, 0.000421462, 0.000401509, 0.00040379, 0.000397762, 0.000405392, 0.000369107, 0.000392408, 0.000378881, 0.000397632, 0.000394284, 0.000394016, 0.000378345, 0.000398965, 0.000379555, 0.000378214, 0.000370317, 0.000399371, 0.00038183, 0.000389329, 0.000372592, 0.000382098, 0.000367368, 0.000395082, 0.000355449, 0.00036844, 0.000370048, 0.000358398, 0.000359738, 0.000369244, 0.000349428, 0.000340321, 0.000364962, 0.000373259, 0.000373259, 0.000359601, 0.000341262, 0.000364557, 0.00034206, 0.000346748, 0.000348225, 0.000354515, 0.000342465, 0.000344204, 0.000343132, 0.000347689, 0.000336039, 0.000341798, 0.000334698, 0.000329475, 0.000333763, 0.000326801, 0.000321982, 0.00034019, 0.000324656, 0.000351834, 0.000322917, 0.000337915, 0.000297746, 0.000337111, 0.000321714, 0.000311672, 0.000320505, 0.000324257, 0.000319302, 0.000337915, 0.000305912, 0.000318367, 0.000318896, 0.000314215, 0.000302297, 0.000308325, 0.000320505, 0.000318367, 0.000290646, 0.000300826, 0.00029011, 0.000296674, 0.00030163, 0.000306043, 0.000296406, 0.000295471, 0.000283285, 0.000300826, 0.000283416, 0.000305644, 0.000285959, 0.000293059, 0.000284756, 0.000294262, 0.000283952, 0.00029587, 0.000276858, 0.00027204, 0.000271634, 0.000304977, 0.000268156, 0.000281676, 0.000275923, 0.000263469, 0.000276185, 0.00026052, 0.000268156, 0.000281814, 0.000258382, 0.000275655, 0.000262802, 0.00026186, 0.000256905, 0.000260389, 0.000242985, 0.000278734, 0.000259454, 0.000257173, 0.000249412, 0.000254499, 0.000256506, 0.000256368, 0.000257042, 0.000255434, 0.000242181, 0.000278061, 0.00025155, 0.000246333, 0.000249942, 0.000253021, 0.00027217, 0.000251151, 0.000233342, 0.000247, 0.000244319, 0.000241645, 0.000230799, 0.000258245, 0.000226517, 0.000239233, 0.000250079, 0.000236958, 0.000231734, 0.000230394, 0.000234277, 0.000232806, 0.000250883, 0.000244587, 0.000214193, 0.000219685, 0.000224503, 0.000216873, 0.000209642, 0.000235081, 0.00022464, 0.000224503, 0.000213127, 0.000221424, 0.000216206, 0.000241913, 0.000204687, 0.000223568, 0.000225843, 0.000225981, 0.000209642, 0.000213794, 0.000194377, 0.000214461, 0.000210048, 0.000211114, 0.000194514, 0.000213395, 0.000193311, 0.00019639, 0.000202549, 0.000217678, 0.000192239, 0.000206962, 0.000202144, 0.000204151, 0.000205223, 0.000216475, 0.000185544, 0.000191167, 0.000194913, 0.000189558, 0.000200006, 0.000197724, 0.00018474, 0.000185139, 0.000194377, 0.000201477, 0.000183537, 0.000198528, 0.00018795, 0.000212323, 0.000187015, 0.0001996, 0.00019371, 0.000181929, 0.000177509, 0.000192906, 0.000174024, 0.000190761, 0.000179653, 0.000184066, 0.00016867, 0.000184871, 0.000185943, 0.000173763, 0.000160903, 0.000178176, 0.000168003, 0.000175234, 0.000177241, 0.000181392, 0.000163721, 0.000169206, 0.000171886, 0.000156621, 0.000176705, 0.000173227, 0.000169206, 0.000194109, 0.000154476, 0.000173894, 0.000161308, 0.000157961, 0.000146847, 0.000170415, 0.000141891, 0.000163851, 0.000141224, 0.000142963, 0.000179921, 0.000160504, 0.000151802, 0.000175234, 0.00015207, 0.000149122, 0.000148854, 0.000169075, 0.000151665, 0.000168134, 0.000147781, 0.000150599, 0.000153142, 0.000149527, 0.000140152, 0.000155418, 0.000148049, 0.00014497, 0.000153541, 0.000148586, 0.000146847, 0.000147651, 0.000150462, 0.000164924, 0.000143767, 0.000142695, 0.000141087, 0.000139884, 0.00013721, 0.000141623, 0.000131718, 0.000127436, 0.000143767, 0.000157961, 0.000129711, 0.000140557, 0.000119669, 0.000141355, 0.000130378, 0.000139753, 0.000137609, 0.000156889, 0.000128233, 0.000137877, 0.000127168, 0.000136406, 0.000125559, 0.000124356, 0.00011083, 0.000132385, 0.000119133, 0.00011672, 0.000129574, 0.000123284, 0.000119799, 0.000143904, 0.000113242, 0.000139348, 0.000125958, 0.000129175, 0.000119263, 0.000135732, 0.000124088, 0.000128371, 0.000122611, 0.000115249, 0.000123951, 0.000103331, 0.00012127, 0.000107887, 0.000121009, 0.000117256, 0.000110561, 0.00013908, 0.000114844, 0.00012114, 0.000119799, 0.000105743, 0.00010775, 0.000127834, 0.000117524, 0.000125559, 0.00010909, 0.00011083, 9.8911E-05, 0.000131718, 0.000109757, 0.000111634, 0.00010454, 0.000119937, 0.000109359, 0.000108149, 0.00011538, 0.000127298, 0.00011083, 0.000111634, 9.38246E-05, 0.000114177, 0.000107619, 0.000104802, 8.92743E-05, 0.000109489, 8.71298E-05, 0.000113778, 0.000104802, 0.000106142, 0.000114713, 0.000111764, 9.82442E-05, 8.37825E-05, 8.63257E-05, 0.000107214, 9.91791E-05, 9.54329E-05, 7.93629E-05, 0.000103867, 0.00010775, 9.51649E-05, 9.27524E-05, 9.04772E-05, 9.5701E-05, 0.000105606, 9.5969E-05, 9.744E-05, 8.32464E-05, 8.48547E-05, 9.27524E-05, 7.76238E-05, 0.000104004, 9.38246E-05, 9.30204E-05, 8.87382E-05, 9.22163E-05, 8.71298E-05, 9.40926E-05, 8.05724E-05, 6.74509E-05, 0.000103736, 7.92321E-05, 0.000102258, 8.59269E-05, 9.22163E-05, 9.73093E-05, 9.38246E-05, 8.17753E-05, 8.05724E-05, 8.96731E-05, 0.000101592, 7.33415E-05, 8.47239E-05, 9.87803E-05, 0.000103331, 7.41457E-05, 8.49854E-05, 8.53908E-05, 8.39198E-05, 7.89641E-05, 8.39198E-05, 7.38776E-05, 8.07031E-05, 8.07031E-05, 8.07031E-05, 8.07031E-05, 8.07031E-05, 8.07031E-05]
plt.plot(input_wavelengths, input_cfl_mwcmsq)
plt.show()
plt.plot(input_wavelengths, input_halogen_mwcmsq)
plt.show()

In [ ]:
# Sub-sample CFL and Halogen to match simulation wavelengths
# Also scale irradiance to match simulation
wavelengths = numpy.linspace(350,900,56)
# CFL
f_cubic_cfl = interp1d(input_wavelengths, input_cfl_mwcmsq, kind='cubic')
cfl_sample = 4.5*f_cubic_cfl(wavelengths)
plt.plot(wavelengths,cfl_sample)
plt.show()
#Halogen
f_cubic_halogen = interp1d(input_wavelengths, input_halogen_mwcmsq, kind='cubic')
halogen_sample = 4.8*f_cubic_halogen(wavelengths)
plt.plot(wavelengths,halogen_sample)
plt.show()

In [ ]:
# Generic Red and Blue narrowband light
# Red light peak at 633, bandwidth of 20nm
# FWHM = 2.355*standard deviation
wavelengths = numpy.linspace(350,900,56)
red_led = 100*gaussian(wavelengths, 633, 20/2.355)
blue_led = 100*gaussian(wavelengths, 415, 20/2.355)
plt.plot(wavelengths,red_led)
plt.show()
plt.plot(wavelengths,blue_led)
plt.show()

In [ ]:
# Define irradiance values of interest

# Integrate each light source from 350nm to 700 to find total irradiance
total_simulation = integrate_spectrum( 0.1*numpy.ones(len(wavelengths)),wavelengths, 350,700)
total_sun = integrate_spectrum(sampled_sun, wavelengths, 350,700)
total_halogen = integrate_spectrum(halogen_sample, wavelengths, 350,700)
total_cfl = integrate_spectrum(cfl_sample, wavelengths, 350,700)
total_blue = integrate_spectrum(blue_led, wavelengths, 350,700)
total_red = integrate_spectrum(red_led, wavelengths, 350,700)

# Scale each light broadband source to be 35mW/cm^2
comparison_scale_sim = 1
comparison_scale_sun = 35/total_sun
comparison_scale_halogen = 35/total_halogen
comparison_scale_cfl = 35/total_cfl

# Figure 2a

In [ ]:
# Generate Figure 2a
plt.figure(figsize=([20,20]))
ax=plt.gca()
ax.tick_params(labelsize=40)
ax.set_xlabel('Wavelength (nm)', size=40)
ax.set_ylabel('Surface Irradiance (mW cm$^{-2}$)', size=40)


comparison_scale_sim = 1
comparison_scale_sun = 35/total_sun
comparison_scale_halogen = 35/total_halogen
comparison_scale_cfl = 35/total_cfl

sampled_sun = comparison_scale_sun*sampled_sun
halogen_sample = comparison_scale_halogen*halogen_sample
cfl_sample = comparison_scale_cfl*cfl_sample

total_simulation = integrate_spectrum( 0.1*numpy.ones(len(wavelengths)),wavelengths, 350,700)
total_sun = integrate_spectrum(sampled_sun, wavelengths, 350,700)
total_halogen = integrate_spectrum(halogen_sample, wavelengths, 350,700)
total_cfl = integrate_spectrum(cfl_sample, wavelengths, 350,700)
total_blue = integrate_spectrum(blue_led, wavelengths, 350,700)
total_red = integrate_spectrum(red_led, wavelengths, 350,700)

xx = numpy.linspace(wavelengths.min(),wavelengths.max(),550)
f_cubic_sim = interp1d(wavelengths, 0.1*numpy.ones(len(wavelengths)), kind='cubic')
f_cubic_sun = interp1d(wavelengths, sampled_sun, kind='cubic')
f_cubic_halogen = interp1d(wavelengths, halogen_sample, kind='cubic')
f_cubic_cfl = interp1d(wavelengths, cfl_sample, kind='cubic')
f_cubic_blue = interp1d(wavelengths, blue_led, kind='cubic')
f_cubic_red = interp1d(wavelengths, red_led, kind='cubic')

ax.plot(xx, f_cubic_sim(xx), ':', label='Simulation: {:0.1f}mW/cm$^2$'.format(total_simulation), lw=8, color='black', alpha=0.8)
ax.plot(xx, f_cubic_sun(xx), label='Sun: {:0.1f}mW/cm$^2$'.format(total_sun), lw=8, color='darkorange', alpha=0.8)
ax.plot(xx, f_cubic_halogen(xx), label='Halogen: {:0.1f}mW/cm$^2$'.format(total_halogen), lw=8, color='forestgreen', alpha=0.8)
ax.plot(xx, f_cubic_cfl(xx), label='CFL: {:0.1f}mW/cm$^2$'.format(total_cfl), lw=8, color='silver', alpha=0.8)
ax.plot(xx, f_cubic_blue(xx), '--', label='Blue LED: {:0.1f}mW/cm$^2$'.format(total_blue), lw=8, color='indigo', alpha=0.8)
ax.plot(xx, f_cubic_red(xx), '--', label='Red LED: {:0.1f}mW/cm$^2$'.format(total_red), lw=8,color='firebrick', alpha=0.8)


ax.legend(loc='upper center', fontsize=35)
ax.set_ylim(bottom=0.01, top=5)
ax.set_yscale('log')
ax.set_xlim([350,700])
ax.grid(True)
ax.grid(True, which='minor', axis='y', ls=':')
plt.savefig(os.path.join(output_dir, 'Figure2A.png'), bbox_inches='tight')
plt.show()

# Figure 2b

In [ ]:
#Generate Figure 2b
plt.figure(figsize=([20,20]))
ax=plt.gca()
ax.tick_params(labelsize=40)
ax.set_xlabel('Wavelength (nm)', size=40)
ax.set_ylabel('PpIX Effective Surface Irradiance (mW$_{eff.}$ cm$^{-2}$)', size=40)



total_ppix_simulation = integrate_spectrum( sampled_ppix*0.1*numpy.ones(len(wavelengths)),wavelengths, 350,700)
total_ppix_sun = integrate_spectrum(sampled_ppix*sampled_sun, wavelengths, 350,700)
total_ppix_halogen = integrate_spectrum(sampled_ppix*halogen_sample, wavelengths, 350,700)
total_ppix_cfl = integrate_spectrum(sampled_ppix*cfl_sample, wavelengths, 350,700)
total_ppix_blue = integrate_spectrum(sampled_ppix*blue_led, wavelengths, 350,700)
total_ppix_red = integrate_spectrum(sampled_ppix*red_led, wavelengths, 350,700)

xx = numpy.linspace(wavelengths.min(),wavelengths.max(),550)
f_cubic_sim = interp1d(wavelengths, sampled_ppix*0.1*numpy.ones(len(wavelengths)), kind='cubic')
f_cubic_sun = interp1d(wavelengths, sampled_ppix*sampled_sun, kind='cubic')
f_cubic_halogen = interp1d(wavelengths, sampled_ppix*halogen_sample, kind='cubic')
f_cubic_cfl = interp1d(wavelengths, sampled_ppix*cfl_sample, kind='cubic')
f_cubic_blue = interp1d(wavelengths, sampled_ppix*blue_led, kind='cubic')
f_cubic_red = interp1d(wavelengths, sampled_ppix*red_led, kind='cubic')

plt.plot(xx, f_cubic_sim(xx), ':', label='Simulation: {:0.1f}mW/cm$^2$'.format(total_ppix_simulation), lw=8, color='black', alpha=0.8)
plt.plot(xx, f_cubic_sun(xx), label='Sun: {:0.1f}mW/cm$^2$'.format(total_ppix_sun), lw=8, color='darkorange', alpha=0.8)
plt.plot(xx, f_cubic_halogen(xx), label='Halogen: {:0.1f}mW/cm$^2$'.format(total_ppix_halogen), lw=8, color='forestgreen', alpha=0.8)
plt.plot(xx, f_cubic_cfl(xx), label='CFL: {:0.1f}mW/cm$^2$'.format(total_ppix_cfl), lw=8, color='silver', alpha=0.8)
plt.plot(xx, f_cubic_blue(xx), '--', label='Blue LED: {:0.1f}mW/cm$^2$'.format(total_ppix_blue), lw=8, color='indigo', alpha=0.8)
plt.plot(xx, f_cubic_red(xx), '--', label='Red LED: {:0.1f}mW/cm$^2$'.format(total_ppix_red), lw=8, color='firebrick', alpha=0.8)

plt.legend(loc='upper center', fontsize=35)
ax.set_ylim([0.001, 5])

ax.set_xlim([350,700])
ax.set_yscale('log')
ax.grid(True)
ax.grid(True, which='minor', axis='y', ls=':')
plt.savefig(os.path.join(output_dir, 'Figure2B.png'), bbox_inches='tight')
plt.show()

In [ ]:
# Tile PpIX absorption over 5mm
ppix_sampled_spectrum = numpy.reshape(numpy.repeat(sampled_ppix,500), [len(sampled_ppix),500])
plt.figure(figsize=(20,20))
ax=plt.gca()
ax.yaxis.set_ticks([0,10,20,30,40,50])
ax.yaxis.set_ticklabels(['350', '450', '550', '650', '750', '850'])
ax.yaxis.set_label_text('\n\nWavelength (nm)')
ax.xaxis.set_ticks([0, 100, 200, 300, 400, 500])
ax.xaxis.set_ticklabels(['0', '1.0', '2.0', '3.0', '4.0', '5.0'])
ax.xaxis.set_label_text('\n\nDepth (mm)')
plt.imshow(ppix_sampled_spectrum)
plt.show()

In [ ]:
# Show simulated fluence rate over 5mm
rotate_depth_fluence = numpy.rot90(watts_fluence_map_crop)
plt.figure(figsize=(20,20))
ax = plt.gca()
ax.yaxis.set_ticks([0,10,20,30,40,50])
ax.yaxis.set_ticklabels(['350', '450', '550', '650', '750', '850'])
ax.yaxis.set_label_text('\n\nWavelength (nm)')
ax.xaxis.set_ticks([0, 100, 200, 300, 400, 500])
ax.xaxis.set_ticklabels(['0', '1.0', '2.0', '3.0', '4.0', '5.0'])
ax.xaxis.set_label_text('\n\nDepth (mm)')
plt.imshow(rotate_depth_fluence)
plt.show()

In [ ]:
# Show effective fluence rate over 5mm
plt.figure(figsize=(20,20))
plt.imshow(rotate_depth_fluence*ppix_sampled_spectrum,norm=matplotlib.colors.LogNorm())
ax = plt.gca()
ax.yaxis.set_ticks([0,10,20,30,40,50])
ax.yaxis.set_ticklabels(['350', '450', '550', '650', '750', '850'])
ax.yaxis.set_label_text('\n\nWavelength (nm)')
ax.xaxis.set_ticks([0, 100, 200, 300, 400, 500])
ax.xaxis.set_ticklabels(['0', '1.0', '2.0', '3.0', '4.0', '5.0'])
ax.xaxis.set_label_text('\n\nDepth (mm)')
plt.show()

In [ ]:
# Tile all light sources over 5mm
sun_sampled_spectrum = numpy.reshape(numpy.repeat(sampled_sun,500), [len(sampled_sun),500])
cfl_sampled_spectrum = numpy.reshape(numpy.repeat(cfl_sample,500), [len(cfl_sample),500])
halogen_sampled_spectrum = numpy.reshape(numpy.repeat(halogen_sample,500), [len(halogen_sample),500])
blue_sampled_spectrum = numpy.reshape(numpy.repeat(blue_led,500), [len(blue_led),500])
red_sampled_spectrum = numpy.reshape(numpy.repeat(red_led,500), [len(red_led),500])

In [ ]:
# Caluclate the PpIX-effective fluence rate for all light sources
depth_ppix_corrected_spectrum = numpy.rot90(0.1*rotate_depth_fluence*ppix_sampled_spectrum,-1)

depth_sun_corrected_spectrum = numpy.rot90(rotate_depth_fluence*sun_sampled_spectrum,-1)
depth_sun_ppix_corrected_spectrum = numpy.rot90(rotate_depth_fluence*sun_sampled_spectrum*ppix_sampled_spectrum,-1)

depth_halogen_corrected_spectrum = numpy.rot90(rotate_depth_fluence*halogen_sampled_spectrum,-1)
depth_halogen_ppix_corrected_spectrum = numpy.rot90(rotate_depth_fluence*halogen_sampled_spectrum*ppix_sampled_spectrum,-1)

depth_cfl_corrected_spectrum = numpy.rot90(rotate_depth_fluence*cfl_sampled_spectrum,-1)
depth_cfl_ppix_corrected_spectrum = numpy.rot90(rotate_depth_fluence*cfl_sampled_spectrum*ppix_sampled_spectrum,-1)

depth_blue_corrected_spectrum = numpy.rot90(rotate_depth_fluence*blue_sampled_spectrum,-1)
depth_blue_ppix_corrected_spectrum = numpy.rot90(rotate_depth_fluence*blue_sampled_spectrum*ppix_sampled_spectrum,-1)

depth_red_corrected_spectrum = numpy.rot90(rotate_depth_fluence*red_sampled_spectrum,-1)
depth_red_ppix_corrected_spectrum = numpy.rot90(rotate_depth_fluence*red_sampled_spectrum*ppix_sampled_spectrum,-1)



In [ ]:
# Organize results
ordered_lights = ['sim', 'sun', 'halogen', 'cfl', 'blue', 'red']
sun_irradiance = [5,10,15,20,25,30,35,40,45,50,55,60,65]
bulb_irradiance = [1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40]
blue_irradiance =[5,10,15,20,25,30,35]
red_irradiance =[30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105]

all_sources = {
    'sim': {
        'spectrum':0.1*watts_fluence_map_crop,#numpy.reshape(numpy.repeat( numpy.ones(len(wavelengths)),500), [len(wavelengths),500]),
        'wavelengths': wavelengths,
        'ppix_weighted': depth_ppix_corrected_spectrum,
        'irradiance_0': total_simulation,
        'ppix_irradiance_0':total_ppix_simulation,
        'chart_irradiance': numpy.array([5,10,15,20,25,30,35,40,45,50,55,60,65]),
        'plot_color': 'black',
        'line_style': ':',
        'display_name': 'Simulation'
    },
    'sun': {
        'spectrum': depth_sun_corrected_spectrum,
        'wavelengths': wavelengths,
        'ppix_weighted':depth_sun_ppix_corrected_spectrum,
        'irradiance_0': total_sun,
        'ppix_irradiance_0':total_ppix_sun,
        'chart_irradiance': numpy.array([5,10,15,20,25,30,35,40,45,50,55,60,65]),
        'plot_color': 'darkorange',
        'line_style': '-',
        'display_name': 'Sun'
    },
    'halogen': {
        'spectrum': depth_halogen_corrected_spectrum,
        'wavelengths': wavelengths,
        'ppix_weighted': depth_halogen_ppix_corrected_spectrum,
        'irradiance_0': total_halogen,
        'ppix_irradiance_0':total_ppix_halogen,
        'chart_irradiance': numpy.array([1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40]),
        'plot_color': 'forestgreen',
        'line_style': '-',
        'display_name': 'Halogen'
    },
    'cfl': {
        'spectrum': depth_cfl_corrected_spectrum,
        'wavelengths': wavelengths,
        'ppix_weighted': depth_cfl_ppix_corrected_spectrum,
        'irradiance_0': total_cfl,
        'ppix_irradiance_0':total_ppix_cfl,
        'chart_irradiance': numpy.array([1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40]),
        'plot_color': 'silver',
        'line_style': '-',
        'display_name': 'CFL'
    },
    'blue': {
        'spectrum': depth_blue_corrected_spectrum,
        'wavelengths': wavelengths,
        'ppix_weighted': depth_blue_ppix_corrected_spectrum,
        'irradiance_0': total_blue,
        'ppix_irradiance_0':total_ppix_blue,
        'chart_irradiance': numpy.array([5,10,15,20,25,30,35,40,50,60,70,80,90,100]),
        'plot_color': 'indigo',
        'line_style': '--',
        'display_name': 'Blue'
    },
    'red': {
        'spectrum': depth_red_corrected_spectrum,
        'wavelengths': wavelengths,
        'ppix_weighted': depth_red_ppix_corrected_spectrum,
        'irradiance_0': total_red,
        'ppix_irradiance_0':total_ppix_red,
        'chart_irradiance': numpy.array([5,10,20,30,40,50,60,70,80,90,100,110]),
        'plot_color': 'firebrick',
        'line_style': '--',
        'display_name': 'Red'
    }
    
}

# Figure 3

In [ ]:
full_file = os.path.join(input_dir, "S1_MC_geometry_LaRochelle.xlsx")
xl = pandas.ExcelFile(full_file)
skin_layers = []
for i in range(0,7):
    skin_layers.append(xl.parse("skin{}".format(i+1), skiprows=6))

In [ ]:

# define the colormap
cmap = plt.cm.inferno
layers_of_interest = [0,2,3,4,5,6]
# extract all colors from the .viridis map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_layers = len(layers_of_interest)
layer_colors = cmap_array[0::int(cmap.N/num_layers)]
color_cycler = cycle(layer_colors)


fig = plt.figure(figsize=[20,20])
ax = fig.gca()

for i in layers_of_interest:
    cur_color = next(color_cycler)
    cur_wavelengths = skin_layers[i]['mpt_wavelength'].values
    cur_mu_s = skin_layers[i]['mpt_miescat'].values
    cur_mu_a = skin_layers[i]['mpt_abs'].values
    cur_n = numpy.unique(skin_layers[i]['mpt_rindex'].values)[0]
    cur_g = numpy.unique(skin_layers[i]['mpt_g'].values)[0]
    f_cubic_mu_a = interp1d(cur_wavelengths, cur_mu_a, kind='cubic')
    xx = numpy.linspace(cur_wavelengths.min(),cur_wavelengths.max(),550)
    if i==0:
        label_i = 'Layers 1-2'
    else:
        label_i = 'Layer {}'.format(i+1)
    plt.semilogy(xx, f_cubic_mu_a(xx), color=cur_color, ls='-', lw=6, label=label_i)
plt.ylim(0.01, 50)
plt.xlim(350, 900)
plt.grid(True)
plt.ylabel('$\mu_a$ (mm$^{-1}$)', fontsize=55)
ax.yaxis.set_tick_params(labelsize=40)

plt.xlabel('Wavelength (nm)', fontsize=55)
ax.xaxis.set_tick_params(labelsize=40, rotation=-45)

plt.legend(loc='best', fontsize=45)
plt.savefig(os.path.join(output_dir, 'Figure3_mu_a.png'),bbox_inches='tight')
plt.show()

In [ ]:

# define the colormap
cmap = plt.cm.inferno
layers_of_interest = [1,2,3,7]
# extract all colors from the .viridis map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_layers = len(layers_of_interest)
layer_colors = cmap_array[0::int(cmap.N/num_layers)]
color_cycler = cycle(layer_colors)


fig = plt.figure(figsize=[20,20])
ax = fig.gca()

for i in [0,1,2,6]:
    cur_color = next(color_cycler)
    cur_wavelengths = skin_layers[i]['mpt_wavelength'].values
    cur_mu_s = skin_layers[i]['mpt_miescat'].values
    f_cubic_mu_s = interp1d(cur_wavelengths, cur_mu_s, kind='cubic')
    xx = numpy.linspace(cur_wavelengths.min(),cur_wavelengths.max(),550)
    if i==2:
        label_i = 'Layers 3-6'
    else:
        label_i = 'Layer {}'.format(i+1)
    plt.semilogy(xx, f_cubic_mu_s(xx), color=cur_color, ls='-', lw=6, label='{}'.format(label_i))
plt.ylim(1, 200)
plt.xlim(350, 900)
plt.grid(True)
plt.ylabel('$\mu_s$ (mm$^{-1}$)', fontsize=55)
ax.yaxis.set_tick_params(labelsize=40)

plt.xlabel('Wavelength (nm)', fontsize=55)
ax.xaxis.set_tick_params(labelsize=40, rotation=-45)

plt.legend(loc='best', fontsize=45)
plt.savefig(os.path.join(output_dir, 'Figure3_mu_s.png'),bbox_inches='tight')
plt.show()

# Figure 4a

In [ ]:
# Generate Figure 4a

# define the colormap
cmap = plt.cm.inferno
depths_of_interest = [10, 60, 175, 290, 500, 1000, 1870, 3000]
# extract all colors from the .viridis map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_depths = len(depths_of_interest)
depth_colors = cmap_array[0::int(cmap.N/num_depths)]


for each_source in ['sim']:
    color_cycler = cycle(depth_colors)
    plt.figure(figsize=[20,20])
    plt.xlabel('Wavelength (nm)', size=40)
    plt.title('{} Total Fluence\n'.format(each_source.upper()), size=70)
    ax1 = plt.gca()
    ax1.set_ylim([0.0001,10])
    ax1.tick_params(labelsize=40)
    ax1.set_ylabel('Fluence Rate (mW $cm^{-2}$)', size=40)
    ax1.set_yscale('log')
    
    for each_depth in depths_of_interest:
        index = int(each_depth/10)
        cur_color = next(color_cycler)
        spectrum_at_depth = all_sources[each_source]['spectrum'][index][::-1]
        wavelengths = all_sources[each_source]['wavelengths']
        f_cubic_sim = interp1d(wavelengths, spectrum_at_depth, kind='cubic')
        xx = numpy.linspace(wavelengths.min(),wavelengths.max(),550)
        
        ax1.plot(xx, f_cubic_sim(xx),
                 color=cur_color , lw=8,alpha=0.7, 
                 label='{}um'.format(each_depth))
    
    ax1.grid(True)
    ax1.grid(True, which='minor', axis='y', ls=':')
    ax1_legend = ax1.legend(loc='upper center', fontsize=45, ncol=2)
    plt.savefig(os.path.join(output_dir, 'Figure4A_{}.png'.format(each_source.upper())), bbox_inches='tight')

    plt.show()

# Figure 4b

In [ ]:
# Generate Figure 4b

cmap = plt.cm.inferno
depths_of_interest = [10, 60, 175, 290, 500, 1000, 1870, 3000]
# extract all colors from the .viridis map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_depths = len(depths_of_interest)
depth_colors = cmap_array[0::int(cmap.N/num_depths)]
from matplotlib.lines import Line2D
for each_source in ['sim']:
    color_cycler = cycle(depth_colors)
    plt.figure(figsize=[20,20])
    plt.xlabel('Wavelength (nm)', size=40)
    plt.title('{} Total Effective Fluence\n'.format(each_source.upper()), size=70)
    ax1 = plt.gca()
    ax1.set_ylim([0.00001,1])
    ax1.tick_params(labelsize=45)
    ax1.set_ylabel('Effective Fluence Rate (mW$_{eff.} cm^{-2})$', size=40)
    ax1.set_yscale('log')
    
    for each_depth in depths_of_interest:
        index = int(each_depth/10)
        cur_color = next(color_cycler)
        eff_spectrum_at_depth = all_sources[each_source]['ppix_weighted'][index][::-1]
        wavelengths = all_sources[each_source]['wavelengths']
        
        f_cubic_sim = interp1d(wavelengths, eff_spectrum_at_depth, kind='cubic')
        xx = numpy.linspace(wavelengths.min(),wavelengths.max(),550)
        #spectrum_at_depth = slice_fluence_total[each_source][stop_index]
        #eff_spectrum_at_depth = slice_eff_fluence_total[each_source][stop_index]
        ax1.plot(xx, f_cubic_sim(xx),
                 color=cur_color , lw=8, alpha=0.7,
                 label='{}um'.format(each_depth))
    
    ax1.grid(True)
    ax1.set_xlim([350,650])
    ax1.grid(True, which='minor', axis='y', ls=':')
    ax1_legend = ax1.legend(loc='upper center', fontsize=45, ncol=2)
    plt.savefig(os.path.join(output_dir, 'Figure4B_{}.png'.format(each_source.upper())), bbox_inches='tight')

    plt.show()

In [ ]:
# Generate 3D plot of PpIX-effective fluence rate
color_map = plt.get_cmap('inferno')
im_shape = numpy.shape(depth_ppix_corrected_spectrum)
ny = im_shape[0]
nx = im_shape[1]
y = range(ny)
x = range(nx)
X, Y = numpy.meshgrid(x, y)
fig = plt.figure(figsize=(30,20))
ax = plt.gca(projection='3d')
#)
ax.plot_wireframe(X,Y,depth_ppix_corrected_spectrum)
surf = ax.plot_surface(X,Y,depth_ppix_corrected_spectrum, cmap=color_map,norm=matplotlib.colors.LogNorm() )
ax.xaxis.set_ticks([0,10,20,30,40,50])
ax.xaxis.set_ticklabels(['900', '800', '700', '600', '500', '400'], fontsize=40)
ax.xaxis.set_label_text('\n\nWavelength (nm)', fontsize=35)
ax.yaxis.set_ticks([0, 100, 200, 300, 400, 500])
ax.yaxis.set_ticklabels(['0', '1.0', '2.0', '3.0', '4.0', '5.0'], fontsize=40)
ax.yaxis.set_label_text('\n\nDepth (mm)', fontsize=35)
#ax.zaxis.set_ticklabels([0,5,10,15,20], size=30)
#ax.zaxis.set_label_text('\n\n\nFluence rate$_{ eff.}$\n($\mu$W/cm$^2$)', fontsize=35)
ax.view_init(45, 45)
# Add a color bar which maps values to colors.
cb = fig.colorbar(surf, shrink=0.6, aspect=6)
cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=40)
plt.savefig(os.path.join(output_dir, 'fluence_ppix_surface_plot.png'),bbox_inches='tight' )
plt.show()

# Investigate depths

In [ ]:
# Organize fluence rate data over the first 5mm
slice_fluence_total = {}
slice_eff_fluence_total = {}
volumetric_fluence_total = {}
volumetric_eff_fluence_total = {}

for each_source in all_sources.keys():
    print(each_source)
    slice_fluence_total[each_source] = [] 
    slice_eff_fluence_total[each_source] = []
    volumetric_fluence_total[each_source] = []
    volumetric_eff_fluence_total[each_source] = []
    
    for cur_depth_um in numpy.linspace(10,5000,500):#[10, 60, 175, 290, 1000, 1870, 2000, 3000, 4000]:
        #cur_depth_um = 50
        fluence_color = 'firebrick'
        eff_fluence_color = 'indigo'

        ind = int(cur_depth_um/10-1)
        wavelengths = numpy.linspace(350,900,56)

        # "[::-1]" Reverses array
        cur_fluence_slice = all_sources[each_source]['spectrum'][ind,:][::-1]
        cur_fluence_vol = numpy.sum(all_sources[each_source]['spectrum'][0:ind,:],axis=0)[::-1]
        cur_volumetric_fluence = 1/(0.0001*cur_depth_um)*cur_fluence_vol

        slice_fluence_total[each_source].append(integrate_spectrum(cur_fluence_slice, wavelengths, 350, 700))
        volumetric_fluence_total[each_source].append(integrate_spectrum(cur_volumetric_fluence, wavelengths, 350, 700))

        cur_eff_fluence_slice = all_sources[each_source]['ppix_weighted'][ind,:][::-1]
        cur_eff_fluence_vol = numpy.sum(all_sources[each_source]['ppix_weighted'][0:ind,:],axis=0)[::-1]
        cur_volumetric_eff_fluence = 1/(0.0001*cur_depth_um)*cur_eff_fluence_vol
        
        slice_eff_fluence_total[each_source].append(integrate_spectrum(cur_eff_fluence_slice, wavelengths, 350, 700))
        volumetric_eff_fluence_total[each_source].append(integrate_spectrum(cur_volumetric_eff_fluence, wavelengths, 350, 700))
     
    
    slice_fluence_total[each_source] = numpy.array(slice_fluence_total[each_source])
    slice_eff_fluence_total[each_source] = numpy.array(slice_eff_fluence_total[each_source])
    volumetric_fluence_total[each_source] = numpy.array(volumetric_fluence_total[each_source])
    volumetric_eff_fluence_total[each_source] = numpy.array(volumetric_eff_fluence_total[each_source])





# Figure 5a

In [ ]:
# Generate Figure 5A

plt.figure(figsize=[20,20])
plt.xlabel('Depth (mm)', size=40)

ax1 = plt.gca()
ax1.set_ylim([0.005,500])
ax1.set_xlim([0,5])
ax1.tick_params(labelsize=45)
ax1.set_ylabel('Fluence Rate (mW cm$^{-2}$)', size=40)
#ax1.tick_params('y', colors=fluence_color)
ax1.set_yscale('log')
ax1.grid(True)
ax1.grid(True, which='minor', axis='y', ls=':')


for each_source in ordered_lights:
    cur_color = all_sources[each_source]['plot_color']
    line_style = all_sources[each_source]['line_style']
    display_name = all_sources[each_source]['display_name']

    ax1.plot(numpy.linspace(10,5000,500)/1000, slice_fluence_total[each_source], line_style,
             color=cur_color , lw=8, label=display_name, alpha =0.8)
    
plt.legend(loc='lower right', fontsize=45)
plt.savefig(os.path.join(output_dir, 'Figure5A.png'), bbox_inches='tight')
plt.show()

# Figure 5b

In [ ]:
# Generate Figure 5B

plt.figure(figsize=[20,20])
plt.xlabel('Depth (mm)', size=40)

ax1 = plt.gca()
ax1.set_ylim([0.005,500])
ax1.set_xlim([0,5])
ax1.tick_params(labelsize=45)
ax1.set_ylabel('Effective Fluence Rate (mW$_{eff.}$ cm$^{-2}$)', size=40)
ax1.set_yscale('log')
ax1.grid(True)
ax1.grid(True, which='minor', axis='y', ls=':')


for each_source in ordered_lights:
    cur_color = all_sources[each_source]['plot_color']
    line_style = all_sources[each_source]['line_style']
    display_name = all_sources[each_source]['display_name']

    ax1.plot(numpy.linspace(10,5000,500)/1000, slice_eff_fluence_total[each_source], line_style,
             color=cur_color , lw=8, label=display_name, alpha =0.8)

   
    
plt.legend(loc='upper right', fontsize=45)
plt.savefig(os.path.join(output_dir, 'Figure5B.png'), bbox_inches='tight')
plt.show()

In [ ]:
# Tabulate fluence rate at depths of interest
depths_of_interest = [50,100,500,1000,1500,2000,3000]

for each_source in all_sources.keys():
    print(each_source)
    for each_depth in depths_of_interest:
        stop_index = int(each_depth/10)
        total_fluence_so_far = numpy.sum(slice_fluence_total[each_source][0:stop_index])
        cur_fluence = slice_eff_fluence_total[each_source][stop_index]
        print('\t{}um: {:0.4f}'.format(each_depth, cur_fluence) )



# Figure 6

In [ ]:
# Generate Figure 6, change plot time in minutes
plot_times = [5, 15,45,90,120]
threshold = 0.699
for dose_time in plot_times:
    plt.figure(figsize=[30,30])
    plt.xlabel('Depth (mm)', size=55)
    plt.title('{} min. Total Eff. Fluence\n'.format(dose_time), size=70)

    ax1 = plt.gca()
    ax1.set_ylim([0.005,100])
    ax1.tick_params(labelsize=50)
    ax1.set_ylabel('Total Effective Fluence ($J_{eff.} cm^{-2}$)', size=55)
    ax1.set_yscale('log')
    ax1.axhline(threshold, lw=7, ls='-', c='k', zorder=101)

    ax1.set_xlim(0,5)
    # Shrink current axis's height by 10% on the bottom
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])

    depths_of_activation = {}
    for each_source in ordered_lights:
        depths_of_activation[each_source] = {}

        cur_color = all_sources[each_source]['plot_color']
        line_style = all_sources[each_source]['line_style']
        display_name = all_sources[each_source]['display_name']

        if (each_source=='red') or (each_source=='blue'):
            minutes = numpy.array([5,15,25,30,45,90, 120])
        else:
            minutes = numpy.array([15,30,45,60,90,120,150,180])

        for each_time in minutes:

            cur_seconds = each_time*60
            surface_ppix_irradiance = all_sources[each_source]['ppix_irradiance_0']
            surface_light_dose = (cur_seconds*surface_ppix_irradiance)/1000
            tissue_light_dose = (cur_seconds*slice_eff_fluence_total[each_source])/1000
            # Deepest location of activation, even if some more shallow areas are not activated
            depths_of_activation[each_source][each_time] = numpy.argwhere(tissue_light_dose>=threshold)[-1][0]
            if each_time==dose_time:
                ax1.plot(numpy.linspace(10,5000,500)/1000, tissue_light_dose,
                         color=cur_color , lw=10, ls=line_style, alpha=0.8, label=display_name)

    ax1.grid(True)
    ax1.grid(True, which='minor', axis='y', ls=':')
    ax1.fill_between(numpy.linspace(10,5000,500)/1000, 0, threshold, color='whitesmoke', alpha=0.5, zorder=100)
    # Put a legend below current axis
    ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=3, fontsize=55)
    plt.savefig(os.path.join(output_dir, 'Figure6_{}min.png'.format(dose_time)), bbox_inches='tight')

    plt.show()

In [ ]:
# Display depth to reach fluence threshold for different treatement times using each light
depths_of_activation

In [ ]:
# Generate data frame containing information on fluence values at depth
lookup_df = pandas.DataFrame()
threshold=0.699
depths_of_interest = [10,20,30,40,50,100,150,250,500,1000,1500,2000,2500,3000,3500]
n = 0
for each_source in all_sources.keys():
    print(each_source)
    for each_irradiance in all_sources[each_source]['chart_irradiance']:
        target_irradiance = each_irradiance
        orig_irradiance = all_sources[each_source]['irradiance_0']
        orig_ppix_irradiance = all_sources[each_source]['ppix_irradiance_0']
        scale_factor = target_irradiance/orig_irradiance
        new_irradiance = scale_factor* orig_irradiance
        new_ppix_irradiance = scale_factor*(orig_ppix_irradiance)
        tmp_dict = {
            'Source': each_source,
            'Irradiance (mW/cm^2)': new_irradiance,
            'Eff. Irradiance (mW_eff./cm^2)': '{:0.1f}'.format(new_ppix_irradiance)
            }
        for each_depth in depths_of_interest:
            index = int((each_depth/10)-1)
            time_sec = ((threshold*1000)/(scale_factor*slice_eff_fluence_total[each_source][index]))
            time_min = time_sec/60
            time_min_str = '{:0.1f}'.format(time_min)
            tmp_dict[each_depth] = time_min_str


        tmp_df = pandas.DataFrame(data=tmp_dict, index=[n])
        n+=1
        # Append current data to dataframe
        lookup_df = lookup_df.append(tmp_df)


sorted_lookup_df = lookup_df.sort_values(by=['Source', 'Irradiance (mW/cm^2)',]).drop_duplicates() 

In [ ]:
# Show example table from data table
sorted_lookup_df[sorted_lookup_df['Source']=='sun'][['Source', 'Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', 10,20,30,40,50,100,150,250,500,1500,2000,2500,3000,3500]]

In [ ]:
# Write data frame to Excel file
# blue_slice = sorted_lookup_df[sorted_lookup_df['Source']=='blue'][['Source', 'Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', 10,20,30,40,50,100,150,250,500,1000,1500,2000,2500,3000,3500]]
# cfl_slice = sorted_lookup_df[sorted_lookup_df['Source']=='cfl'][['Source', 'Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', 10,20,30,40,50,100,150,250,500,1000,1500,2000,2500,3000,3500]]
# halogen_slice = sorted_lookup_df[sorted_lookup_df['Source']=='halogen'][['Source', 'Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', 10,20,30,40,50,100,150,250,500,1000,1500,2000,2500,3000,3500]]
# sim_slice = sorted_lookup_df[sorted_lookup_df['Source']=='sim'][['Source', 'Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', 10,20,30,40,50,100,150,250,500,1000,1500,2000,2500,3000,3500]]
# sun_slice = sorted_lookup_df[sorted_lookup_df['Source']=='sun'][['Source', 'Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', 10,20,30,40,50,100,150,250,500,1000,1500,2000,2500,3000,3500]]
# red_slice = sorted_lookup_df[sorted_lookup_df['Source']=='red'][['Source', 'Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', 10,20,30,40,50,100,150,250,500,1000,1500,2000,2500,3000,3500]]

# writer = pandas.ExcelWriter(os.path.join(output_dir, 'Lookup_table_based_ONLY_on fluence_no_formatting.xlsx'))
# sim_slice.to_excel(writer, 'Uniform Simulation')
# sun_slice.to_excel(writer, 'Sun')
# cfl_slice.to_excel(writer, 'CFL')
# halogen_slice.to_excel(writer, 'Halogen')
# blue_slice.to_excel(writer, 'Blue LED')
# red_slice.to_excel(writer, 'Red LED')
# writer.save()

# PpIX build up and photobleaching

In [ ]:
depths_in_cm = numpy.linspace(10,5000,500)/10000
cmap = plt.cm.inferno
times_of_interest = [300,900,1800,3600,5400,7200,9000,10800]
# extract all colors from the .viridis map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_times = len(times_of_interest)
time_colors = cmap_array[0::int(cmap.N/num_times)]
color_cycler = cycle(time_colors)

fig = plt.figure(figsize=[10,10])
for t in times_of_interest:
    cur_color = next(color_cycler)
    z = numpy.linspace(0.001,0.5,500)
    pro_drug_conc = get_pro_drug_conc(z,t)
    plt.semilogy(z,pro_drug_conc, label='{:0.0f} min.'.format(t/60), lw=6, c=cur_color)
plt.legend(loc='best', fontsize=20)
plt.ylabel('Pro-drug concentration(cm$^{-3}$)', size=30)
plt.xlabel('Depth (cm)', size=30)
plt.tick_params(axis='both', which='major', labelsize=20)
plt.ylim([10**12, 10**17])
plt.xlim([0,0.5])
plt.grid(True)
plt.show()

In [ ]:
all_z = numpy.linspace(0.001,0.5,500)
num_z_points = len(all_z)
num_t_points = len(times_of_interest)
ppix_conc = numpy.zeros([num_t_points, num_z_points])
color_cycler = cycle(time_colors)
fig = plt.figure(figsize=[10,10])

j = 0
for each_t in times_of_interest:
    cur_color = next(color_cycler)
    k = 0
    for each_z in all_z:
        cur_result = quad(get_ppix_conc_with_mal, 0, each_t, args=(each_z,each_t))
        ppix_conc[j,k] = cur_result[0]
        k += 1
    ppix_conc_uM = get_umolar_from_particles_ppix(ppix_conc[j,:]) 
    plt.semilogy(all_z,ppix_conc_uM, label='{:0.0f} min.'.format(each_t/60), lw=6, c=cur_color)
    j += 1
plt.legend(loc='best', fontsize=20)
plt.ylabel('PpIX concentration($\mu$M)', size=30)
plt.xlabel('Depth (cm)', size=30)
plt.tick_params(axis='both', which='major', labelsize=20)
plt.grid(True)
plt.ylim([10**-3, 3*10**1])
plt.xlim([0,0.5])
plt.show()

In [ ]:
# Convert normalized PpIX absorption spectrum to
# Cross section function referenced in
# Campbell et al 2016
ppix_cross_section = 2.4*sampled_ppix*10**-14
display_depths = [0,50,100,150,200]

cmap = plt.cm.magma
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_depths = len(display_depths)
depth_colors = cmap_array[0::int(cmap.N/num_depths)]
color_cycler = cycle(depth_colors)


u_a_ppix = numpy.zeros([numpy.shape(ppix_conc)[0], numpy.shape(ppix_conc)[1],len(ppix_cross_section) ])
j = 0
fig = plt.figure(figsize=[12,8])
for each_t in times_of_interest:
    #cur_color = next(color_cycler)
    k = 0
    for each_z in all_z:
        # u_a in inverse mm (0.1 converts from cm^-1 to mm^-1)
        u_a_ppix[j,k,:,] = (ppix_conc[j,k]*ppix_cross_section)*0.1
       
        if each_t==1800:
            if k in display_depths:
                cur_color = next(color_cycler)
                plt.semilogy(wavelengths,u_a_ppix[j,k,:].clip(10**-5), lw=6, label='{:0.1f}mm'.format(k/100), color=cur_color)
        k += 1
    j += 1

plt.legend(loc='best', fontsize=20)
plt.title('$\mu_a$ of PpIX after 30 min. incubation', size=38)
plt.ylabel('$\mu_a$ (mm$^{-1}$)', size=30)
plt.xlabel('Wavelength (nm)', size=30)
plt.tick_params(axis='both', which='major', labelsize=20)
plt.grid(True)
plt.xlim([350, 640])
plt.show()

# Show wavelength-dependent photobleaching constant
photobleaching_rate = get_betas(wavelengths, u_a_ppix[0,0,:,])
fig = plt.figure(figsize=[12,8])
plt.plot(wavelengths, photobleaching_rate, lw=6, color='k')
plt.title(r'Photobleaching rate $\beta(\lambda)$ ', size=38)
plt.ylabel(r'$\beta$ (J $cm^{-2}$)', size=30)
plt.xlabel('Wavelength (nm)', size=30)
plt.tick_params(axis='both', which='major', labelsize=20)
plt.grid(True)
plt.xlim([350, 640])
plt.show()
plt.show()


# fig = plt.figure(figsize=[12,8])
# plt.plot(wavelengths, photobleaching_rate*sampled_ppix, lw=6, color='k')
# plt.title(r'Effective Photobleaching rate $\beta_{eff.}$ ', size=38)
# plt.ylabel(r'$\beta$ (J$_{eff}$ $cm^{-2}$)', size=30)
# plt.xlabel('Wavelength (nm)', size=30)
# plt.tick_params(axis='both', which='major', labelsize=20)
# plt.grid(True)
# plt.xlim([350, 640])
# plt.show()
# plt.show()

## Calculate PDD based on PpIX concentration
* Multiply light dose by PpIX concentration at specific times
* PDD: J uM /cm^2

In [ ]:
# Estimate threshold
#0.1 uM = 100nM
#0.01 uM = 10nM
#J_eff/cm^2 = 0.70
fluence_threshold = 0.699 #J/cm^2
ppix_conc_threshold = 0.05 #uM
pdd_threshold = fluence_threshold*ppix_conc_threshold

In [ ]:
# The following estimates PpIX concentration, assuming:
# - a 30min incubation
# - Using ppix-weighted fluence rate
# - constant photo bleaching of 0.65 J/cm^2
# - ALA/MAL remains present at surface and throughout skin for duration of treatment


beta = 0.65#14
all_times = numpy.linspace(1800,10800,11)
#all_times = numpy.linspace(10800,21600,7)
incubation_t = all_times[0]
initial_ppix_conc = numpy.zeros(len(all_z))

k = 0
for each_z in all_z:
    cur_result = quad(get_ppix_conc_with_mal, 0, incubation_t, args=(each_z,incubation_t))
    initial_ppix_conc[k] = cur_result[0]
    k += 1
    
cmap = plt.cm.inferno
# extract all colors from the .viridis map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_times = len(all_times)
time_colors = cmap_array[0::int(cmap.N/num_times)]
for each_source in slice_fluence_total.keys():
    color_cycler = cycle(time_colors)
    fig = plt.figure(figsize=[10,10])

    ppix_conc_w_photobleaching = numpy.zeros([len(all_times), len(all_z)])
    cur_time = all_times[0]

    t_index = 0
    while cur_time<numpy.nanmax(all_times):
        cur_color = next(color_cycler)
        if cur_time==all_times[0]:
            ppix_conc_w_photobleaching[0,:] = initial_ppix_conc

        else:
            # Find the elapsed time
            diff_time = cur_time - all_times[t_index-1]
            for z_index in range(len(all_z)):
                # PpIX photobleaching
                # Use the light does for the given depth
                # Reverse the array [::-1] so it is ordered from 350 to 900 nm
                cur_eff_fluence_rate = integrate_spectrum(all_sources[each_source]['ppix_weighted'][z_index,:][::-1], wavelengths, 350, 700)
                eff_light_dose = diff_time*cur_eff_fluence_rate/1000
                # Divide by the wavelength-dependent photo bleaching constant
                exponent_at_wavelengths = -eff_light_dose/beta
                reduction_factor = numpy.exp(exponent_at_wavelengths)
                # Apply scalar reduction factor to previous concentration
                ppix_conc_w_photobleaching[t_index,z_index] = ppix_conc_w_photobleaching[t_index-1,z_index]*reduction_factor
                # PpIX Production
                # Since it is assumes ALA/MAL is still in the skin,
                # PpIX production is also occuring
                # Here we find the amount that has been produced during
                # the previous time window
                prev_result = quad(get_ppix_conc_with_mal, 0, all_times[t_index-1], args=(all_z[z_index],all_times[t_index-1]))
                cur_result = quad(get_ppix_conc_with_mal, 0, cur_time, args=(all_z[z_index],cur_time))
                ppix_production = cur_result[0] - prev_result[0]
                # Add the PpIX production to the current concentration
                ppix_conc_w_photobleaching[t_index,z_index] = ppix_conc_w_photobleaching[t_index,z_index] + ppix_production
        ppix_conc_uM = get_umolar_from_particles_ppix(ppix_conc_w_photobleaching[t_index,:]) 
        plt.semilogy(all_z,ppix_conc_uM, label='{:0.0f} min.'.format((all_times[t_index]-all_times[0])/60), lw=6, c=cur_color)    
        t_index += 1
        cur_time = all_times[t_index]



    plt.legend(loc='best', fontsize=20)
    plt.title('{}'.format(each_source.title()), size=40)
    plt.ylabel('PpIX concentration ($\mu$M)', size=30)
    plt.xlabel('Depth (cm)', size=30)
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.grid(True)
    plt.ylim([10**-3, 3*10**1])
    plt.xlim([0,0.5])
    plt.savefig(os.path.join(output_dir,'PpIX_conc-{}_total_eff_fluence-beta_of_0.65-inf_MAL.png'.format(each_source)), bbox_tight='inches')
    plt.show()        

# Figure 7

In [ ]:
# The following estimates PpIX concentration, assuming:
# - a different incubation times
# - Using ppix-weighted fluence rate
# - constant photo bleaching of 0.65 J/cm^2
# - ALA/MAL remains present at surface and throughout skin for duration of treatment


display_time=7200  #120min  1800 # 30min

beta = 0.65
all_times = numpy.linspace(1800,10800,11)
#all_times = numpy.linspace(10800,21600,7)
incubation_t = all_times[0]
initial_ppix_conc = numpy.zeros(len(all_z))

k = 0
for each_z in all_z:
    cur_result = quad(get_ppix_conc_with_mal, 0, incubation_t, args=(each_z,incubation_t))
    initial_ppix_conc[k] = cur_result[0]
    k += 1

# Colormap for graph
cmap = plt.cm.inferno
# extract all colors from the map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_times = len(all_times)
time_colors = cmap_array[0::int(cmap.N/num_times)]

# MAL parameters
fig = plt.figure(figsize=[20,20])
m_t = get_pro_drug_conc(all_z, all_times[0])
for each_source in slice_fluence_total.keys():
    color_cycler = cycle(time_colors)
    

    ppix_conc_w_photobleaching = numpy.zeros([len(all_times), len(all_z)])
    cur_time = all_times[0]

    t_index = 0
    while cur_time<numpy.nanmax(all_times):
        cur_color = next(color_cycler)
        if cur_time==all_times[0]:
            ppix_conc_w_photobleaching[0,:] = initial_ppix_conc

        else:
            # Find the elapsed time
            diff_time = cur_time - all_times[t_index-1]
            # Calculate light dose by depth and 
            # for each wavelength
            
            for z_index in range(len(all_z)):
                # PpIX photobleaching
                # Use the light does for the given depth
                # Reverse the array so it is ordered from 350 to 900 nm
                cur_eff_fluence_rate = integrate_spectrum(all_sources[each_source]['ppix_weighted'][z_index,:][::-1], wavelengths, 350, 700)
                eff_light_dose = diff_time*cur_eff_fluence_rate/1000
                # Divide by the wavelength-dependent photo bleaching constant
                exponent_at_wavelengths = -eff_light_dose/beta
                reduction_factor = numpy.exp(exponent_at_wavelengths)
                
                # Apply scalar reduction factor to previous concentration
                ppix_conc_w_photobleaching[t_index,z_index] = ppix_conc_w_photobleaching[t_index-1,z_index]*reduction_factor
                # PpIX Production
                # Since it is assumes ALA/MAL is still in the skin,
                # PpIX production is also occuring
                # Here we find the amount that has been produced during
                # the previous time window
                prev_result = quad(get_ppix_conc_with_mal, 0, all_times[t_index-1], args=(all_z[z_index],all_times[t_index-1]))
                cur_result = quad(get_ppix_conc_with_mal, 0, cur_time, args=(all_z[z_index],cur_time))
                ppix_production = cur_result[0] - prev_result[0]
                #m_t[z_index] = m_t[z_index]-ppix_production
                #if z_index==10:
                #    print(m_t[z_index])
                # Add the PpIX production to the current concentration
                ppix_conc_w_photobleaching[t_index,z_index] = ppix_conc_w_photobleaching[t_index,z_index] + ppix_production
        ppix_conc_uM = get_umolar_from_particles_ppix(ppix_conc_w_photobleaching[t_index,:]) 
        if cur_time==(all_times[0]+display_time):
            cur_color = all_sources[each_source]['plot_color']
            line_style = all_sources[each_source]['line_style']
            display_name = all_sources[each_source]['display_name']
            plt.semilogy(all_z,ppix_conc_uM,
                         color=cur_color, 
                         lw=10, ls=line_style,
                         alpha=0.8, label=display_name)    
        t_index += 1
        cur_time = all_times[t_index]



plt.legend(loc='best', fontsize=40)
plt.title('{:0.0f} min. incubation, {:0.0f} min. treatment\n'.format(all_times[0]/60, display_time/60), size=50)
plt.ylabel('PpIX concentration ($\mu$M)', size=40)
plt.xlim([0,0.5])
ax = plt.gca()
xtick_locs = ax.get_xticks()
ax.set_xticklabels([str(int(label)) for label in xtick_locs*10], fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=30)
plt.xlabel('Depth (mm)', size=35)
plt.tick_params(axis='both', which='major', labelsize=30)
plt.grid(True)
plt.ylim([10**-2, 10**1])
plt.grid(True, which='minor', axis='y', ls=':')    
plt.fill_between(numpy.linspace(0,5000,500)/1000, 0, ppix_conc_threshold, color='whitesmoke', alpha=0.5, zorder=100)
plt.axhline(ppix_conc_threshold, ls='-', c='k', lw=7, zorder=101)
plt.savefig(os.path.join(output_dir,'PpIX_conc-ALL_total_eff_fluence-beta_of_0.65-inf_MAL_{:0.0f}min_treatment.png'.format(display_time/60)), bbox_tight='inches')
plt.show()        
        

In [ ]:
# The following estimates Photodynamic dose, assuming:
# - a 30min incubation
# - Using ppix-weighted fluence rate
# - constant photo bleaching of 0.65 J/cm^2
# - ALA/MAL remains present at surface and throughout skin for duration of treatment


beta = 0.65#14
all_times = numpy.linspace(1800,10800,11)
incubation_t = all_times[0]

k = 0
for each_z in all_z:
    cur_result = quad(get_ppix_conc_with_mal, 0, incubation_t, args=(each_z,incubation_t))
    initial_ppix_conc[k] = cur_result[0]
    k += 1
    
cmap = plt.cm.inferno
# extract all colors from the .viridis map
cmap_array = numpy.array([cmap(i) for i in range(cmap.N)])
num_times = len(all_times)
time_colors = cmap_array[0::int(cmap.N/num_times)]
for each_source in slice_fluence_total.keys():
    color_cycler = cycle(time_colors)
    fig = plt.figure(figsize=[10,10])

    ppix_conc_w_photobleaching = numpy.zeros([len(all_times), len(all_z)])
    photodynamic_dose = numpy.zeros([len(all_times), len(all_z)])
    ppix_conc_uM = numpy.zeros([len(all_times), len(all_z)])
    cur_time = all_times[0]

    t_index = 0
    while cur_time<numpy.nanmax(all_times):
        cur_color = next(color_cycler)
        if cur_time==all_times[0]:
            ppix_conc_w_photobleaching[0,:] = initial_ppix_conc

        else:
            # Find the elapsed time
            diff_time = cur_time - all_times[t_index-1]
            for z_index in range(len(all_z)):
                # PpIX photobleaching
                # Use the light does for the given depth
                # Reverse the array [::-1] so it is ordered from 350 to 900 nm
                cur_eff_fluence_rate = integrate_spectrum(all_sources[each_source]['ppix_weighted'][z_index,:][::-1], wavelengths, 350, 700)
                eff_light_dose = diff_time*cur_eff_fluence_rate/1000
                # Divide by the wavelength-dependent photo bleaching constant
                exponent_at_wavelengths = -eff_light_dose/beta
                reduction_factor = numpy.exp(exponent_at_wavelengths)
                # Apply scalar reduction factor to previous concentration
                ppix_conc_w_photobleaching[t_index,z_index] = ppix_conc_w_photobleaching[t_index-1,z_index]*reduction_factor
                # PpIX Production
                # Since it is assumes ALA/MAL is still in the skin,
                # PpIX production is also occuring
                # Here we find the amount that has been produced during
                # the previous time window
                prev_result = quad(get_ppix_conc_with_mal, 0, all_times[t_index-1], args=(all_z[z_index],all_times[t_index-1]))
                cur_result = quad(get_ppix_conc_with_mal, 0, cur_time, args=(all_z[z_index],cur_time))
                ppix_production = cur_result[0] - prev_result[0]
                # Add the PpIX production to the current concentration
                ppix_conc_w_photobleaching[t_index,z_index] = ppix_conc_w_photobleaching[t_index,z_index] + ppix_production
                ppix_conc_uM[t_index,z_index] = get_umolar_from_particles_ppix(ppix_conc_w_photobleaching[t_index,z_index]) 
                photodynamic_dose[t_index,z_index] = eff_light_dose*ppix_conc_uM[t_index,z_index]

        #plt.plot(all_z,light_dose*ppix_conc_uM, lw=6, c=cur_color, label='{} min.'.format((cur_time-all_times[0])/60))
        if t_index>0:
            plt.semilogy(all_z,photodynamic_dose[t_index,:], label='{:0.0f} min.'.format((all_times[t_index]-all_times[0])/60), lw=6, c=cur_color)    
        t_index += 1
        cur_time = all_times[t_index]



    plt.legend(loc='best', fontsize=20)
    plt.title('{}'.format(each_source.title()), size=40)
    plt.ylabel('Photodynamic Dose ($\mu$M x J$_{eff}$ cm$^{-2}$)', size=30)
    plt.xlabel('Depth (cm)', size=30)
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.grid(True)
    plt.axhline(pdd_threshold, ls='--')
    plt.ylim([10**-3, 10**2])
    plt.xlim([0,0.5])
    plt.savefig(os.path.join(output_dir,'PDD-{}_total_eff_fluence-beta_of_0.65-inf_MAL.png'.format(each_source)), bbox_tight='inches')
    plt.show()        
        
        
        
    


## PDD(z,t)
* The following takes a long time
* It has been reduced to look at fewer lights, and less time points than what is discussed in the paper

In [ ]:
# Generate data frame containing information for S4
#
# Inputs:
# - Light source spectrum
# - Surface irradiance
# - Incubation time (30min)
# - Depth of interest
#
# Output:
# - Treatment time to reach PDD threshold



# Define tissue depths
all_z_cm = numpy.linspace(0.001,0.5,500)


# Define incubation time 
# ** Reduced to 2 to save time **
all_incubation_times_s = [1800, 7200]#[300,1800,3600,7200]
# Initialize empty dictionary
photodynamic_dose_all = {}

for incubation_time_s in all_incubation_times_s:
    #time_by_10s = numpy.linspace(incubation_time_s,incubation_time_s+12600,22)
    # ** Reduced to 2 to save time **
    time_by_10s = numpy.linspace(incubation_time_s,incubation_time_s+7200,5)
    all_times = numpy.zeros(len(time_by_10s)+2)
    all_times[3:] = time_by_10s[1:]
    # Add 0min, 1min, 5 min before initial 10 minutes
    all_times[0:3] = [incubation_time_s, incubation_time_s+60, incubation_time_s+300]

    # Determine PpIX distribution after initial incubation
    k = 0
    for each_z in all_z_cm:
        cur_result = quad(get_ppix_conc_with_mal, 0, incubation_time_s, args=(each_z,incubation_time_s))
        initial_ppix_conc[k] = cur_result[0]
        k += 1
    photodynamic_dose_all[incubation_time_s] = {}

    

    # Loop trhough each light source 
    # ** Reduced to 3 to save time **
    for each_source in ['red','blue','sun']:#all_sources.keys():
        print(incubation_time_s, each_source)
        photodynamic_dose_all[incubation_time_s][each_source] = {}
        # Calculate PDD for different surface irradiance values of the current light source
        for each_irradiance in all_sources[each_source]['chart_irradiance']:
            # Determine scaling needed for given surface irradiance
            target_irradiance = each_irradiance
            orig_irradiance = all_sources[each_source]['irradiance_0']
            orig_ppix_irradiance = all_sources[each_source]['ppix_irradiance_0']
            scale_factor = target_irradiance/orig_irradiance
            new_irradiance = scale_factor*orig_irradiance
            new_ppix_irradiance = scale_factor*(orig_ppix_irradiance)

            # Define empty arrays to calculate PDD for different depths and times
            ppix_conc_w_photobleaching = numpy.zeros([len(all_times), len(all_z)])
            photodynamic_dose_all[incubation_time_s][each_source][each_irradiance] = numpy.zeros([len(all_times), len(all_z_cm)])
            ppix_conc_uM = numpy.zeros([len(all_times), len(all_z_cm)])

            # Set starting time
            cur_time = all_times[0]
            t_index = 0
            while cur_time<numpy.nanmax(all_times):
                cur_color = next(color_cycler)
                # If starting time, make PpIX the initial concentration
                if cur_time==all_times[0]:
                    ppix_conc_w_photobleaching[0,:] = initial_ppix_conc

                else:
                    # Find the elapsed time
                    diff_time = cur_time - all_times[t_index-1]
                    for z_index in range(len(all_z)):
                        # PpIX photobleaching
                        # Use the light does for the given depth
                        # Reverse the array [::-1] so it is ordered from 350 to 900 nm
                        cur_scaled_eff_fluence_rate = integrate_spectrum(scale_factor*all_sources[each_source]['ppix_weighted'][z_index,:][::-1], wavelengths, 350, 700)
                        scaled_eff_light_dose = diff_time*cur_scaled_eff_fluence_rate/1000
                        # Divide by the wavelength-dependent photo bleaching constant
                        exponent_at_wavelengths = -scaled_eff_light_dose/beta
                        reduction_factor = numpy.exp(exponent_at_wavelengths)
                        # Apply scalar reduction factor to previous concentration
                        ppix_conc_w_photobleaching[t_index,z_index] = ppix_conc_w_photobleaching[t_index-1,z_index]*reduction_factor
                        # PpIX Production
                        # ALA/MAL is assumed to be removed from surface, so
                        # any PpIX that is produced is subtracted from the previous MAL/ALA concentration
                        # Here we find the amount that has been produced during
                        # the previous time window
                        prev_result = quad(get_ppix_conc_with_mal, 0, all_times[t_index-1], args=(all_z_cm[z_index],all_times[t_index-1]))
                        cur_result = quad(get_ppix_conc_with_mal, 0, cur_time, args=(all_z_cm[z_index],cur_time))
                        ppix_production = cur_result[0] - prev_result[0]
                        #m_t[z_index] = m_t[z_index]-ppix_production
                        # Add the PpIX production to the current concentration
                        ppix_conc_w_photobleaching[t_index,z_index] = ppix_conc_w_photobleaching[t_index,z_index] + ppix_production
                        ppix_conc_uM[t_index,z_index] = get_umolar_from_particles_ppix(ppix_conc_w_photobleaching[t_index,z_index]) 
                        photodynamic_dose_all[incubation_time_s][each_source][each_irradiance][t_index,z_index] = scaled_eff_light_dose*ppix_conc_uM[t_index,z_index]




                t_index += 1
                cur_time = all_times[t_index]
                    

In [ ]:
# Define threshold dose
fluence_threshold = 0.699 #J/cm^2
ppix_conc_threshold = 0.05 #uM
pdd_threshold = fluence_threshold*ppix_conc_threshold


depths_of_interest_um = numpy.array([100,250,500,750,1000,1500,2000,2500,3000])
depths_of_interest_cm = depths_of_interest_um/10000

# Initialize empty data frame
pdd_lookup_df = pandas.DataFrame()
n = 0

for incubation_time_s in all_incubation_times_s:
    #time_by_10s = numpy.linspace(incubation_time_s,incubation_time_s+12600,22)
    # ** Reduced to 2 to save time **
    time_by_10s = numpy.linspace(incubation_time_s,incubation_time_s+7200,5)
    all_times = numpy.zeros(len(time_by_10s)+2)
    all_times[3:] = time_by_10s[1:]
    # Add 0min, 1min, 5 min before initial 10 minutes
    all_times[0:3] = [incubation_time_s, incubation_time_s+60, incubation_time_s+300]
    # Loop trhough each light source
    # ** Reduced to 3 to save time **
    for each_source in ['red','blue','sun']:#all_sources.keys():
        print(each_source)
        # Calculate PDD for different surface irradiance values of the current light source
        for each_irradiance in all_sources[each_source]['chart_irradiance']:
            # Determine scaling needed for given surface irradiance
            target_irradiance = each_irradiance
            orig_irradiance = all_sources[each_source]['irradiance_0']
            orig_ppix_irradiance = all_sources[each_source]['ppix_irradiance_0']
            scale_factor = target_irradiance/orig_irradiance
            new_irradiance = scale_factor*orig_irradiance
            new_ppix_irradiance = scale_factor*(orig_ppix_irradiance)

            tmp_dict = {
                            'Source': each_source,
                            'Irradiance (mW/cm^2)': new_irradiance,
                            'Eff. Irradiance (mW_eff./cm^2)': '{:0.1f}'.format(new_ppix_irradiance),
                            'Incubation time (min.)': incubation_time_s/60
                            }
            for each_depth in depths_of_interest_cm:
                index = numpy.where(numpy.isclose(all_z_cm,each_depth))[0][0]
                any_greater_than_pdd_threshold = any(photodynamic_dose_all[incubation_time_s][each_source][each_irradiance][:-1,index]>pdd_threshold)
                if any_greater_than_pdd_threshold:
                    index_when_greater =numpy.argmax(photodynamic_dose_all[incubation_time_s][each_source][each_irradiance][:-1,index]>pdd_threshold)
                    time_when_greater = all_times[index_when_greater]
                    time_before_threshold = all_times[index_when_greater-1]
                    delta_time = time_when_greater - time_before_threshold
                    pdd_when_greater = photodynamic_dose_all[incubation_time_s][each_source][each_irradiance][:-1,index][index_when_greater]
                    pdd_before_threshold = photodynamic_dose_all[incubation_time_s][each_source][each_irradiance][:-1,index][index_when_greater-1]
                    delta_pdd = pdd_when_greater-pdd_before_threshold
                    pdd_slope = delta_pdd/delta_time
                    time_crossing_threshold = (pdd_threshold - pdd_before_threshold)/pdd_slope + time_before_threshold
                    time_crossing_threshold = time_crossing_threshold-incubation_time_s

                    time_min_crossing_threshold = time_crossing_threshold/60
                    time_min_str = '{:0.1f}'.format(time_min_crossing_threshold)
                else:
                    max_time_min = (all_times[-1] - incubation_time_s)/60
                    time_min_str = '>{:0.1f}'.format(max_time_min)
                tmp_dict['{:0.1f}'.format(each_depth*10000)] = time_min_str



            tmp_df = pandas.DataFrame(data=tmp_dict, index=[n])
            n+=1
            # Append current data to dataframe
            pdd_lookup_df = pdd_lookup_df.append(tmp_df)


sorted_pdd_lookup_df = pdd_lookup_df.sort_values(by=['Source', 'Irradiance (mW/cm^2)', 'Incubation time (min.)']).drop_duplicates() 

In [ ]:
# Write data frame to Excel file
blue_pdd_slice = sorted_pdd_lookup_df[sorted_pdd_lookup_df['Source']=='blue'][['Source', 'Incubation time (min.)', 'Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', '100.0','250.0','500.0','750.0','1000.0','1500.0','2000.0','2500.0','3000.0']]
cfl_pdd_slice = sorted_pdd_lookup_df[sorted_pdd_lookup_df['Source']=='cfl'][['Source', 'Incubation time (min.)','Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', '100.0','250.0','500.0','750.0','1000.0','1500.0','2000.0','2500.0','3000.0']]
halogen_pdd_slice = sorted_pdd_lookup_df[sorted_pdd_lookup_df['Source']=='halogen'][['Source', 'Incubation time (min.)','Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', '100.0','250.0','500.0','750.0','1000.0','1500.0','2000.0','2500.0','3000.0']]
sim_pdd_slice = sorted_pdd_lookup_df[sorted_pdd_lookup_df['Source']=='sim'][['Source', 'Incubation time (min.)','Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', '100.0','250.0','500.0','750.0','1000.0','1500.0','2000.0','2500.0','3000.0']]
sun_pdd_slice = sorted_pdd_lookup_df[sorted_pdd_lookup_df['Source']=='sun'][['Source', 'Incubation time (min.)','Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', '100.0','250.0','500.0','750.0','1000.0','1500.0','2000.0','2500.0','3000.0']]
red_pdd_slice = sorted_pdd_lookup_df[sorted_pdd_lookup_df['Source']=='red'][['Source', 'Incubation time (min.)','Irradiance (mW/cm^2)','Eff. Irradiance (mW_eff./cm^2)', '100.0','250.0','500.0','750.0','1000.0','1500.0','2000.0','2500.0','3000.0']]

writer = pandas.ExcelWriter(os.path.join(output_dir, 'S4_pdd_lookup_table_multi_incubation_no_formatting.xlsx'.format(incubation_time_s/60)))
sim_pdd_slice.to_excel(writer, 'Uniform Simulation')
sun_pdd_slice.to_excel(writer, 'Sun')
cfl_pdd_slice.to_excel(writer, 'CFL')
halogen_pdd_slice.to_excel(writer, 'Halogen')
blue_pdd_slice.to_excel(writer, 'Blue LED')
red_pdd_slice.to_excel(writer, 'Red LED')
writer.save()

# Figure 8
* Plot Photodynamic Dose curves

In [ ]:
treatment_times_of_interest = [30,60,90]
for each_treatment_time in treatment_times_of_interest:
    for incubation_time_s in all_incubation_times_s:
        #time_by_10s = numpy.linspace(incubation_time_s,incubation_time_s+12600,22)
        # ** Reduced to 2 to save time **
        time_by_10s = numpy.linspace(incubation_time_s,incubation_time_s+7200,5)
        all_times = numpy.zeros(len(time_by_10s)+2)
        all_times[3:] = time_by_10s[1:]
        # Add 0min, 1min, 5 min before initial 10 minutes
        all_times[0:3] = [incubation_time_s, incubation_time_s+60, incubation_time_s+300]
        treatment_times_min = (all_times-incubation_time_s)/60
        t_index = numpy.where(numpy.isclose(each_treatment_time,treatment_times_min))[0][0]
        plt.figure(figsize=[20,20])
        # ** Reduced to 3 to save time **
        for each_source in ['red','blue','sun']:#ordered_lights:
            cur_color = all_sources[each_source]['plot_color']
            line_style = all_sources[each_source]['line_style']
            display_name = all_sources[each_source]['display_name']
            if each_source in ['blue', 'red']:
                each_irradiance = 100
            else:
                each_irradiance = 35
            plt.semilogy(all_z_cm,photodynamic_dose_all[incubation_time_s][each_source][each_irradiance][t_index,:],
                     color=cur_color , lw=10, ls=line_style, alpha=0.8, label=display_name)
        #plt.legend(loc='best', fontsize=40)
        plt.title('{:0.0f} min. incubation, {:0.0f} min. treatment\n'.format(incubation_time_s/60, each_treatment_time), size=40)
        plt.ylabel('Photodynamic Dose ($\mu$M x J$_{eff}$ cm$^{-2}$)', size=35)
        plt.xlim([0,0.5])
        ax = plt.gca()
        xtick_locs = ax.get_xticks()
        ax.set_xticklabels([str(int(label)) for label in xtick_locs*10], fontsize=18)
        plt.tick_params(axis='both', which='major', labelsize=30)
        plt.xlabel('Depth (mm)', size=35)
        
        plt.grid(True)
        plt.grid(True, which='minor', axis='y', ls=':')
        

    
        plt.fill_between(numpy.linspace(0,5000,500)/1000, 0, pdd_threshold, color='whitesmoke', alpha=0.5, zorder=100)
        plt.axhline(pdd_threshold, ls='-', c='k', lw=7, zorder=101)
        # Put a legend below current axis
        ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=3, fontsize=45)
        plt.ylim([10**-3, 10**2])
        
        plt.savefig(os.path.join(output_dir, 'Figure8_PDD_{:0.0f}incubation_{:0.0f}treatment.png'.format(incubation_time_s/60, each_treatment_time)))
        plt.show()

In [ ]:
# Determine the depths of activation for Table 2
cur_incubation_time = 1800
depths_of_pdd_threshold = {}
for each_source in ['red','blue','sun']:#ordered_lights:
    depths_of_pdd_threshold[each_source] = {}
    for each_treatment_time in [30,60,90]:
        #time_by_10s = numpy.linspace(cur_incubation_time,cur_incubation_time+12600,22)
        # ** Reduced to 2 to save time **
        time_by_10s = numpy.linspace(cur_incubation_time,cur_incubation_time+7200,5)
        all_times = numpy.zeros(len(time_by_10s)+2)
        all_times[3:] = time_by_10s[1:]
        # # Add 0min, 1min, 5 min before initial 10 minutes
        all_times[0:3] = [cur_incubation_time, cur_incubation_time+60, cur_incubation_time+300]
        treatment_times_min = (all_times-cur_incubation_time)/60
        t_index = numpy.where(numpy.isclose(each_treatment_time,treatment_times_min))[0][0]
        if each_source in ['blue', 'red']:
            irradiance = 100
        else:
            irradiance = 35
        z_index = numpy.argmax(numpy.where(photodynamic_dose_all[cur_incubation_time][each_source][irradiance][t_index,:]>pdd_threshold))
        depths_of_pdd_threshold[each_source][each_treatment_time] = z_index

In [ ]:
# time(min): depth (index)
# Convert index to um by adding 1 and multiplying by 10
depths_of_pdd_threshold

# Light fluence threshold
* Based on Faria paper

In [ ]:
# Input data from Faria paper
fwhm_to_sigma = 2*numpy.sqrt(2*numpy.log(2))
wv_center = numpy.array([630, 514, 514, 405, 405, 634])
wv_spread = numpy.array([20, 10, 10, 20, 20, 20])
wv_sigma = wv_spread/fwhm_to_sigma
d_p = numpy.array([0.79, 0.16, 1.11, 0.14, 0.14, 1.54])
delta_d = numpy.array([0.64, 0.22, 0.56, 0.31, 0.31, 3.51]) 

d_sigma = delta_d/fwhm_to_sigma

In [ ]:
# Display simulated distribution of light sources referenced in paper
plt.figure(figsize=[10,5])
plt.xlim([0.01,5])
plt.xscale('log')
plt.xlabel('Threshold (J cm$^{-2}$)', size=30)
plt.ylabel('Normalized intensity (a.u.)')

for i in range(len(d_p)):
    mu = d_p[i]
    sigma = d_sigma[i]
    xx = numpy.linspace(mu - 3*sigma, mu + 3*sigma, 1000).clip(0.001)
    d_pdf = gaussian(xx, mu, sigma)
    plt.plot(xx,d_pdf)
plt.show()

In [ ]:
# Re-sample PpIX absorption spectrum
ppix_wavelengths = ppix_abs['wavelength'].values
sample_points = numpy.linspace(350,900,56)
sampled_ppix = []
for each_point in sample_points:
    if each_point in ppix_wavelengths:
        sampled_ppix.append(ppix_abs[ppix_abs['wavelength']==each_point]['spectrum'].values[0])
        
    else:
        sampled_ppix.append(0)

sampled_ppix = numpy.array(sampled_ppix).clip(0)


In [ ]:
# Find effective PpIX threshold for each light source
initial_j =[]
eff_j = []

plt.figure(figsize=[10,10])
plt.xlabel('Wavelength (nm)', size=30)

for i in range(len(d_p)):
    mu = wv_center[i]
    sigma = wv_sigma[i]
    xx = numpy.linspace(mu - 3*sigma, mu + 3*sigma, 100)
    light_spectrum = gaussian(xx, mu, sigma)
    initial_j.append(integrate_spectrum(light_spectrum, xx, 350,700))
    plt.plot(xx,light_spectrum)
    ppix_slice = ppix_abs[(ppix_abs['wavelength']>=numpy.min(numpy.floor(xx))) & (ppix_abs['wavelength']<=numpy.max(numpy.ceil(xx)))]
    f_ppix_slice = interp1d(ppix_slice['wavelength'], ppix_slice['spectrum'].values, kind='cubic')
    eff_j.append(integrate_spectrum(f_ppix_slice(xx)*light_spectrum, xx, 350,700))
    plt.plot(xx, f_ppix_slice(xx)*light_spectrum)
plt.show()

In [ ]:
# Center wavelengths
wv_center

In [ ]:
# Scaling factor
d_scaling = numpy.array(eff_j)/numpy.array(initial_j)

In [ ]:
# Effective standard deviation
d_sigma*d_scaling

In [ ]:
# Effective peak fluence
d_scaling*d_p

In [ ]:
# Standard deviation
d_sigma

In [ ]:
# Peak of threshold probability
d_p

In [ ]:
# FInal effective threshold fluence
eff_j

In [ ]:
# Display effective theshold fluence distributions
d_p_scaled = d_scaling*d_p
d_sigma_scaled = d_scaling*d_sigma
d_pdf_scaled = []
xx = []

plt.figure(figsize=[10,5])
plt.xlim([0.001,0.25])
#plt.xscale('log')
plt.xlabel('Threshold (J$_{eff.}$ cm$^{-2}$)', size=30)

for i in range(len(d_p)):
    mu = d_p_scaled[i]
    sigma = d_sigma_scaled[i]
    xx.append(numpy.linspace(mu - 3*sigma, mu + 3*sigma, 1000).clip(0.001))
    d_pdf_scaled.append(gaussian(xx[i], mu, sigma))
    plt.plot(xx[i],d_pdf_scaled[i])
plt.show()

In [ ]:
# Pooled standard deviation
pooled_std_scaled = numpy.sqrt(numpy.sum(d_sigma_scaled)/len(d_p_scaled))

In [ ]:
pooled_std_scaled

In [ ]:
# Effective threshold to use in paper
numpy.mean(d_p_scaled) + 3 * pooled_std_scaled

# Monte Carlo run times
* 10$^7$ photons, 25mm radius source
* *AWS Batch* where each job had
  * vcpus: 2
  * memory 2000

In [ ]:
aws_total_run_time = 107.53 #min
aws_run_time_min = [
    26.21854266, 21.07641444, 18.64554503, 15.472993980000002, 13.278155100000001, 12.576264750000002, 12.22736165, 12.03727364, 
    11.666799560000001, 11.51035161, 14.410981620000001, 18.97464417, 22.7110413, 26.24033035, 23.32814803, 22.152112870000003, 
    24.027737920000003, 18.704423470000002, 16.21239183, 15.100969590000002, 13.470176830000002, 14.09420161, 14.545325150000002, 
    14.52265395, 19.161331500000003, 27.01653556, 40.251231970000006, 47.09141640000001, 52.423282560000004, 57.776102910000006, 
    57.168047990000005, 58.39375975000001, 63.212423290000004, 68.75624850000001, 76.93460051000001, 76.45525466000001, 82.3244616, 
    86.29228834000001, 85.21262245000001, 88.45850483000001, 81.46305602000001, 76.79282216, 83.67348135, 90.0718441, 95.31060831, 
    97.10390023000001, 101.85368333000001, 107.37737118000001, 98.25499707, 97.06517582000001, 96.63840715, 94.18131583, 
    95.17163052000001, 92.23596017000001, 90.15879482000001, 88.52110068]

aws_wavelengths = [
    350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490,
    500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640,
    650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790,
    800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900]
total_serial_time = numpy.sum(aws_run_time_min)
print('AWS time (3 c4.8x, 1 c4.2x): {:0.1f} hours'.format(aws_total_run_time/60))
print( 'Total sequential time: {:0.1f} hours'.format(total_serial_time/60))

In [ ]:
plt.figure(figsize=[20,20])
plt.xlabel('Wavelength (nm)', size=40)
ax1 = plt.gca()
#ax1.set_ylim([0.00001,1])
ax1.tick_params(labelsize=45)
ax1.set_ylabel('AWS job run time (min.)', size=40)
plt.grid(True)
plt.title('10$^7$ photons, 25mm radius source\nAWS time (3 c4.8x, 1 c4.2x): {:0.1f} hours'.format(aws_total_run_time/60), size=50)
plt.plot(aws_wavelengths, aws_run_time_min, lw=8)
plt.savefig(os.path.join(output_dir, 'AWS_runtime_10million_events_per_wavelength.png'), bbox_inches='tight' )
plt.show()

# Environment Information

In [ ]:
"""
Show system information 
(Platform, Python Version, available pakcages)
where platforms are mapped with the following:
  * 'linux2': Linux (2.x and 3.x): 
  * 'win32': Windows
  * 'cygwin': Windows/Cygwin
  * 'darwin': Mac OS X
"""
# Package imports
import sys
import pip

    
available_packages = pip.get_installed_distributions()
sorted_available_packages = sorted(["{}=={}".format(p.key, p.version) 
                                    for p in available_packages])

print("Platform: {}\nPython Version: {}".format(sys.platform, 
                                                sys.version.split("\n")[0]))
print("Python Packages: \n\t{}".format("\n\t".join(sorted_available_packages)))